In [ ]:
# Helper classes for testing #
##############################

# Imports #
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.patches import Rectangle 
import math

# UNITS USED
# Time: Astronomical Units (AU). 1AU = distance between Sun and earth
# Mass: Solar Mass (M_sun). 1M_sun = mass of the Sun
# Time: Year (yr). 1yr = 1 year (one rotation of the Earth around the Sun)
# Luminosity: Solar Luminosity (L_sun). 1L_sun = luminosity of the Sun
# Velocity: AU/yr

G = 4*(math.pi)**2  # Universal gravitational constant (AU^3/M_sun/yr^2)
scoeff = 22.62      # Scorching coefficient, approx. 0.745 AU from the Sun (L_sun/AU^2)
fcoeff = 3.14       # Freezing coefficient, approx. 2 AU from the Sun (L_sun/AU^2)

###############################
# Common Classes
###############################
class Box:
    def __init__(self,x0,y0,x1,y1):
        self.x0, self.y0, self.x1, self.y1 = x0, y0, x1, y1
        self.maxSide = max(self.x1-self.x0,self.y1-self.y0)
        self.mx = (self.x0+self.x1)/2
        self.my = (self.y0+self.y1)/2
        
    def isIn(self,p):
        return self.x0 <= p.x <= self.x1 and self.y0 <= p.y <= self.y1
    
    def asTuple(self):
        return (self.x0,self.y0,self.x1,self.y1,self.mx,self.my)
    
    def split4(self):
        bNE = Box(self.mx,self.my,self.x1,self.y1)
        bNW = Box(self.x0,self.my,self.mx,self.y1)
        bSW = Box(self.x0,self.y0,self.mx,self.my)
        bSE = Box(self.mx,self.y0,self.x1,self.my)
        return (bNE,bNW,bSW,bSE)

    # Build a box that encloses all Bodies from an array 
    @staticmethod
    def getBox(P):
        if len(P) == 0: return None
        x0 = x1 = P[0].x
        y0 = y1 = P[0].y
        for p in P: 
            if p.x < x0: x0 = p.x
            if p.y < y0: y0 = p.y
            if p.x > x1: x1 = p.x
            if p.y > y1: y1 = p.y
        return Box(x0,y0,x1,y1)    
    
    def __str__(self):
        return f"Box({self.x0},{self.y0},{self.x1},{self.y1})"
        
class GNode:
    def __init__(self,box):
        self.box = box
        self.COM = None           # this node’s COM
        self.nbodies = 0          # number of bodies contained in node
        self.p = None             # if this node is a leaf with a Body
        self.children = None      # children are: [NE, NW, SW, SE]
        self.updateCOM()

    def isLeaf(self):
        return self.nbodies < 2
        
    def updateCOM(self):
        if self.isLeaf(): 
            if self.p == None:
                self.COM = Body(0, self.box.mx, self.box.my)
            else:
                self.COM = Body(self.p.m, self.p.x, self.p.y)
            return
        x = y = m = 0
        for c in self.children:
            x += c.COM.x*c.COM.m
            y += c.COM.y*c.COM.m
            m += c.COM.m
        self.COM = Body(m, x/m, y/m)

    def niceStr(self): 
        S = ("├","─","└","│")
        angle = S[2]+S[1]+" "
        vdash = S[0]+S[1]+" "
        
        def niceRec(ptr,acc,pre,A):
            if ptr == None: raise Exception("A None GNode was found")
            val = f"{len(A)}:{ptr.box},{ptr.nbodies}"
            A.append(f"({ptr.COM.m}, {ptr.COM.x}, {ptr.COM.y})")
            if ptr.children==None: return acc+pre+val
            if pre == vdash: pre2 = S[3]+"  "
            elif pre == angle: pre2 = "   "
            else: pre2 = ""
            T = [vdash,vdash,vdash,angle]
            for i in range(4):
                T[i] = niceRec(ptr.children[i],acc+pre2,T[i],A)
            return acc+pre+val+"\n"+T[0]+"\n"+T[1]+"\n"+T[2]+"\n"+T[3]
            
        A = []
        s = niceRec(self,"","",A)+"\n"
        for i in range(len(A)):
            s += f"\n{i}{' '*(3-len(str(i)))}-> {A[i]}"
        return s
    
class Stack:
    def __init__(self):
        self.inList = []
        
    def push(self,v):
        self.inList.append(v)

    def pop(self):
        if len(self.inList) == 0: raise Exception("Popping from an empty stack")
        return self.inList.pop()
    
    def isEmpty(self):
        return len(self.inList) == 0
    
    def size(self):
        return len(self.inList)

    def toArray(self):
        return self.inList
    
    def __str__(self):
        return str(self.inList)
    

####################
# Test Classes
####################
class TestBody:
    def __init__(self,m,x,y,vx=0,vy=0):
        self.m = m     # mass
        self.x = x     # position
        self.y = y     # position
        self.vx = vx   # velocity
        self.vy = vy   # velocity
        
    def squareDist(self,other):
        return (self.x-other.x)**2+(self.y-other.y)**2

    def __str__(self):
        return f"Body({self.m},{self.x},{self.y},{self.vx},{self.vy})"    

    def __repr__(self):
        return str(self)    
    
    def __eq__(self, other):
        if other is None: return False
        return self.m == other.m and self.x == other.x and self.y == other.y and self.vx == other.vx and self.vy == other.vy

    def asTuple(self):
        return (self.m,self.x,self.y,self.vx,self.vy)    
    
    def next(self, Bodies, dt):
        ret = Body(self.m, self.x, self.y)
        ax = ay = 0       
        for p in Bodies:
            if p == self: continue         
            sq_distance = self.squareDist(p)
            ax += (p.x - self.x) *  p.m * G / (sq_distance**1.5)
            ay += (p.y - self.y) *  p.m * G / (sq_distance**1.5)
        ret.x += dt*dt*ax + dt*self.vx
        ret.y += dt*dt*ay + dt*self.vy
        ret.vx = (ret.x-self.x)/dt
        ret.vy = (ret.y-self.y)/dt

        return ret
        

# Utility functions
####################


# ---------- GNode Representation (For testing) -----------------------# 

import json, zlib, base64

def _gnode_from_repr_dict(rep: dict) -> GNode:
    x0, y0, x1, y1 = rep["b"]
    nb = int(rep["n"])

    node = GNode(Box(x0, y0, x1, y1))
    node.nbodies = nb
    node.children = None
    node.p = None

    if rep.get("c") is not None:
        ch = rep["c"]
        node.children = [
            _gnode_from_repr_dict(ch[0]),
            _gnode_from_repr_dict(ch[1]),
            _gnode_from_repr_dict(ch[2]),
            _gnode_from_repr_dict(ch[3]),
        ]

    if rep.get("p") is not None:
        p = rep["p"]
        if len(p) == 5:
            m, x, y, vx, vy = p
            node.p = Body(m, x, y, vx, vy)                 
        else:
            m, x, y = p
            node.p = Body(m, x, y)

    def _recompute(n: GNode):
        if n.children is None:
            if n.p is None:
                n.COM = TestBody(0, n.box.mx, n.box.my)
            else:
                n.COM = TestBody(n.p.m, n.p.x, n.p.y)
            return n.COM
        total_m = total_x = total_y = 0.0
        for c in n.children:
            sub = _recompute(c)
            total_m += sub.m
            total_x += sub.x * sub.m
            total_y += sub.y * sub.m
        if total_m == 0:
            n.COM = TestBody(0, n.box.mx, n.box.my)
        else:
            n.COM = TestBody(total_m, total_x / total_m, total_y / total_m)
        return n.COM
    cm = rep.get("cm")
    if cm is not None:
        node.COM = TestBody(cm[0], cm[1], cm[2])
    else:
        _recompute(node)
    return node


def GNode_fromRepresentationStr(s: str, compressed: bool=False) -> GNode:
    if compressed:
        raw = base64.b64decode(s.encode('ascii'))
        s = zlib.decompress(raw).decode('utf-8')
    s, size = s.split("#")
    rep = json.loads(s)
    return _gnode_from_repr_dict(rep), int(size)

# Generators for test bodies

def get_bodies_1():
    return [
        (10, -10, 0, 3.14, -5.44),
        (10, 0, 0, 3.24, 5.44),
        (10, -5, 8.660, -6.28, 0),
    ]

def get_bodies_2():
    return [
        (1,50,50),
        (1,-50,50),
        (1,-50,-50),
        (1,50,-50),
    ]

def get_bodies_3():
    return [
        (1,100,100,0,0),
        (2,-100,100,0,0),    
        (3,-100,-100,0,0),
        (4,100,-100,0,0)
    ]


def get_bodies_4():
    P = [None]*20
    for i in range(20):
        x  = 50.0 * math.cos(i * 0.3)
        y  = 50.0 * math.sin(i * 0.3)
        vx =  5.0 * math.cos(i * 0.7)
        vy =  5.0 * math.sin(i * 0.7)
        m  = 1.0 + (i % 5)            # masses: 1..5
        P[i] = (m, x, y, vx, vy)
    return P

def get_three_circles(n):
    P = [None]*3*n
    for i in range(n):
        x  = 50.0 * math.cos(i * 0.3)
        y  = 50.0 * math.sin(i * 0.3)
        vx = 50.0 * math.cos(i * 0.7)
        vy = 50.0 * math.sin(i * 0.7)
        m  = 1.0 + (i % 5)            # masses: 1..5
        P[3*i] = (m, x, y, 3*vx, 3*vy)
        P[3*i+1] = (m, 2*x, 2*y, 2*vx, 2*vy)
        P[3*i+2] = (m, 3*x, 3*y, vx, vy)
    return P


## Testing script ##
####################

## in case python package import-ipynb is not installed, comment out next line
## BUT then the solution needs to be in file solutions in plain text
import import_ipynb

import importlib,signal

######################################################################
# initialisations 
######################################################################

testMap = {}
fullnames = {
    ("BOD", "T"): "Body.threeBodyProblem",
    ("SIM", "C"): "Simulation.closestDistance",
    ("GAD", "A"): "Gadget.add",
    ("GAD", "R"): "Gadget.remove",
    ("FSI", "G"): "FastSimulation.getBodies"
}

printsize = 35

######################################################################
# code for handling timeouts (for infinite loops)
# main function for running a list of tests
# helper function for converting test results into string
######################################################################
    
def timeoutHandler(s,f):
    raise Exception("timeout")

signal.signal(signal.SIGALRM, timeoutHandler)
timeout = 30

def tryWithTimeout(thunk):
    signal.alarm(timeout)
    v = mes = None
    try: v = thunk()
    except BaseException as e: mes = f"E: {e}"
    signal.alarm(0)
    return (v,mes)

def runTests(tests):
    awarded, total, res = 0, 0, ""
    for (UID,test,args) in tests:
        (name,mark,msg,grade) = test(*args) # grade: max; mark: awarded
        s = " "*(2-len(str(UID)))
        s = f"{s}[{UID}]{name}: "
        dots = ''.join(['.' for i in range(50-len(s))])
        res += s+dots
        awarded += grade*mark
        if round(mark,2) == 0:
            res += "error ["+msg+"], awarded: 0 of "+str(grade)
        elif round(mark,2) == 1: 
            res += "success, awarded: "+str(grade)
        else:
            res += "partial success ["+msg+"], awarded: "+str(mark*grade)+" of "+str(grade)
        res += "\n"
        total += grade
    (awarded, total) = (round(awarded,2), round(total,2))
    return (awarded,total,res)

######################################################################
# Parsing and Comparison functions
######################################################################

def markmesClosestDistance(pair):
    A, B = pair
    if math.isclose(B, A, rel_tol=1e-8, abs_tol=1e-12):
        return 1, ""
    return 0, f"Expected: {B}, Got: {A}"
    
def markmesThreeBody(pair):
    A, B = pair
    if math.isclose(B, A, rel_tol=1e-8, abs_tol=1e-12):
        return 1, ""
    return 0, f"Expected: {B}, Got: {A}"
    
def markmesGadget(pair):
    A, B = pair
    test_str = str(A).replace("\n", "")
    test_size = A.size
    actual_str = B[0]
    actual_size = B[1]
    if test_str == actual_str and test_size == actual_size: return 1, ""
    if test_str == actual_str: return 0.9, "Size mismatch"
    return 0, "Mismatch"

def markmesFastsim(pair):
    A, B = pair
    test_result = parseBodies(A)
    actual_result = B
    if test_result == actual_result: return 1, ""
    return 0, "Mismatch"

def parseBodies(p):
    s = ""
    for entity in p:
        s += f"{len(entity)} {entity};"
    return s


def parseStack(A):
    s = Stack();
    for i in range(len(A)-1,-1,-1):
        s.push(A[i])
    return s


######################################################################
# Simulation test functions
######################################################################

# nm: test number
# max_mark: max total marks for test
# initl_bodies: initial bodies given as (array list elts, undo stack elts)
# result: desired output
# args: additional arguments for simulation

def SIM_test(nm,max_mark,initl_bodies,result,args=(None, None)):
    s = fullnames['SIM',"C"]
    testName = f"[{s}{'.'*(printsize-len(s))}{nm}]"
    out = SIMClosestHelper(initl_bodies,args, result, max_mark)
    mark, msg = out
    return (testName,mark,msg,max_mark)

testMap["SIM"] = SIM_test

def SIMClosestHelper(initl, args, result, max_mark):
    total_time, dt = args
    if not total_time:
        total_time = 10
    if not dt:
        dt = 0.01
    sim = Simulation(initl, total_time, dt)
    sim_results = []
    _,mes = tryWithTimeout(lambda: sim_results.append(sim.closestDistance()))
    if mes != None: return (0, mes)
    test_result = sim_results[0]
    mark, msg = markmesClosestDistance((test_result, result))
    return mark*max_mark, msg
    


testMap["SIM", "C"] = [None] * 11

# Cases from description

P = [None]*81
for y in range(9):
    for x in range(9):
        P[9*y+x] = TestBody(1,2*x,2*y)

testMap["SIM", "C"][0] = (
    "01", 1, 
    P, 
    0.015527720571708991
)

testMap["SIM", "C"][1] = (
    "02", 1, 
    P, 
    0.00019088314702779433,
    (0.5, 0.0001)
)

# Generated Test Cases

testMap["SIM", "C"][2] = (
    "03", 1, 
    [TestBody(40,9,79,41,24), TestBody(40,24,21,78,59), TestBody(65,24,77,17,7), TestBody(65,21,1,90,55), TestBody(40,86,19,92,21), TestBody(10,80,17,17,49), TestBody(50,22,74,36,42), TestBody(25,63,41,79,98), TestBody(65,68,81,47,63), TestBody(60,54,24,76,54), TestBody(80,51,95,96,54), TestBody(80,25,81,17,100), TestBody(5,29,66,56,78), TestBody(90,20,63,23,83), TestBody(30,98,52,97,48), TestBody(50,13,28,51,45), TestBody(15,17,81,24,57), TestBody(80,43,84,95,28), TestBody(55,26,70,84,16), TestBody(90,79,81,55,100), TestBody(10,79,9,27,40), TestBody(95,59,21,23,43), TestBody(5,72,50,30,45), TestBody(35,88,23,63,12), TestBody(85,85,90,47,52), TestBody(40,86,62,51,9), TestBody(80,69,93,33,95), TestBody(65,32,32,23,38), TestBody(75,51,24,55,62), TestBody(45,30,25,86,54)],
    0.12770092636645747,
    (7.5, 0.008)
    )


testMap["SIM", "C"][3] = (
    "04", 1, 
    [TestBody(35,14,10,56,67), TestBody(15,35,100,81,89), TestBody(25,9,15,32,13), TestBody(20,33,40,7,60), TestBody(35,18,93,38,31), TestBody(15,4,25,80,48), TestBody(90,96,93,78,97), TestBody(20,92,2,56,37), TestBody(55,39,92,46,26), TestBody(15,31,62,67,21), TestBody(90,83,58,10,79), TestBody(15,65,67,26,100), TestBody(50,47,58,27,62), TestBody(55,19,5,32,100), TestBody(90,93,18,35,69), TestBody(85,37,42,92,53), TestBody(45,87,51,76,20), TestBody(50,51,51,46,75), TestBody(60,2,36,91,95), TestBody(100,82,24,77,97), TestBody(45,66,87,66,14), TestBody(85,58,9,68,84), TestBody(55,9,72,100,73), TestBody(95,51,69,72,38), TestBody(90,73,84,12,15), TestBody(90,75,7,85,82), TestBody(5,31,33,35,73), TestBody(75,19,69,75,80), TestBody(55,74,6,96,97), TestBody(100,31,86,51,72), TestBody(30,92,41,40,92), TestBody(10,50,63,66,65), TestBody(40,67,75,22,53), TestBody(45,45,51,83,66), TestBody(90,16,95,82,85), TestBody(45,93,42,81,41), TestBody(50,72,91,55,37), TestBody(75,6,32,38,100), TestBody(40,28,31,42,17), TestBody(70,5,1,58,55), TestBody(30,44,97,97,29), TestBody(30,68,59,98,36), TestBody(10,53,55,34,85), TestBody(80,59,77,97,83), TestBody(5,30,50,32,63), TestBody(75,15,87,52,66), TestBody(40,67,94,98,78), TestBody(80,48,74,26,31), TestBody(35,21,34,17,33), TestBody(75,27,28,11,52), TestBody(100,25,89,45,63), TestBody(85,83,75,59,99), TestBody(90,39,84,69,46), TestBody(55,91,70,19,72), TestBody(70,66,53,23,67), TestBody(100,68,51,45,91), TestBody(70,58,100,83,48), TestBody(20,32,9,45,40), TestBody(40,74,86,48,37), TestBody(25,47,39,44,30), TestBody(70,55,97,84,8), TestBody(90,96,26,66,82), TestBody(5,29,100,40,7), TestBody(95,26,90,22,79), TestBody(15,95,24,10,60), TestBody(60,30,35,33,5), TestBody(30,89,78,79,13), TestBody(55,47,98,47,41), TestBody(5,3,77,97,59), TestBody(15,63,79,51,31), TestBody(40,31,95,83,65), TestBody(5,1,91,11,13), TestBody(30,87,26,26,95), TestBody(80,7,89,24,63), TestBody(45,15,81,75,25), TestBody(70,15,66,76,21)],
    0.048302230978053515,
    (7.0, 0.005)
    )


testMap["SIM", "C"][4] = (
    "05", 1, 
    [TestBody(100,68,50,59,30), TestBody(90,18,84,69,58), TestBody(90,65,76,86,64), TestBody(90,6,58,37,16), TestBody(5,68,2,83,10), TestBody(75,52,18,30,83), TestBody(5,44,61,23,68), TestBody(100,67,42,51,28), TestBody(85,57,51,84,15), TestBody(65,22,42,54,82), TestBody(60,38,11,39,81), TestBody(70,5,84,30,24), TestBody(100,16,99,15,99), TestBody(15,8,91,99,83), TestBody(70,44,92,60,91), TestBody(5,49,2,9,18), TestBody(10,21,6,38,92), TestBody(15,60,43,33,94), TestBody(40,35,15,73,91), TestBody(80,46,43,11,60), TestBody(15,20,7,43,41), TestBody(85,38,89,66,28), TestBody(50,55,57,72,20), TestBody(35,80,45,61,17), TestBody(45,57,83,29,7), TestBody(30,93,95,34,44), TestBody(10,95,31,79,81), TestBody(35,99,34,82,74), TestBody(100,94,69,26,81), TestBody(40,59,68,46,55), TestBody(55,91,72,62,10), TestBody(10,29,65,47,54), TestBody(95,84,51,72,59), TestBody(55,32,37,97,76), TestBody(75,77,45,33,52), TestBody(85,54,12,34,30)],
    0.13467537660214712,
    (2.5, 0.004)
    )


testMap["SIM", "C"][5] = (
    "06", 1, 
    [TestBody(70,10,100,33,90), TestBody(65,91,92,33,20), TestBody(80,9,54,36,12), TestBody(30,53,72,81,51), TestBody(45,32,37,88,70), TestBody(60,6,9,46,34), TestBody(55,78,15,55,19), TestBody(15,32,84,44,14), TestBody(25,80,1,93,31), TestBody(80,3,24,85,39), TestBody(55,47,66,73,44), TestBody(15,49,56,54,93), TestBody(40,18,74,36,62), TestBody(50,92,98,7,60), TestBody(100,12,76,32,95), TestBody(40,74,97,53,21), TestBody(35,33,40,12,13), TestBody(20,73,93,38,91), TestBody(5,26,89,37,90), TestBody(100,2,81,32,28), TestBody(85,18,66,41,51), TestBody(85,49,41,19,84), TestBody(10,34,34,15,84), TestBody(95,72,5,64,92), TestBody(80,5,62,57,59), TestBody(20,18,49,72,10), TestBody(35,24,53,49,11), TestBody(55,25,60,92,43), TestBody(20,95,35,77,24), TestBody(100,53,10,45,83), TestBody(75,81,88,64,97), TestBody(100,17,52,91,26), TestBody(75,21,87,25,67), TestBody(40,89,22,19,63), TestBody(30,74,93,45,40), TestBody(55,86,54,21,92), TestBody(45,68,92,49,83), TestBody(10,25,89,17,37), TestBody(25,20,80,33,51), TestBody(50,2,7,96,37), TestBody(10,40,37,18,38), TestBody(20,9,99,16,100), TestBody(45,37,12,26,37), TestBody(95,14,28,17,82), TestBody(80,68,68,99,65), TestBody(30,26,5,40,31), TestBody(20,70,87,8,39), TestBody(80,20,43,84,35), TestBody(20,8,10,93,72), TestBody(90,80,40,75,51), TestBody(60,68,27,52,16), TestBody(45,45,43,41,53), TestBody(65,26,35,69,18), TestBody(40,84,87,62,49), TestBody(80,96,86,14,98), TestBody(20,56,90,38,35), TestBody(95,5,63,46,36), TestBody(90,7,26,73,66), TestBody(45,97,74,48,69), TestBody(30,91,42,22,51), TestBody(55,96,71,45,93), TestBody(70,71,47,68,7), TestBody(90,41,61,51,38), TestBody(90,77,63,27,95), TestBody(30,26,37,50,9), TestBody(50,12,86,98,39), TestBody(95,57,68,26,57), TestBody(60,38,44,52,60), TestBody(100,3,4,40,9), TestBody(45,73,45,88,58), TestBody(5,92,58,32,10), TestBody(45,38,24,82,70), TestBody(10,3,54,67,63), TestBody(70,48,51,60,19), TestBody(20,84,50,13,72), TestBody(80,77,39,21,13), TestBody(20,58,80,75,83), TestBody(30,28,57,68,47), TestBody(90,65,91,98,17), TestBody(85,64,6,42,100), TestBody(35,3,95,89,78), TestBody(50,91,35,86,26), TestBody(5,54,90,16,8), TestBody(50,22,73,47,27), TestBody(85,7,86,55,93), TestBody(15,44,89,95,68), TestBody(50,52,88,7,100), TestBody(25,32,63,72,5), TestBody(90,19,5,77,62), TestBody(25,85,18,9,85), TestBody(10,28,9,36,8), TestBody(65,71,81,60,20), TestBody(80,1,56,17,68), TestBody(35,57,8,25,36), TestBody(55,5,31,59,52), TestBody(100,50,18,65,58)],
    0.01926733832618305,
    (2.5, 0.001)
    )


testMap["SIM", "C"][6] = (
    "07", 1, 
    [TestBody(60,7,26,31,14), TestBody(45,33,28,30,63), TestBody(25,14,57,25,53), TestBody(25,82,87,30,19), TestBody(95,21,68,92,65), TestBody(15,85,12,72,63), TestBody(90,75,49,54,93), TestBody(55,34,98,85,54), TestBody(30,31,24,6,57), TestBody(10,27,26,74,79), TestBody(45,56,66,91,99), TestBody(20,100,81,86,70), TestBody(55,35,19,40,65), TestBody(60,15,13,13,18), TestBody(60,48,82,97,7), TestBody(55,85,57,50,16), TestBody(55,64,29,55,100), TestBody(45,15,94,88,42), TestBody(20,74,29,84,84), TestBody(50,64,77,34,48), TestBody(65,75,54,54,54), TestBody(80,11,61,54,61), TestBody(100,69,58,59,67), TestBody(20,22,84,37,52), TestBody(75,49,53,12,30), TestBody(60,40,62,15,51), TestBody(85,72,91,22,78), TestBody(65,7,86,84,88), TestBody(25,25,64,81,30), TestBody(45,23,20,58,60), TestBody(5,96,68,57,15), TestBody(20,100,54,81,12), TestBody(80,6,70,16,31), TestBody(75,85,28,49,93), TestBody(65,3,77,9,29), TestBody(45,52,41,35,28), TestBody(100,92,53,62,24), TestBody(10,94,15,97,36), TestBody(65,40,45,47,41), TestBody(55,90,56,68,16), TestBody(5,56,81,17,12), TestBody(25,96,88,30,29), TestBody(60,58,73,49,85), TestBody(45,62,9,79,67), TestBody(15,97,21,37,27), TestBody(35,88,72,42,34), TestBody(5,33,75,48,81), TestBody(40,59,37,8,19), TestBody(80,19,60,70,66), TestBody(100,63,88,60,69), TestBody(10,92,21,91,88), TestBody(50,58,63,89,55), TestBody(75,49,50,9,84), TestBody(50,11,20,48,34), TestBody(65,74,18,33,96), TestBody(95,82,46,52,52)],
    0.09919572580431214,
    (9.5, 0.009)
    )


testMap["SIM", "C"][7] = (
    "08", 1, 
    [TestBody(100,19,39,69,55), TestBody(75,2,68,68,26), TestBody(85,36,50,42,32), TestBody(65,10,33,99,11), TestBody(15,64,21,85,88), TestBody(40,96,61,83,11), TestBody(75,55,97,70,33), TestBody(95,86,68,41,47), TestBody(45,96,20,80,6), TestBody(50,2,56,96,54), TestBody(65,22,4,39,42), TestBody(35,27,6,92,36), TestBody(55,61,64,49,17), TestBody(65,68,88,22,85), TestBody(90,97,41,88,10), TestBody(100,19,6,82,78), TestBody(20,19,60,69,57), TestBody(55,22,36,52,77), TestBody(45,22,28,94,26), TestBody(60,24,43,62,68), TestBody(90,81,81,76,45), TestBody(15,76,30,53,62), TestBody(10,14,9,77,31), TestBody(90,94,54,87,70), TestBody(10,40,15,61,7), TestBody(50,45,39,8,5), TestBody(75,59,33,97,18), TestBody(10,98,97,46,80), TestBody(20,25,94,96,39), TestBody(20,54,65,74,38), TestBody(70,100,92,80,56), TestBody(55,16,5,22,55), TestBody(5,100,16,61,58), TestBody(15,20,88,66,15), TestBody(70,8,22,23,64), TestBody(80,88,23,75,15), TestBody(45,46,36,96,39), TestBody(50,59,16,7,91), TestBody(95,15,76,76,75), TestBody(95,72,11,92,89), TestBody(25,1,69,59,45), TestBody(20,94,21,75,40), TestBody(75,57,86,7,61), TestBody(75,59,46,100,16), TestBody(65,26,42,24,36), TestBody(100,84,1,86,19), TestBody(55,91,93,80,31), TestBody(95,31,82,84,80), TestBody(50,36,27,64,29), TestBody(10,54,58,43,17), TestBody(80,28,19,86,22), TestBody(90,46,56,21,84), TestBody(60,28,37,12,37), TestBody(45,5,25,39,25), TestBody(65,4,86,65,41), TestBody(30,43,69,60,69), TestBody(85,4,45,75,13), TestBody(10,42,69,28,51), TestBody(35,79,44,7,24), TestBody(100,75,52,10,66), TestBody(30,94,74,85,38), TestBody(10,62,55,89,71), TestBody(85,76,64,49,18), TestBody(45,31,13,88,73), TestBody(15,29,83,40,27), TestBody(40,92,39,73,8), TestBody(5,87,56,10,75)],
    0.06539424837382928,
    (1.5, 0.006)
    )


testMap["SIM", "C"][8] = (
    "09", 1, 
    [TestBody(25,61,67,37,87), TestBody(15,47,56,34,82), TestBody(30,61,55,33,88), TestBody(50,8,26,98,15), TestBody(10,97,54,28,31), TestBody(25,35,73,72,9), TestBody(75,82,74,23,65), TestBody(20,48,53,80,69), TestBody(85,22,68,48,55), TestBody(20,21,52,21,16), TestBody(55,2,37,9,12), TestBody(60,65,95,58,72), TestBody(100,82,26,8,96), TestBody(25,68,90,41,66), TestBody(20,4,74,20,51), TestBody(95,82,79,66,85), TestBody(45,51,14,94,74), TestBody(75,24,19,74,12), TestBody(35,99,76,59,85), TestBody(35,20,70,91,30), TestBody(50,40,55,88,10), TestBody(95,28,20,35,58), TestBody(100,55,88,91,20), TestBody(5,72,10,28,88), TestBody(45,90,33,97,21), TestBody(35,15,42,14,43), TestBody(5,49,77,100,64), TestBody(30,61,54,48,84), TestBody(40,39,86,30,21), TestBody(65,55,29,21,29), TestBody(35,56,6,93,85), TestBody(80,88,51,56,80), TestBody(30,5,53,38,66)],
    0.13785224629906187,
    (9.5, 0.005)
    )


testMap["SIM", "C"][9] = (
    "10", 1, 
    [TestBody(85,69,91,73,80), TestBody(25,47,52,29,32), TestBody(20,61,36,63,71), TestBody(35,76,59,68,26), TestBody(40,92,97,79,60), TestBody(65,14,51,98,74), TestBody(15,63,26,28,62), TestBody(20,65,1,89,57), TestBody(55,22,11,14,88), TestBody(70,73,53,97,65), TestBody(10,29,74,62,58), TestBody(45,88,38,100,78), TestBody(50,47,93,71,66), TestBody(45,73,11,98,63), TestBody(35,51,36,68,40), TestBody(90,65,56,10,61), TestBody(40,88,16,30,26), TestBody(60,58,100,71,100)],
    0.4675908197155302,
    (5.0, 0.009)
    )


testMap["SIM", "C"][10] = (
    "11", 1, 
    [TestBody(50,40,32,19,81), TestBody(55,27,63,11,27), TestBody(70,90,50,77,80), TestBody(95,49,45,50,95), TestBody(25,97,87,21,48), TestBody(65,83,87,26,41), TestBody(25,12,17,15,54), TestBody(80,80,27,78,18), TestBody(100,68,24,9,16), TestBody(55,5,20,53,43), TestBody(40,57,71,39,41), TestBody(80,17,96,11,7), TestBody(15,43,3,18,50), TestBody(35,62,4,18,80), TestBody(90,3,41,31,96), TestBody(45,80,33,74,91), TestBody(30,41,65,8,5), TestBody(80,80,88,50,65), TestBody(80,12,36,20,48), TestBody(25,24,15,64,83), TestBody(15,33,65,72,39), TestBody(75,86,69,28,77), TestBody(35,23,47,15,34), TestBody(50,91,87,83,52), TestBody(20,87,4,98,63), TestBody(95,67,6,87,99), TestBody(20,21,18,90,9), TestBody(40,90,27,27,76), TestBody(75,1,45,99,13), TestBody(20,48,16,38,52), TestBody(95,14,71,79,86), TestBody(45,47,44,30,7), TestBody(65,36,47,87,85), TestBody(55,35,49,9,17), TestBody(30,31,60,77,41), TestBody(100,88,9,24,14), TestBody(55,62,81,71,79), TestBody(55,18,15,13,26), TestBody(95,21,43,97,34), TestBody(55,3,67,34,46), TestBody(95,20,3,33,9), TestBody(25,99,96,28,21), TestBody(5,43,84,40,79), TestBody(25,36,7,16,38), TestBody(100,96,67,88,56), TestBody(95,70,26,96,9), TestBody(90,11,44,81,6), TestBody(95,26,31,63,95), TestBody(50,81,27,96,44), TestBody(30,96,80,6,62), TestBody(60,60,1,19,77), TestBody(55,28,10,57,83), TestBody(100,86,46,23,40), TestBody(75,55,15,39,10), TestBody(60,48,29,93,12), TestBody(60,8,57,79,42), TestBody(90,72,63,5,57), TestBody(55,63,79,91,24), TestBody(100,92,87,99,62), TestBody(75,85,23,80,38), TestBody(60,48,46,41,98), TestBody(40,12,61,64,51), TestBody(60,3,74,50,84), TestBody(100,15,52,99,44), TestBody(5,49,7,25,42), TestBody(15,93,67,46,16), TestBody(5,80,83,57,83), TestBody(75,14,89,11,58), TestBody(90,77,19,29,70), TestBody(70,91,10,82,32), TestBody(100,62,30,45,24), TestBody(25,55,42,10,8), TestBody(85,66,51,68,70), TestBody(10,15,88,17,43), TestBody(50,3,5,45,42), TestBody(75,63,97,98,69)],
    0.07686616648654135,
    (7.5, 0.002)
    )
    


    



######################################################################
# Body test functions
######################################################################

# nm: test number
# max_mark: max total marks for test
# arg_bodies: pair as (B, [B1, B2, B3]) such that we call B.threeBodyProblem(B1, B2, B3, ...)
# result: desired output
# luminosities: argument luminosities 

def BOD_test(nm,max_mark,arg_bodies,result,luminosities):
    s = fullnames['BOD',"T"]
    testName = f"[{s}{'.'*(printsize-len(s))}{nm}]"
    out = BODThreeBodyHelper(arg_bodies, luminosities, result, max_mark)
    mark, msg = out
    return (testName,mark,msg,max_mark)

testMap["BOD"] = BOD_test

def BODThreeBodyHelper(args_bodies, luminosities, result, max_mark):
    bod_results = []
    p = Body(*args_bodies[0])
    args = [TestBody(*x) for x in args_bodies[1]]
    _,mes = tryWithTimeout(lambda: bod_results.append(p.threeBodyProblem(*args, *luminosities)))
    if mes != None: return (0, mes)
    test_result = bod_results[0]
    mark, msg = markmesThreeBody((test_result, result))
    return mark*max_mark, msg
    


testMap["BOD", "T"] = [None] * 11

# Cases from Description

testMap["BOD", "T"][0] = (
    "01", 1, 
    ((3.0e-6, -10, 6, -3, -15), get_bodies_1()), 
    0.42,
    (50,50,50)
)

testMap["BOD", "T"][1] = (
    "02", 1, 
    ((3.0e-6, -10, 6, -3, -15), get_bodies_1()), 
    0,
    (45,45,45)
)

# Generated Test Cases
 
testMap["BOD", "T"][2] = (
    "03", 1,
    [(80,11,13,34,47), [(90,86,55,68,63), (75,12,10,67,87), (80,72,83,55,32)]],
    0.14,
    [52, 44, 41]
    )


testMap["BOD", "T"][3] = (
    "04", 1,
    [(20,42,61,58,39), [(80,4,41,24,6), (15,42,59,58,22), (100,15,89,93,30)]],
    0.14,
    [40, 52, 48]
    )


testMap["BOD", "T"][4] = (
    "05", 1,
    [(70,69,44,54,53), [(65,59,38,40,89), (10,66,47,99,97), (50,64,87,26,52)]],
    0.02,
    [47, 51, 60]
    )


testMap["BOD", "T"][5] = (
    "06", 1,
    [(15,62,47,57,7), [(95,58,49,44,97), (80,7,40,47,95), (65,68,46,27,46)]],
    0.02,
    [42, 51, 52]
    )


testMap["BOD", "T"][6] = (
    "07", 1,
    [(90,72,28,69,82), [(75,8,97,80,26), (15,70,25,90,32), (20,28,44,65,72)]],
    0.01,
    [48, 44, 58]
    )


testMap["BOD", "T"][7] = (
    "08", 1,
    [(40,92,91,20,96), [(85,19,15,90,84), (75,75,89,87,67), (85,95,89,73,23)]],
    0.01,
    [48, 43, 49]
    )


testMap["BOD", "T"][8] = (
    "09", 1,
    [(10,46,13,40,72), [(75,50,13,89,28), (60,12,3,32,38), (30,96,42,62,8)]],
    0.01,
    [59, 50, 57]
    )


testMap["BOD", "T"][9] = (
    "10", 1,
    [(60,47,93,100,50), [(75,29,12,66,24), (80,68,77,28,68), (15,49,93,78,94)]],
    0.03,
    [55, 52, 52]
    )


testMap["BOD", "T"][10] = (
    "11", 1,
    [(35,46,57,9,8), [(30,43,68,26,53), (55,89,24,51,91), (40,44,58,62,85)]],
    0.05,
    [58, 49, 49]
    )




######################################################################
# Gadget test functions
######################################################################

# nm: test number
# max_mark: max total marks for test
# subtype: A for add, R for remove (as string)
# initl: list of all bodies passed to the constructor of the gadget
# result: desired output
# args: list of bodies to be added / removed from the gadget

def GAD_test(nm,max_mark,subtype,initl,result,args):
    if subtype == "A":
        s = fullnames["GAD", "A"]
        fun = Gadget.add
        for index in range(len(initl)):
            initl[index] = TestBody(*initl[index])
        g = Gadget(Box.getBox(initl))
    else:
        s = fullnames["GAD", "R"]
        fun = Gadget.remove
        g = Gadget(Box(0,0,0,0))
        g.root, g.size = GNode_fromRepresentationStr(initl)
    testName = f"[{s}{'.'*(printsize-len(s))}{nm}]"
    out = GADHelper(g, fun, args, result, max_mark)
    mark, msg = out
    return (testName,mark,msg,max_mark)

testMap["GAD"] = GAD_test

def GADHelper(g, fun, args, result, max_mark):
    args = [TestBody(*x) for x in args]
    _,mes = tryWithTimeout(lambda: [fun(g, body) for body in args])
    if mes != None: return (0, mes)
    mark, msg = markmesGadget((g, result))
    return mark*max_mark, msg



testMap["GAD", "A"] = [None] * 11

# From Description

testMap["GAD", "A"][0] = (
    "01", 1, "A",
    [b for b in get_bodies_2()], 
    ("0:Box(-50,-50,50,50),4├─ 1:Box(0.0,0.0,50,50),1├─ 2:Box(-50,0.0,0.0,50),1├─ 3:Box(-50,-50,0.0,0.0),1└─ 4:Box(0.0,-50,50,0.0),10  -> (4, 0.0, 0.0)1  -> (1, 50, 50)2  -> (1, -50, 50)3  -> (1, -50, -50)4  -> (1, 50, -50)", 4),
    [b for b in get_bodies_2()]
)

testMap["GAD", "A"][1] = (
    "02", 1, "A",
    [p for p in get_bodies_3()], 
    ("0:Box(-100,-100,100,100),4├─ 1:Box(0.0,0.0,100,100),1├─ 2:Box(-100,0.0,0.0,100),1├─ 3:Box(-100,-100,0.0,0.0),1└─ 4:Box(0.0,-100,100,0.0),10  -> (10, 0.0, -40.0)1  -> (1, 100, 100)2  -> (2, -100, 100)3  -> (3, -100, -100)4  -> (4, 100, -100)",4),
    [p for p in get_bodies_3()],
)

testMap["GAD", "A"][2] = (
    "03", 1, "A",
    [p for p in get_bodies_4()], 
    ("0:Box(-49.49962483002227,-49.808230441792034,50.0,49.874749330202725),20├─ 1:Box(0.25018758498886484,0.033259444205345545,50.0,49.874749330202725),5│  ├─ 2:Box(25.125093792494432,24.954004387204037,50.0,49.874749330202725),2│  │  ├─ 3:Box(37.56254689624721,37.41437685870338,50.0,49.874749330202725),0│  │  ├─ 4:Box(25.125093792494432,37.41437685870338,37.56254689624721,49.874749330202725),1│  │  ├─ 5:Box(25.125093792494432,24.954004387204037,37.56254689624721,37.41437685870338),0│  │  └─ 6:Box(37.56254689624721,24.954004387204037,50.0,37.41437685870338),1│  ├─ 7:Box(0.25018758498886484,24.954004387204037,25.125093792494432,49.874749330202725),2│  │  ├─ 8:Box(12.687640688741649,37.41437685870338,25.125093792494432,49.874749330202725),1│  │  ├─ 9:Box(0.25018758498886484,37.41437685870338,12.687640688741649,49.874749330202725),1│  │  ├─ 10:Box(0.25018758498886484,24.954004387204037,12.687640688741649,37.41437685870338),0│  │  └─ 11:Box(12.687640688741649,24.954004387204037,25.125093792494432,37.41437685870338),0│  ├─ 12:Box(0.25018758498886484,0.033259444205345545,25.125093792494432,24.954004387204037),0│  └─ 13:Box(25.125093792494432,0.033259444205345545,50.0,24.954004387204037),1├─ 14:Box(-49.49962483002227,0.033259444205345545,0.25018758498886484,49.874749330202725),5│  ├─ 15:Box(-24.624718622516703,24.954004387204037,0.25018758498886484,49.874749330202725),1│  ├─ 16:Box(-49.49962483002227,24.954004387204037,-24.624718622516703,49.874749330202725),2│  │  ├─ 17:Box(-37.06217172626948,37.41437685870338,-24.624718622516703,49.874749330202725),1│  │  ├─ 18:Box(-49.49962483002227,37.41437685870338,-37.06217172626948,49.874749330202725),0│  │  ├─ 19:Box(-49.49962483002227,24.954004387204037,-37.06217172626948,37.41437685870338),0│  │  └─ 20:Box(-37.06217172626948,24.954004387204037,-24.624718622516703,37.41437685870338),1│  ├─ 21:Box(-49.49962483002227,0.033259444205345545,-24.624718622516703,24.954004387204037),2│  │  ├─ 22:Box(-37.06217172626948,12.493631915704691,-24.624718622516703,24.954004387204037),0│  │  ├─ 23:Box(-49.49962483002227,12.493631915704691,-37.06217172626948,24.954004387204037),1│  │  ├─ 24:Box(-49.49962483002227,0.033259444205345545,-37.06217172626948,12.493631915704691),1│  │  └─ 25:Box(-37.06217172626948,0.033259444205345545,-24.624718622516703,12.493631915704691),0│  └─ 26:Box(-24.624718622516703,0.033259444205345545,0.25018758498886484,24.954004387204037),0├─ 27:Box(-49.49962483002227,-49.808230441792034,0.25018758498886484,0.033259444205345545),5│  ├─ 28:Box(-24.624718622516703,-24.887485498793346,0.25018758498886484,0.033259444205345545),0│  ├─ 29:Box(-49.49962483002227,-24.887485498793346,-24.624718622516703,0.033259444205345545),2│  │  ├─ 30:Box(-37.06217172626948,-12.427113027294,-24.624718622516703,0.033259444205345545),0│  │  ├─ 31:Box(-49.49962483002227,-12.427113027294,-37.06217172626948,0.033259444205345545),1│  │  ├─ 32:Box(-49.49962483002227,-24.887485498793346,-37.06217172626948,-12.427113027294),1│  │  └─ 33:Box(-37.06217172626948,-24.887485498793346,-24.624718622516703,-12.427113027294),0│  ├─ 34:Box(-49.49962483002227,-49.808230441792034,-24.624718622516703,-24.887485498793346),1│  └─ 35:Box(-24.624718622516703,-49.808230441792034,0.25018758498886484,-24.887485498793346),2│     ├─ 36:Box(-12.187265518763919,-37.347857970292694,0.25018758498886484,-24.887485498793346),0│     ├─ 37:Box(-24.624718622516703,-37.347857970292694,-12.187265518763919,-24.887485498793346),0│     ├─ 38:Box(-24.624718622516703,-49.808230441792034,-12.187265518763919,-37.347857970292694),1│     └─ 39:Box(-12.187265518763919,-49.808230441792034,0.25018758498886484,-37.347857970292694),1└─ 40:Box(0.25018758498886484,-49.808230441792034,50.0,0.033259444205345545),5   ├─ 41:Box(25.125093792494432,-24.887485498793346,50.0,0.033259444205345545),1   ├─ 42:Box(0.25018758498886484,-24.887485498793346,25.125093792494432,0.033259444205345545),0   ├─ 43:Box(0.25018758498886484,-49.808230441792034,25.125093792494432,-24.887485498793346),2   │  ├─ 44:Box(12.687640688741649,-37.347857970292694,25.125093792494432,-24.887485498793346),0   │  ├─ 45:Box(0.25018758498886484,-37.347857970292694,12.687640688741649,-24.887485498793346),0   │  ├─ 46:Box(0.25018758498886484,-49.808230441792034,12.687640688741649,-37.347857970292694),1   │  └─ 47:Box(12.687640688741649,-49.808230441792034,25.125093792494432,-37.347857970292694),1   └─ 48:Box(25.125093792494432,-49.808230441792034,50.0,-24.887485498793346),2      ├─ 49:Box(37.56254689624721,-37.347857970292694,50.0,-24.887485498793346),1      ├─ 50:Box(25.125093792494432,-37.347857970292694,37.56254689624721,-24.887485498793346),0      ├─ 51:Box(25.125093792494432,-49.808230441792034,37.56254689624721,-37.347857970292694),1      └─ 52:Box(37.56254689624721,-49.808230441792034,50.0,-37.347857970292694),00  -> (60.0, -2.4025184922444547, 0.20781343257294413)1  -> (15.0, 29.185485567046587, 36.91988629485968)2  -> (7.0, 35.44604798436923, 34.48025041925028)3  -> (0, 43.781273448123606, 43.64456309445305)4  -> (4.0, 31.080498413533224, 39.166345481374165)5  -> (0, 31.343820344370823, 31.184190622953707)6  -> (3.0, 41.26678074548391, 28.23212366975177)7  -> (6.0, 15.687716450425595, 47.147420137001546)8  -> (5.0, 18.117887723833682, 46.60195429836131)9  -> (1.0, 3.5368600833851453, 49.874749330202725)10 -> (0, 6.468914136865257, 31.184190622953707)11 -> (0, 18.90636724061804, 31.184190622953707)12 -> (0, 12.687640688741649, 12.493631915704691)13 -> (2.0, 47.7668244562803, 14.776010333066978)14 -> (15.0, -34.76290190678826, 31.72398497712211)15 -> (2.0, -11.360104734654344, 48.69238154390976)16 -> (7.0, -31.886522685461102, 37.79629158679446)17 -> (3.0, -25.24230522999288, 43.160468332443685)18 -> (0, -43.28089827814588, 43.64456309445305)19 -> (0, -43.28089827814588, 31.184190622953707)20 -> (4.0, -36.86968577706227, 33.77315902755755)21 -> (6.0, -45.91961005571458, 18.983495076908486)22 -> (0, -30.843445174393093, 18.723818151454363)23 -> (5.0, -45.20360710085305, 21.36899401169151)24 -> (1.0, -49.49962483002227, 7.0560004029933605)25 -> (0, -30.843445174393093, 6.263445679955018)26 -> (0, -12.187265518763919, 12.493631915704691)27 -> (15.0, -34.103546372516696, -32.43175444879996)28 -> (0, -12.187265518763919, -12.427113027294)29 -> (5.0, -46.65234788820171, -16.430527181710524)30 -> (0, -30.843445174393093, -6.196926791544327)31 -> (2.0, -49.37398849544324, -7.887284707162411)32 -> (3.0, -44.83792081670736, -22.126022164742604)33 -> (0, -30.843445174393093, -18.657299263043672)34 -> (4.0, -36.29661521000701, -34.38830795919869)35 -> (6.0, -22.18416588445231, -44.461741497775314)36 -> (0, -5.968538966887527, -31.11767173454302)37 -> (0, -18.40599207064031, -31.11767173454302)38 -> (5.0, -24.513041067034973, -43.57878862067941)39 -> (1.0, -10.539789971538985, -48.87650588325485)40 -> (15.0, 30.070888743280555, -35.38086309289006)41 -> (1.0, 50.0, 0.0)42 -> (0, 12.687640688741649, -12.427113027294)43 -> (5.0, 13.089311950178336, -47.697732646548786)44 -> (0, 18.90636724061804, -31.11767173454302)45 -> (0, 6.468914136865257, -31.11767173454302)46 -> (2.0, 4.37494917197232, -49.808230441792034)47 -> (3.0, 18.898887135649012, -46.29073411638662)48 -> (9.0, 37.29075237759074, -32.46936479562299)49 -> (5.0, 41.73563924195799, -27.53427712988188)50 -> (0, 31.343820344370823, -31.11767173454302)51 -> (4.0, 31.734643797131696, -38.63822437779938)52 -> (0, 43.781273448123606, -43.578044206042364)", 20),
    [p for p in get_bodies_4()]
)

testMap["GAD", "A"][3] = (
    "04", 1, "A",[(1.0, 50, 50), (1.0, -50, -50)], 
    ("0:Box(-50,-50,50,50),20├─ 1:Box(0.0,0.0,50,50),6│  ├─ 2:Box(25.0,25.0,50,50),2│  │  ├─ 3:Box(37.5,37.5,50,50),0│  │  ├─ 4:Box(25.0,37.5,37.5,50),1│  │  ├─ 5:Box(25.0,25.0,37.5,37.5),0│  │  └─ 6:Box(37.5,25.0,50,37.5),1│  ├─ 7:Box(0.0,25.0,25.0,50),2│  │  ├─ 8:Box(12.5,37.5,25.0,50),1│  │  ├─ 9:Box(0.0,37.5,12.5,50),1│  │  ├─ 10:Box(0.0,25.0,12.5,37.5),0│  │  └─ 11:Box(12.5,25.0,25.0,37.5),0│  ├─ 12:Box(0.0,0.0,25.0,25.0),0│  └─ 13:Box(25.0,0.0,50,25.0),2│     ├─ 14:Box(37.5,12.5,50,25.0),1│     ├─ 15:Box(25.0,12.5,37.5,25.0),0│     ├─ 16:Box(25.0,0.0,37.5,12.5),0│     └─ 17:Box(37.5,0.0,50,12.5),1├─ 18:Box(-50,0.0,0.0,50),5│  ├─ 19:Box(-25.0,25.0,0.0,50),1│  ├─ 20:Box(-50,25.0,-25.0,50),2│  │  ├─ 21:Box(-37.5,37.5,-25.0,50),1│  │  ├─ 22:Box(-50,37.5,-37.5,50),0│  │  ├─ 23:Box(-50,25.0,-37.5,37.5),0│  │  └─ 24:Box(-37.5,25.0,-25.0,37.5),1│  ├─ 25:Box(-50,0.0,-25.0,25.0),2│  │  ├─ 26:Box(-37.5,12.5,-25.0,25.0),0│  │  ├─ 27:Box(-50,12.5,-37.5,25.0),1│  │  ├─ 28:Box(-50,0.0,-37.5,12.5),1│  │  └─ 29:Box(-37.5,0.0,-25.0,12.5),0│  └─ 30:Box(-25.0,0.0,0.0,25.0),0├─ 31:Box(-50,-50,0.0,0.0),5│  ├─ 32:Box(-25.0,-25.0,0.0,0.0),0│  ├─ 33:Box(-50,-25.0,-25.0,0.0),2│  │  ├─ 34:Box(-37.5,-12.5,-25.0,0.0),0│  │  ├─ 35:Box(-50,-12.5,-37.5,0.0),1│  │  ├─ 36:Box(-50,-25.0,-37.5,-12.5),1│  │  └─ 37:Box(-37.5,-25.0,-25.0,-12.5),0│  ├─ 38:Box(-50,-50,-25.0,-25.0),1│  └─ 39:Box(-25.0,-50,0.0,-25.0),2│     ├─ 40:Box(-12.5,-37.5,0.0,-25.0),0│     ├─ 41:Box(-25.0,-37.5,-12.5,-25.0),0│     ├─ 42:Box(-25.0,-50,-12.5,-37.5),1│     └─ 43:Box(-12.5,-50,0.0,-37.5),1└─ 44:Box(0.0,-50,50,0.0),4   ├─ 45:Box(25.0,-25.0,50,0.0),0   ├─ 46:Box(0.0,-25.0,25.0,0.0),0   ├─ 47:Box(0.0,-50,25.0,-25.0),2   │  ├─ 48:Box(12.5,-37.5,25.0,-25.0),0   │  ├─ 49:Box(0.0,-37.5,12.5,-25.0),0   │  ├─ 50:Box(0.0,-50,12.5,-37.5),1   │  └─ 51:Box(12.5,-50,25.0,-37.5),1   └─ 52:Box(25.0,-50,50,-25.0),2      ├─ 53:Box(37.5,-37.5,50,-25.0),1      ├─ 54:Box(25.0,-37.5,37.5,-25.0),0      ├─ 55:Box(25.0,-50,37.5,-37.5),1      └─ 56:Box(37.5,-50,50,-37.5),00  -> (60.0, -2.4025184922444547, 0.20781343257294413)1  -> (16.0, 30.486392719106174, 34.61239340143095)2  -> (7.0, 35.44604798436923, 34.48025041925028)3  -> (0, 43.75, 43.75)4  -> (4.0, 31.080498413533224, 39.166345481374165)5  -> (0, 31.25, 31.25)6  -> (3.0, 41.26678074548391, 28.23212366975177)7  -> (6.0, 15.687716450425595, 47.147420137001546)8  -> (5.0, 18.117887723833682, 46.60195429836131)9  -> (1.0, 3.5368600833851453, 49.874749330202725)10 -> (0, 6.25, 31.25)11 -> (0, 18.75, 31.25)12 -> (0, 12.5, 12.5)13 -> (3.0, 48.51121630418686, 9.850673555377986)14 -> (2.0, 47.7668244562803, 14.776010333066978)15 -> (0, 31.25, 18.75)16 -> (0, 31.25, 6.25)17 -> (1.0, 50.0, 0.0)18 -> (15.0, -34.76290190678826, 31.72398497712211)19 -> (2.0, -11.360104734654344, 48.69238154390976)20 -> (7.0, -31.886522685461102, 37.79629158679446)21 -> (3.0, -25.24230522999288, 43.160468332443685)22 -> (0, -43.75, 43.75)23 -> (0, -43.75, 31.25)24 -> (4.0, -36.86968577706227, 33.77315902755755)25 -> (6.0, -45.91961005571458, 18.983495076908486)26 -> (0, -31.25, 18.75)27 -> (5.0, -45.20360710085305, 21.36899401169151)28 -> (1.0, -49.49962483002227, 7.0560004029933605)29 -> (0, -31.25, 6.25)30 -> (0, -12.5, 12.5)31 -> (15.0, -34.103546372516696, -32.43175444879996)32 -> (0, -12.5, -12.5)33 -> (5.0, -46.65234788820171, -16.430527181710524)34 -> (0, -31.25, -6.25)35 -> (2.0, -49.37398849544324, -7.887284707162411)36 -> (3.0, -44.83792081670736, -22.126022164742604)37 -> (0, -31.25, -18.75)38 -> (4.0, -36.29661521000701, -34.38830795919869)39 -> (6.0, -22.18416588445231, -44.461741497775314)40 -> (0, -6.25, -31.25)41 -> (0, -18.75, -31.25)42 -> (5.0, -24.513041067034973, -43.57878862067941)43 -> (1.0, -10.539789971538985, -48.87650588325485)44 -> (14.0, 28.647380796372026, -37.90806759952506)45 -> (0, 37.5, -12.5)46 -> (0, 12.5, -12.5)47 -> (5.0, 13.089311950178336, -47.697732646548786)48 -> (0, 18.75, -31.25)49 -> (0, 6.25, -31.25)50 -> (2.0, 4.37494917197232, -49.808230441792034)51 -> (3.0, 18.898887135649012, -46.29073411638662)52 -> (9.0, 37.29075237759074, -32.46936479562299)53 -> (5.0, 41.73563924195799, -27.53427712988188)54 -> (0, 31.25, -31.25)55 -> (4.0, 31.734643797131696, -38.63822437779938)56 -> (0, 43.75, -43.75)", 20),
    [p for p in get_bodies_4()]
)

# Generated test cases

testMap["GAD", "A"][4] = (
    "05", 1, "A",
    [(75,23,79,17,21), (85,38,62,91,41), (15,3,44,30,61), (95,14,94,7,44), (40,44,91,63,69), (45,84,70,49,27), (45,90,40,74,50), (25,10,72,74,95), (60,2,1,5,85), (40,17,21,26,46), (60,49,91,36,63), (20,99,23,100,92), (75,89,92,23,8), (55,70,21,30,51), (25,44,67,15,93), (15,90,56,57,47), (45,94,99,30,30), (25,27,6,27,53), (20,78,29,58,10), (70,94,50,6,52), (65,54,9,66,89), (25,66,96,91,71), (40,57,28,48,43), (40,3,41,5,74), (100,66,63,49,85), (5,25,64,51,9), (15,77,23,95,60)],
    ("0:Box(2,1,99,99),10  -> (90, 66, 91)",1),
    [(90,66,91,53,46)]
) 


testMap["GAD", "A"][5] = (
    "06", 1, "A",
    [(80,77,40,54,56), (35,69,57,85,28), (90,32,24,84,36), (25,21,33,64,74), (70,78,17,28,28), (60,25,83,65,59), (100,37,26,66,29), (85,68,13,31,12), (100,85,37,12,16), (15,58,18,45,8)],
    ("0:Box(21,13,85,83),4├─ 1:Box(53.0,48.0,85,83),2│  ├─ 2:Box(69.0,65.5,85,83),0│  ├─ 3:Box(53.0,65.5,69.0,83),0│  ├─ 4:Box(53.0,48.0,69.0,65.5),0│  └─ 5:Box(69.0,48.0,85,65.5),2│     ├─ 6:Box(77.0,56.75,85,65.5),0│     ├─ 7:Box(69.0,56.75,77.0,65.5),0│     ├─ 8:Box(69.0,48.0,77.0,56.75),1│     └─ 9:Box(77.0,48.0,85,56.75),1├─ 10:Box(21,48.0,53.0,83),0├─ 11:Box(21,13,53.0,48.0),0└─ 12:Box(53.0,13,85,48.0),2   ├─ 13:Box(69.0,30.5,85,48.0),1   ├─ 14:Box(53.0,30.5,69.0,48.0),1   ├─ 15:Box(53.0,13,69.0,30.5),0   └─ 16:Box(69.0,13,85,30.5),00  -> (235, 70.48936170212765, 40.276595744680854)1  -> (75, 79.0, 54.333333333333336)2  -> (0, 77.0, 74.25)3  -> (0, 61.0, 74.25)4  -> (0, 61.0, 56.75)5  -> (75, 79.0, 54.333333333333336)6  -> (0, 81.0, 61.125)7  -> (0, 73.0, 61.125)8  -> (25, 77, 53)9  -> (50, 80, 55)10 -> (0, 37.0, 65.5)11 -> (0, 37.0, 30.5)12 -> (160, 66.5, 33.6875)13 -> (70, 71, 32)14 -> (90, 63, 35)15 -> (0, 61.0, 21.75)16 -> (0, 77.0, 21.75)",4),
    [(70,71,32,82,21), (50,80,55,44,16), (90,63,35,92,89), (25,77,53,87,84)]
) 


testMap["GAD", "A"][6] = (
    "07", 1, "A",
    [(65,40,35,68,57), (95,74,29,92,87), (25,54,99,75,92), (15,19,84,59,82), (15,43,84,11,14), (40,8,16,32,39), (90,8,20,80,34), (50,28,99,30,14), (95,97,21,52,93), (55,28,76,84,46), (5,54,41,66,43), (45,64,4,74,90), (100,29,20,15,84), (85,91,28,6,75)],
    ("0:Box(8,4,97,99),5├─ 1:Box(52.5,51.5,97,99),2│  ├─ 2:Box(74.75,75.25,97,99),1│  ├─ 3:Box(52.5,75.25,74.75,99),1│  ├─ 4:Box(52.5,51.5,74.75,75.25),0│  └─ 5:Box(74.75,51.5,97,75.25),0├─ 6:Box(8,51.5,52.5,99),1├─ 7:Box(8,4,52.5,51.5),2│  ├─ 8:Box(30.25,27.75,52.5,51.5),1│  ├─ 9:Box(8,27.75,30.25,51.5),0│  ├─ 10:Box(8,4,30.25,27.75),0│  └─ 11:Box(30.25,4,52.5,27.75),1└─ 12:Box(52.5,4,97,51.5),00  -> (200, 64.725, 77.475)1  -> (145, 72.06896551724138, 90.3103448275862)2  -> (55, 82, 81)3  -> (90, 66, 96)4  -> (0, 63.625, 63.375)5  -> (0, 85.875, 63.375)6  -> (30, 51, 56)7  -> (25, 38.6, 28.8)8  -> (10, 44, 48)9  -> (0, 19.125, 39.625)10 -> (0, 19.125, 15.875)11 -> (15, 35, 16)12 -> (0, 74.75, 27.75)",5),
    [(90,66,96,14,79), (10,44,48,7,23), (15,35,16,52,90), (55,82,81,45,97), (30,51,56,85,7)]
) 


testMap["GAD", "A"][7] = (
    "08", 1, "A",
    [(60,13,2,27,93), (85,12,96,62,37), (5,93,52,13,54), (75,80,50,92,71), (85,18,52,39,32), (80,44,78,62,9), (25,11,90,71,30), (100,46,14,89,76), (45,91,75,22,56), (55,95,21,73,66), (90,76,59,90,28), (90,4,49,93,11), (20,79,67,29,54), (95,72,40,97,78), (50,73,92,80,82), (85,39,26,30,85), (35,12,81,35,82), (10,17,27,57,59), (60,92,31,75,83), (20,94,29,55,57), (40,11,74,51,40), (65,41,69,61,25)],
    ("0:Box(4,2,95,96),2├─ 1:Box(49.5,49.0,95,96),0├─ 2:Box(4,49.0,49.5,96),0├─ 3:Box(4,2,49.5,49.0),1└─ 4:Box(49.5,2,95,49.0),10  -> (80, 72.0625, 45.0625)1  -> (0, 72.25, 72.5)2  -> (0, 26.75, 72.5)3  -> (5, 28, 16)4  -> (75, 75, 47)",2),
    [(75,75,47,87,50), (5,28,16,31,39)]
) 


testMap["GAD", "A"][8] = (
    "09", 1, "A",
    [(10,53,31,71,22), (5,22,12,71,17), (80,42,30,26,24), (95,25,60,45,56), (50,19,31,62,85), (25,50,41,56,37), (100,100,11,29,31), (65,10,27,39,90), (85,40,71,83,9)],
    ("0:Box(10,11,100,71),10  -> (35, 40, 11)",1),
    [(35,40,11,87,15)]
) 


testMap["GAD", "A"][9] = (
    "10", 1, "A",
    [(45,9,63,7,97), (25,39,62,78,59), (50,33,74,96,27), (35,80,40,83,27), (25,63,75,35,34), (80,18,53,29,100), (20,24,90,95,57), (55,52,11,29,28), (45,37,53,92,47), (35,71,59,34,52), (10,32,84,76,49), (100,98,51,90,8), (85,10,39,99,31), (5,33,12,62,10), (25,87,65,50,60), (50,90,23,19,59), (50,17,56,9,95), (50,71,39,78,51), (40,43,73,78,26), (20,95,75,81,94), (90,38,79,42,45), (10,45,18,18,25), (80,83,11,68,97), (70,85,78,56,7), (35,40,53,49,75), (45,94,76,29,13), (25,95,35,35,64), (100,82,29,83,13), (90,23,63,11,55)],
    ("0:Box(9,11,98,90),3├─ 1:Box(53.5,50.5,98,90),2│  ├─ 2:Box(75.75,70.25,98,90),0│  ├─ 3:Box(53.5,70.25,75.75,90),1│  ├─ 4:Box(53.5,50.5,75.75,70.25),0│  └─ 5:Box(75.75,50.5,98,70.25),1├─ 6:Box(9,50.5,53.5,90),1├─ 7:Box(9,11,53.5,50.5),0└─ 8:Box(53.5,11,98,50.5),00  -> (255, 69.6078431372549, 69.45098039215686)1  -> (175, 79.48571428571428, 73.77142857142857)2  -> (0, 86.875, 80.125)3  -> (80, 67, 83)4  -> (0, 64.625, 60.375)5  -> (95, 90, 66)6  -> (80, 48, 60)7  -> (0, 31.25, 30.75)8  -> (0, 75.75, 30.75)",3),
    [(80,48,60,95,47), (95,90,66,39,18), (80,67,83,43,81)]
) 


testMap["GAD", "A"][10] = (
    "11", 1, "A",
    [(45,21,50,85,16), (30,32,28,6,47), (80,5,96,72,23), (90,8,38,13,88), (50,40,13,36,82), (90,70,92,30,88), (25,29,90,55,45), (85,15,66,49,62), (5,28,54,45,46), (80,80,61,48,95), (45,1,88,31,66), (70,76,23,21,55), (35,1,85,71,51), (70,84,93,93,85), (35,17,43,61,56), (35,86,47,18,81)],
    ("0:Box(1,13,86,96),2├─ 1:Box(43.5,54.5,86,96),1├─ 2:Box(1,54.5,43.5,96),0├─ 3:Box(1,13,43.5,54.5),0└─ 4:Box(43.5,13,86,54.5),10  -> (90, 82.22222222222223, 82.88888888888889)1  -> (85, 84, 86)2  -> (0, 22.25, 75.25)3  -> (0, 22.25, 33.75)4  -> (5, 52, 30)",2),
    [(5,52,30,76,18), (85,84,86,13,37)]
) 
    


testMap["GAD", "R"] = [None]*11

# From Description

testMap["GAD", "R"][0] = (
    "01", 1, "R",'{"b":[-50,-50,50,50],"n":4,"cm":[4,0.0,0.0],"p":null,"c":[{"b":[0.0,0.0,50,50],"n":1,"cm":[1,50,50],"p":[1,50,50,0,0],"c":null},{"b":[-50,0.0,0.0,50],"n":1,"cm":[1,-50,50],"p":[1,-50,50,0,0],"c":null},{"b":[-50,-50,0.0,0.0],"n":1,"cm":[1,-50,-50],"p":[1,-50,-50,0,0],"c":null},{"b":[0.0,-50,50,0.0],"n":1,"cm":[1,50,-50],"p":[1,50,-50,0,0],"c":null}]}#4', 
    ("0:Box(-50,-50,50,50),3├─ 1:Box(0.0,0.0,50,50),0├─ 2:Box(-50,0.0,0.0,50),1├─ 3:Box(-50,-50,0.0,0.0),1└─ 4:Box(0.0,-50,50,0.0),10  -> (3, -16.666666666666668, -16.666666666666668)1  -> (0, 25.0, 25.0)2  -> (1, -50, 50)3  -> (1, -50, -50)4  -> (1, 50, -50)", 3),
    [(1,50,50)]
)

testMap["GAD", "R"][1] = (
    "02", 1, "R", '{"b":[-49.49962483002227,-49.808230441792034,50.0,49.874749330202725],"n":20,"cm":[60.0,-2.4025184922444547,0.20781343257294413],"p":null,"c":[{"b":[0.25018758498886484,0.033259444205345545,50.0,49.874749330202725],"n":5,"cm":[15.0,29.185485567046587,36.91988629485968],"p":null,"c":[{"b":[25.125093792494432,24.954004387204037,50.0,49.874749330202725],"n":2,"cm":[7.0,35.44604798436923,34.48025041925028],"p":null,"c":[{"b":[37.56254689624721,37.41437685870338,50.0,49.874749330202725],"n":0,"cm":[0,43.781273448123606,43.64456309445305],"p":null,"c":null},{"b":[25.125093792494432,37.41437685870338,37.56254689624721,49.874749330202725],"n":1,"cm":[4.0,31.080498413533224,39.166345481374165],"p":[4.0,31.080498413533224,39.166345481374165,-2.524230522999286,4.3160468332443696],"c":null},{"b":[25.125093792494432,24.954004387204037,37.56254689624721,37.41437685870338],"n":0,"cm":[0,31.343820344370823,31.184190622953707],"p":null,"c":null},{"b":[37.56254689624721,24.954004387204037,50.0,37.41437685870338],"n":1,"cm":[3.0,41.26678074548391,28.23212366975177],"p":[3.0,41.26678074548391,28.23212366975177,0.8498357145012052,4.9272486499423005],"c":null}]},{"b":[0.25018758498886484,24.954004387204037,25.125093792494432,49.874749330202725],"n":2,"cm":[6.0,15.687716450425595,47.147420137001546],"p":null,"c":[{"b":[12.687640688741649,37.41437685870338,25.125093792494432,49.874749330202725],"n":1,"cm":[5.0,18.117887723833682,46.60195429836131],"p":[5.0,18.117887723833682,46.60195429836131,-4.71111170334329,1.6749407507795255],"c":null},{"b":[0.25018758498886484,37.41437685870338,12.687640688741649,49.874749330202725],"n":1,"cm":[1.0,3.5368600833851453,49.874749330202725],"p":[1.0,3.5368600833851453,49.874749330202725,-4.682283436453981,-1.7539161384480992],"c":null},{"b":[0.25018758498886484,24.954004387204037,12.687640688741649,37.41437685870338],"n":0,"cm":[0,6.468914136865257,31.184190622953707],"p":null,"c":null},{"b":[12.687640688741649,24.954004387204037,25.125093792494432,37.41437685870338],"n":0,"cm":[0,18.90636724061804,31.184190622953707],"p":null,"c":null}]},{"b":[0.25018758498886484,0.033259444205345545,25.125093792494432,24.954004387204037],"n":0,"cm":[0,12.687640688741649,12.493631915704691],"p":null,"c":null},{"b":[25.125093792494432,0.033259444205345545,50.0,24.954004387204037],"n":1,"cm":[2.0,47.7668244562803,14.776010333066978],"p":[2.0,47.7668244562803,14.776010333066978,3.8242109364224426,3.221088436188455],"c":null}]},{"b":[-49.49962483002227,0.033259444205345545,0.25018758498886484,49.874749330202725],"n":5,"cm":[15.0,-34.76290190678826,31.72398497712211],"p":null,"c":[{"b":[-24.624718622516703,24.954004387204037,0.25018758498886484,49.874749330202725],"n":1,"cm":[2.0,-11.360104734654344,48.69238154390976],"p":[2.0,-11.360104734654344,48.69238154390976,-2.451304106703501,-4.357878862067938],"c":null},{"b":[-49.49962483002227,24.954004387204037,-24.624718622516703,49.874749330202725],"n":2,"cm":[7.0,-31.886522685461102,37.79629158679446],"p":null,"c":[{"b":[-37.06217172626948,37.41437685870338,-24.624718622516703,49.874749330202725],"n":1,"cm":[3.0,-25.24230522999288,43.160468332443685],"p":[3.0,-25.24230522999288,43.160468332443685,0.9325618471128744,-4.912263063121663],"c":null},{"b":[-49.49962483002227,37.41437685870338,-37.06217172626948,49.874749330202725],"n":0,"cm":[0,-43.28089827814588,43.64456309445305],"p":null,"c":null},{"b":[-49.49962483002227,24.954004387204037,-37.06217172626948,37.41437685870338],"n":0,"cm":[0,-43.28089827814588,31.184190622953707],"p":null,"c":null},{"b":[-37.06217172626948,24.954004387204037,-24.624718622516703,37.41437685870338],"n":1,"cm":[4.0,-36.86968577706227,33.77315902755755],"p":[4.0,-36.86968577706227,33.77315902755755,3.877829392551248,-3.1563331893616082],"c":null}]},{"b":[-49.49962483002227,0.033259444205345545,-24.624718622516703,24.954004387204037],"n":2,"cm":[6.0,-45.91961005571458,18.983495076908486],"p":null,"c":[{"b":[-37.06217172626948,12.493631915704691,-24.624718622516703,24.954004387204037],"n":0,"cm":[0,-30.843445174393093,18.723818151454363],"p":null,"c":null},{"b":[-49.49962483002227,12.493631915704691,-37.06217172626948,24.954004387204037],"n":1,"cm":[5.0,-45.20360710085305,21.36899401169151],"p":[5.0,-45.20360710085305,21.36899401169151,4.999293181917076,0.08406950242174857],"c":null},{"b":[-49.49962483002227,0.033259444205345545,-37.06217172626948,12.493631915704691],"n":1,"cm":[1.0,-49.49962483002227,7.0560004029933605],"p":[1.0,-49.49962483002227,7.0560004029933605,3.769511271716523,3.2849329935939453],"c":null},{"b":[-37.06217172626948,0.033259444205345545,-24.624718622516703,12.493631915704691],"n":0,"cm":[0,-30.843445174393093,6.263445679955018],"p":null,"c":null}]},{"b":[-24.624718622516703,0.033259444205345545,0.25018758498886484,24.954004387204037],"n":0,"cm":[0,-12.187265518763919,12.493631915704691],"p":null,"c":null}]},{"b":[-49.49962483002227,-49.808230441792034,0.25018758498886484,0.033259444205345545],"n":5,"cm":[15.0,-34.103546372516696,-32.43175444879996],"p":null,"c":[{"b":[-24.624718622516703,-24.887485498793346,0.25018758498886484,0.033259444205345545],"n":0,"cm":[0,-12.187265518763919,-12.427113027294],"p":null,"c":null},{"b":[-49.49962483002227,-24.887485498793346,-24.624718622516703,0.033259444205345545],"n":2,"cm":[5.0,-46.65234788820171,-16.430527181710524],"p":null,"c":[{"b":[-37.06217172626948,-12.427113027294,-24.624718622516703,0.033259444205345545],"n":0,"cm":[0,-30.843445174393093,-6.196926791544327],"p":null,"c":null},{"b":[-49.49962483002227,-12.427113027294,-37.06217172626948,0.033259444205345545],"n":1,"cm":[2.0,-49.37398849544324,-7.887284707162411],"p":[2.0,-49.37398849544324,-7.887284707162411,0.7668693101893262,4.940841169385001],"c":null},{"b":[-49.49962483002227,-24.887485498793346,-37.06217172626948,-12.427113027294],"n":1,"cm":[3.0,-44.83792081670736,-22.126022164742604],"p":[3.0,-44.83792081670736,-22.126022164742604,-2.5964432705834204,4.272994540441407],"c":null},{"b":[-37.06217172626948,-24.887485498793346,-24.624718622516703,-12.427113027294],"n":0,"cm":[0,-30.843445174393093,-18.657299263043672],"p":null,"c":null}]},{"b":[-49.49962483002227,-49.808230441792034,-24.624718622516703,-24.887485498793346],"n":1,"cm":[4.0,-36.29661521000701,-34.38830795919869],"p":[4.0,-36.29661521000701,-34.38830795919869,-4.738608010655559,1.5954918117467607],"c":null},{"b":[-24.624718622516703,-49.808230441792034,0.25018758498886484,-24.887485498793346],"n":2,"cm":[6.0,-22.18416588445231,-44.461741497775314],"p":null,"c":[{"b":[-12.187265518763919,-37.347857970292694,0.25018758498886484,-24.887485498793346],"n":0,"cm":[0,-5.968538966887527,-31.11767173454302],"p":null,"c":null},{"b":[-24.624718622516703,-37.347857970292694,-12.187265518763919,-24.887485498793346],"n":0,"cm":[0,-18.40599207064031,-31.11767173454302],"p":null,"c":null},{"b":[-24.624718622516703,-49.808230441792034,-12.187265518763919,-37.347857970292694],"n":1,"cm":[5.0,-24.513041067034973,-43.57878862067941],"p":[5.0,-24.513041067034973,-43.57878862067941,-4.65213136052377,-1.8323956462596338],"c":null},{"b":[-12.187265518763919,-49.808230441792034,0.25018758498886484,-37.347857970292694],"n":1,"cm":[1.0,-10.539789971538985,-48.87650588325485],"p":[1.0,-10.539789971538985,-48.87650588325485,-2.3776846399799627,-4.39847879985835],"c":null}]}]},{"b":[0.25018758498886484,-49.808230441792034,50.0,0.033259444205345545],"n":5,"cm":[15.0,30.070888743280555,-35.38086309289006],"p":null,"c":[{"b":[25.125093792494432,-24.887485498793346,50.0,0.033259444205345545],"n":1,"cm":[1.0,50.0,0.0],"p":[1.0,50.0,0.0,5.0,0.0],"c":null},{"b":[0.25018758498886484,-24.887485498793346,25.125093792494432,0.033259444205345545],"n":0,"cm":[0,12.687640688741649,-12.427113027294],"p":null,"c":null},{"b":[0.25018758498886484,-49.808230441792034,25.125093792494432,-24.887485498793346],"n":2,"cm":[5.0,13.089311950178336,-47.697732646548786],"p":null,"c":[{"b":[12.687640688741649,-37.347857970292694,25.125093792494432,-24.887485498793346],"n":0,"cm":[0,18.90636724061804,-31.11767173454302],"p":null,"c":null},{"b":[0.25018758498886484,-37.347857970292694,12.687640688741649,-24.887485498793346],"n":0,"cm":[0,6.468914136865257,-31.11767173454302],"p":null,"c":null},{"b":[0.25018758498886484,-49.808230441792034,12.687640688741649,-37.347857970292694],"n":1,"cm":[2.0,4.37494917197232,-49.808230441792034],"p":[2.0,4.37494917197232,-49.808230441792034,1.015024319093752,-4.895888645756587],"c":null},{"b":[12.687640688741649,-49.808230441792034,25.125093792494432,-37.347857970292694],"n":1,"cm":[3.0,18.898887135649012,-46.29073411638662],"p":[3.0,18.898887135649012,-46.29073411638662,3.9303514807051916,-3.0906855611851736],"c":null}]},{"b":[25.125093792494432,-49.808230441792034,50.0,-24.887485498793346],"n":2,"cm":[9.0,37.29075237759074,-32.46936479562299],"p":null,"c":[{"b":[37.56254689624721,-37.347857970292694,50.0,-24.887485498793346],"n":1,"cm":[5.0,41.73563924195799,-27.53427712988188],"p":[5.0,41.73563924195799,-27.53427712988188,3.7137458635183536,3.347848810983005],"c":null},{"b":[25.125093792494432,-37.347857970292694,37.56254689624721,-24.887485498793346],"n":0,"cm":[0,31.343820344370823,-31.11767173454302],"p":null,"c":null},{"b":[25.125093792494432,-49.808230441792034,37.56254689624721,-37.347857970292694],"n":1,"cm":[4.0,31.734643797131696,-38.63822437779938],"p":[4.0,31.734643797131696,-38.63822437779938,4.997172927505024,0.16811523610568346],"c":null},{"b":[37.56254689624721,-49.808230441792034,50.0,-37.347857970292694],"n":0,"cm":[0,43.781273448123606,-43.578044206042364],"p":null,"c":null}]}]}]}#20',
    ('0:Box(-49.49962483002227,-49.808230441792034,50.0,49.874749330202725),10├─ 1:Box(0.25018758498886484,0.033259444205345545,50.0,49.874749330202725),0├─ 2:Box(-49.49962483002227,0.033259444205345545,0.25018758498886484,49.874749330202725),1├─ 3:Box(-49.49962483002227,-49.808230441792034,0.25018758498886484,0.033259444205345545),5│  ├─ 4:Box(-24.624718622516703,-24.887485498793346,0.25018758498886484,0.033259444205345545),0│  ├─ 5:Box(-49.49962483002227,-24.887485498793346,-24.624718622516703,0.033259444205345545),2│  │  ├─ 6:Box(-37.06217172626948,-12.427113027294,-24.624718622516703,0.033259444205345545),0│  │  ├─ 7:Box(-49.49962483002227,-12.427113027294,-37.06217172626948,0.033259444205345545),1│  │  ├─ 8:Box(-49.49962483002227,-24.887485498793346,-37.06217172626948,-12.427113027294),1│  │  └─ 9:Box(-37.06217172626948,-24.887485498793346,-24.624718622516703,-12.427113027294),0│  ├─ 10:Box(-49.49962483002227,-49.808230441792034,-24.624718622516703,-24.887485498793346),1│  └─ 11:Box(-24.624718622516703,-49.808230441792034,0.25018758498886484,-24.887485498793346),2│     ├─ 12:Box(-12.187265518763919,-37.347857970292694,0.25018758498886484,-24.887485498793346),0│     ├─ 13:Box(-24.624718622516703,-37.347857970292694,-12.187265518763919,-24.887485498793346),0│     ├─ 14:Box(-24.624718622516703,-49.808230441792034,-12.187265518763919,-37.347857970292694),1│     └─ 15:Box(-12.187265518763919,-49.808230441792034,0.25018758498886484,-37.347857970292694),1└─ 16:Box(0.25018758498886484,-49.808230441792034,50.0,0.033259444205345545),4   ├─ 17:Box(25.125093792494432,-24.887485498793346,50.0,0.033259444205345545),0   ├─ 18:Box(0.25018758498886484,-24.887485498793346,25.125093792494432,0.033259444205345545),0   ├─ 19:Box(0.25018758498886484,-49.808230441792034,25.125093792494432,-24.887485498793346),2   │  ├─ 20:Box(12.687640688741649,-37.347857970292694,25.125093792494432,-24.887485498793346),0   │  ├─ 21:Box(0.25018758498886484,-37.347857970292694,12.687640688741649,-24.887485498793346),0   │  ├─ 22:Box(0.25018758498886484,-49.808230441792034,12.687640688741649,-37.347857970292694),1   │  └─ 23:Box(12.687640688741649,-49.808230441792034,25.125093792494432,-37.347857970292694),1   └─ 24:Box(25.125093792494432,-49.808230441792034,50.0,-24.887485498793346),2      ├─ 25:Box(37.56254689624721,-37.347857970292694,50.0,-24.887485498793346),1      ├─ 26:Box(25.125093792494432,-37.347857970292694,37.56254689624721,-24.887485498793346),0      ├─ 27:Box(25.125093792494432,-49.808230441792034,37.56254689624721,-37.347857970292694),1      └─ 28:Box(37.56254689624721,-49.808230441792034,50.0,-37.347857970292694),00  -> (30.0, -5.332982975618814, -33.67110875741189)1  -> (0, 25.125093792494432, 24.954004387204037)2  -> (1.0, -49.49962483002227, 7.0560004029933605)3  -> (15.0, -34.103546372516696, -32.43175444879996)4  -> (0, -12.187265518763919, -12.427113027294)5  -> (5.0, -46.65234788820171, -16.430527181710524)6  -> (0, -30.843445174393093, -6.196926791544327)7  -> (2.0, -49.37398849544324, -7.887284707162411)8  -> (3.0, -44.83792081670736, -22.126022164742604)9  -> (0, -30.843445174393093, -18.657299263043672)10 -> (4.0, -36.29661521000701, -34.38830795919869)11 -> (6.0, -22.18416588445231, -44.461741497775314)12 -> (0, -5.968538966887527, -31.11767173454302)13 -> (0, -18.40599207064031, -31.11767173454302)14 -> (5.0, -24.513041067034973, -43.57878862067941)15 -> (1.0, -10.539789971538985, -48.87650588325485)16 -> (14.0, 28.647380796372026, -37.90806759952506)17 -> (0, 37.56254689624721, -12.427113027294)18 -> (0, 12.687640688741649, -12.427113027294)19 -> (5.0, 13.089311950178336, -47.697732646548786)20 -> (0, 18.90636724061804, -31.11767173454302)21 -> (0, 6.468914136865257, -31.11767173454302)22 -> (2.0, 4.37494917197232, -49.808230441792034)23 -> (3.0, 18.898887135649012, -46.29073411638662)24 -> (9.0, 37.29075237759074, -32.46936479562299)25 -> (5.0, 41.73563924195799, -27.53427712988188)26 -> (0, 31.343820344370823, -31.11767173454302)27 -> (4.0, 31.734643797131696, -38.63822437779938)28 -> (0, 43.781273448123606, -43.578044206042364)',10),
    [get_bodies_4()[i] for i in range(10)]
)

# Generated
testMap["GAD", "R"][2] = (
    "03", 1, "R",'{"b":[7,1,99,98],"n":28,"cm":[1480,49.91216216216216,54.32432432432432],"p":null,"c":[{"b":[53.0,49.5,99,98],"n":9,"cm":[390,67.33333333333333,77.52564102564102],"p":null,"c":[{"b":[76.0,73.75,99,98],"n":0,"cm":[0,87.5,85.875],"p":null,"c":null},{"b":[53.0,73.75,76.0,98],"n":5,"cm":[275,63.836363636363636,82.85454545454546],"p":null,"c":[{"b":[64.5,85.875,76.0,98],"n":0,"cm":[0,70.25,91.9375],"p":null,"c":null},{"b":[53.0,85.875,64.5,98],"n":2,"cm":[45,57.0,90.11111111111111],"p":null,"c":[{"b":[58.75,91.9375,64.5,98],"n":0,"cm":[0,61.625,94.96875],"p":null,"c":null},{"b":[53.0,91.9375,58.75,98],"n":0,"cm":[0,55.875,94.96875],"p":null,"c":null},{"b":[53.0,85.875,58.75,91.9375],"n":2,"cm":[45,57.0,90.11111111111111],"p":null,"c":[{"b":[55.875,88.90625,58.75,91.9375],"n":2,"cm":[45,57.0,90.11111111111111],"p":null,"c":[{"b":[57.3125,90.421875,58.75,91.9375],"n":0,"cm":[0,58.03125,91.1796875],"p":null,"c":null},{"b":[55.875,90.421875,57.3125,91.9375],"n":1,"cm":[25,57,91],"p":[25,57,91,50,98],"c":null},{"b":[55.875,88.90625,57.3125,90.421875],"n":1,"cm":[20,57,89],"p":[20,57,89,58,94],"c":null},{"b":[57.3125,88.90625,58.75,90.421875],"n":0,"cm":[0,58.03125,89.6640625],"p":null,"c":null}]},{"b":[53.0,88.90625,55.875,91.9375],"n":0,"cm":[0,54.4375,90.421875],"p":null,"c":null},{"b":[53.0,85.875,55.875,88.90625],"n":0,"cm":[0,54.4375,87.390625],"p":null,"c":null},{"b":[55.875,85.875,58.75,88.90625],"n":0,"cm":[0,57.3125,87.390625],"p":null,"c":null}]},{"b":[58.75,85.875,64.5,91.9375],"n":0,"cm":[0,61.625,88.90625],"p":null,"c":null}]},{"b":[53.0,73.75,64.5,85.875],"n":1,"cm":[90,58,84],"p":[90,58,84,85,72],"c":null},{"b":[64.5,73.75,76.0,85.875],"n":2,"cm":[140,69.78571428571429,79.78571428571429],"p":null,"c":[{"b":[70.25,79.8125,76.0,85.875],"n":1,"cm":[55,71,81],"p":[55,71,81,40,20],"c":null},{"b":[64.5,79.8125,70.25,85.875],"n":0,"cm":[0,67.375,82.84375],"p":null,"c":null},{"b":[64.5,73.75,70.25,79.8125],"n":1,"cm":[85,69,79],"p":[85,69,79,68,8],"c":null},{"b":[70.25,73.75,76.0,79.8125],"n":0,"cm":[0,73.125,76.78125],"p":null,"c":null}]}]},{"b":[53.0,49.5,76.0,73.75],"n":2,"cm":[35,56.142857142857146,60.285714285714285],"p":null,"c":[{"b":[64.5,61.625,76.0,73.75],"n":0,"cm":[0,70.25,67.6875],"p":null,"c":null},{"b":[53.0,61.625,64.5,73.75],"n":0,"cm":[0,58.75,67.6875],"p":null,"c":null},{"b":[53.0,49.5,64.5,61.625],"n":2,"cm":[35,56.142857142857146,60.285714285714285],"p":null,"c":[{"b":[58.75,55.5625,64.5,61.625],"n":1,"cm":[10,64,61],"p":[10,64,61,13,22],"c":null},{"b":[53.0,55.5625,58.75,61.625],"n":1,"cm":[25,53,60],"p":[25,53,60,16,74],"c":null},{"b":[53.0,49.5,58.75,55.5625],"n":0,"cm":[0,55.875,52.53125],"p":null,"c":null},{"b":[58.75,49.5,64.5,55.5625],"n":0,"cm":[0,61.625,52.53125],"p":null,"c":null}]},{"b":[64.5,49.5,76.0,61.625],"n":0,"cm":[0,70.25,55.5625],"p":null,"c":null}]},{"b":[76.0,49.5,99,73.75],"n":2,"cm":[80,84.25,66.75],"p":null,"c":[{"b":[87.5,61.625,99,73.75],"n":1,"cm":[50,88,66],"p":[50,88,66,26,99],"c":null},{"b":[76.0,61.625,87.5,73.75],"n":1,"cm":[30,78,68],"p":[30,78,68,28,91],"c":null},{"b":[76.0,49.5,87.5,61.625],"n":0,"cm":[0,81.75,55.5625],"p":null,"c":null},{"b":[87.5,49.5,99,61.625],"n":0,"cm":[0,93.25,55.5625],"p":null,"c":null}]}]},{"b":[7,49.5,53.0,98],"n":4,"cm":[320,18.90625,80.9375],"p":null,"c":[{"b":[30.0,73.75,53.0,98],"n":1,"cm":[100,32,98],"p":[100,32,98,27,100],"c":null},{"b":[7,73.75,30.0,98],"n":1,"cm":[85,15,92],"p":[85,15,92,98,93],"c":null},{"b":[7,49.5,30.0,73.75],"n":2,"cm":[135,11.666666666666666,61.333333333333336],"p":null,"c":[{"b":[18.5,61.625,30.0,73.75],"n":0,"cm":[0,24.25,67.6875],"p":null,"c":null},{"b":[7,61.625,18.5,73.75],"n":1,"cm":[90,7,65],"p":[90,7,65,17,82],"c":null},{"b":[7,49.5,18.5,61.625],"n":0,"cm":[0,12.75,55.5625],"p":null,"c":null},{"b":[18.5,49.5,30.0,61.625],"n":1,"cm":[45,21,54],"p":[45,21,54,97,14],"c":null}]},{"b":[30.0,49.5,53.0,73.75],"n":0,"cm":[0,41.5,61.625],"p":null,"c":null}]},{"b":[7,1,53.0,49.5],"n":8,"cm":[425,36.1764705882353,24.305882352941175],"p":null,"c":[{"b":[30.0,25.25,53.0,49.5],"n":3,"cm":[205,45.02439024390244,36.41463414634146],"p":null,"c":[{"b":[41.5,37.375,53.0,49.5],"n":1,"cm":[85,50,40],"p":[85,50,40,90,60],"c":null},{"b":[30.0,37.375,41.5,49.5],"n":0,"cm":[0,35.75,43.4375],"p":null,"c":null},{"b":[30.0,25.25,41.5,37.375],"n":1,"cm":[75,37,35],"p":[75,37,35,44,29],"c":null},{"b":[41.5,25.25,53.0,37.375],"n":1,"cm":[45,49,32],"p":[45,49,32,89,11],"c":null}]},{"b":[7,25.25,30.0,49.5],"n":1,"cm":[45,10,30],"p":[45,10,30,18,31],"c":null},{"b":[7,1,30.0,25.25],"n":2,"cm":[65,18.846153846153847,17.923076923076923],"p":null,"c":[{"b":[18.5,13.125,30.0,25.25],"n":0,"cm":[0,24.25,19.1875],"p":null,"c":null},{"b":[7,13.125,18.5,25.25],"n":1,"cm":[50,17,20],"p":[50,17,20,47,35],"c":null},{"b":[7,1,18.5,13.125],"n":0,"cm":[0,12.75,7.0625],"p":null,"c":null},{"b":[18.5,1,30.0,13.125],"n":1,"cm":[15,25,11],"p":[15,25,11,12,7],"c":null}]},{"b":[30.0,1,53.0,25.25],"n":2,"cm":[110,40.63636363636363,3.1818181818181817],"p":null,"c":[{"b":[41.5,13.125,53.0,25.25],"n":0,"cm":[0,47.25,19.1875],"p":null,"c":null},{"b":[30.0,13.125,41.5,25.25],"n":0,"cm":[0,35.75,19.1875],"p":null,"c":null},{"b":[30.0,1,41.5,13.125],"n":1,"cm":[90,39,1],"p":[90,39,1,78,75],"c":null},{"b":[41.5,1,53.0,13.125],"n":1,"cm":[20,48,13],"p":[20,48,13,93,10],"c":null}]}]},{"b":[53.0,1,99,49.5],"n":7,"cm":[345,75.89855072463769,40.391304347826086],"p":null,"c":[{"b":[76.0,25.25,99,49.5],"n":3,"cm":[135,86.66666666666667,41.18518518518518],"p":null,"c":[{"b":[87.5,37.375,99,49.5],"n":1,"cm":[100,88,46],"p":[100,88,46,47,61],"c":null},{"b":[76.0,37.375,87.5,49.5],"n":0,"cm":[0,81.75,43.4375],"p":null,"c":null},{"b":[76.0,25.25,87.5,37.375],"n":2,"cm":[35,82.85714285714286,27.428571428571427],"p":null,"c":[{"b":[81.75,31.3125,87.5,37.375],"n":0,"cm":[0,84.625,34.34375],"p":null,"c":null},{"b":[76.0,31.3125,81.75,37.375],"n":0,"cm":[0,78.875,34.34375],"p":null,"c":null},{"b":[76.0,25.25,81.75,31.3125],"n":0,"cm":[0,78.875,28.28125],"p":null,"c":null},{"b":[81.75,25.25,87.5,31.3125],"n":2,"cm":[35,82.85714285714286,27.428571428571427],"p":null,"c":[{"b":[84.625,28.28125,87.5,31.3125],"n":0,"cm":[0,86.0625,29.796875],"p":null,"c":null},{"b":[81.75,28.28125,84.625,31.3125],"n":0,"cm":[0,83.1875,29.796875],"p":null,"c":null},{"b":[81.75,25.25,84.625,28.28125],"n":2,"cm":[35,82.85714285714286,27.428571428571427],"p":null,"c":[{"b":[83.1875,26.765625,84.625,28.28125],"n":1,"cm":[15,84,28],"p":[15,84,28,91,15],"c":null},{"b":[81.75,26.765625,83.1875,28.28125],"n":1,"cm":[20,82,27],"p":[20,82,27,5,46],"c":null},{"b":[81.75,25.25,83.1875,26.765625],"n":0,"cm":[0,82.46875,26.0078125],"p":null,"c":null},{"b":[83.1875,25.25,84.625,26.765625],"n":0,"cm":[0,83.90625,26.0078125],"p":null,"c":null}]},{"b":[84.625,25.25,87.5,28.28125],"n":0,"cm":[0,86.0625,26.765625],"p":null,"c":null}]}]},{"b":[87.5,25.25,99,37.375],"n":0,"cm":[0,93.25,31.3125],"p":null,"c":null}]},{"b":[53.0,25.25,76.0,49.5],"n":2,"cm":[145,59.37931034482759,48.6551724137931],"p":null,"c":[{"b":[64.5,37.375,76.0,49.5],"n":0,"cm":[0,70.25,43.4375],"p":null,"c":null},{"b":[53.0,37.375,64.5,49.5],"n":2,"cm":[145,59.37931034482759,48.6551724137931],"p":null,"c":[{"b":[58.75,43.4375,64.5,49.5],"n":1,"cm":[50,62,48],"p":[50,62,48,20,85],"c":null},{"b":[53.0,43.4375,58.75,49.5],"n":1,"cm":[95,58,49],"p":[95,58,49,76,30],"c":null},{"b":[53.0,37.375,58.75,43.4375],"n":0,"cm":[0,55.875,40.40625],"p":null,"c":null},{"b":[58.75,37.375,64.5,43.4375],"n":0,"cm":[0,61.625,40.40625],"p":null,"c":null}]},{"b":[53.0,25.25,64.5,37.375],"n":0,"cm":[0,58.75,31.3125],"p":null,"c":null},{"b":[64.5,25.25,76.0,37.375],"n":0,"cm":[0,70.25,31.3125],"p":null,"c":null}]},{"b":[53.0,1,76.0,25.25],"n":0,"cm":[0,64.5,13.125],"p":null,"c":null},{"b":[76.0,1,99,25.25],"n":2,"cm":[65,90.38461538461539,20.307692307692307],"p":null,"c":[{"b":[87.5,13.125,99,25.25],"n":1,"cm":[25,99,16],"p":[25,99,16,33,9],"c":null},{"b":[76.0,13.125,87.5,25.25],"n":1,"cm":[40,85,23],"p":[40,85,23,12,60],"c":null},{"b":[76.0,1,87.5,13.125],"n":0,"cm":[0,81.75,7.0625],"p":null,"c":null},{"b":[87.5,1,99,13.125],"n":0,"cm":[0,93.25,7.0625],"p":null,"c":null}]}]}]}#28',
    ("0:Box(7,1,99,98),10├─ 1:Box(53.0,49.5,99,98),1├─ 2:Box(7,49.5,53.0,98),3│  ├─ 3:Box(30.0,73.75,53.0,98),0│  ├─ 4:Box(7,73.75,30.0,98),1│  ├─ 5:Box(7,49.5,30.0,73.75),2│  │  ├─ 6:Box(18.5,61.625,30.0,73.75),0│  │  ├─ 7:Box(7,61.625,18.5,73.75),1│  │  ├─ 8:Box(7,49.5,18.5,61.625),0│  │  └─ 9:Box(18.5,49.5,30.0,61.625),1│  └─ 10:Box(30.0,49.5,53.0,73.75),0├─ 11:Box(7,1,53.0,49.5),2│  ├─ 12:Box(30.0,25.25,53.0,49.5),1│  ├─ 13:Box(7,25.25,30.0,49.5),1│  ├─ 14:Box(7,1,30.0,25.25),0│  └─ 15:Box(30.0,1,53.0,25.25),0└─ 16:Box(53.0,1,99,49.5),4   ├─ 17:Box(76.0,25.25,99,49.5),1   ├─ 18:Box(53.0,25.25,76.0,49.5),1   ├─ 19:Box(53.0,1,76.0,25.25),0   └─ 20:Box(76.0,1,99,25.25),2      ├─ 21:Box(87.5,13.125,99,25.25),1      ├─ 22:Box(76.0,13.125,87.5,25.25),1      ├─ 23:Box(76.0,1,87.5,13.125),0      └─ 24:Box(87.5,1,99,13.125),00  -> (500, 40.05, 55.21)1  -> (55, 71, 81)2  -> (220, 12.954545454545455, 73.18181818181819)3  -> (0, 41.5, 85.875)4  -> (85, 15, 92)5  -> (135, 11.666666666666666, 61.333333333333336)6  -> (0, 24.25, 67.6875)7  -> (90, 7, 65)8  -> (0, 12.75, 55.5625)9  -> (45, 21, 54)10 -> (0, 41.5, 61.625)11 -> (90, 29.5, 31.0)12 -> (45, 49, 32)13 -> (45, 10, 30)14 -> (0, 18.5, 13.125)15 -> (0, 41.5, 13.125)16 -> (135, 78.62962962962963, 31.555555555555557)17 -> (20, 82, 27)18 -> (50, 62, 48)19 -> (0, 64.5, 13.125)20 -> (65, 90.38461538461539, 20.307692307692307)21 -> (25, 99, 16)22 -> (40, 85, 23)23 -> (0, 81.75, 7.0625)24 -> (0, 93.25, 7.0625)",10),
    [(15,84,28,91,15), (85,69,79,68,8), (20,57,89,58,94), (90,39,1,78,75), (25,53,60,16,74), (30,78,68,28,91), (25,57,91,50,98), (50,17,20,47,35), (95,58,49,76,30), (100,32,98,27,100), (100,88,46,47,61), (90,58,84,85,72), (50,88,66,26,99), (10,64,61,13,22), (85,50,40,90,60), (15,25,11,12,7), (20,48,13,93,10), (75,37,35,44,29)]
) 


testMap["GAD", "R"][3] = (
    "04", 1, "R",'{"b":[1,12,100,98],"n":8,"cm":[305,48.40983606557377,42.9344262295082],"p":null,"c":[{"b":[50.5,55.0,100,98],"n":2,"cm":[30,96.33333333333333,80.0],"p":null,"c":[{"b":[75.25,76.5,100,98],"n":1,"cm":[10,89,98],"p":[10,89,98,97,73],"c":null},{"b":[50.5,76.5,75.25,98],"n":0,"cm":[0,62.875,87.25],"p":null,"c":null},{"b":[50.5,55.0,75.25,76.5],"n":0,"cm":[0,62.875,65.75],"p":null,"c":null},{"b":[75.25,55.0,100,76.5],"n":1,"cm":[20,100,71],"p":[20,100,71,14,91],"c":null}]},{"b":[1,55.0,50.5,98],"n":1,"cm":[10,1,60],"p":[10,1,60,63,92],"c":null},{"b":[1,12,50.5,55.0],"n":2,"cm":[125,16.52,43.16],"p":null,"c":[{"b":[25.75,33.5,50.5,55.0],"n":0,"cm":[0,38.125,44.25],"p":null,"c":null},{"b":[1,33.5,25.75,55.0],"n":1,"cm":[95,11,53],"p":[95,11,53,87,87],"c":null},{"b":[1,12,25.75,33.5],"n":0,"cm":[0,13.375,22.75],"p":null,"c":null},{"b":[25.75,12,50.5,33.5],"n":1,"cm":[30,34,12],"p":[30,34,12,31,99],"c":null}]},{"b":[50.5,12,100,55.0],"n":3,"cm":[140,70.0,33.57142857142857],"p":null,"c":[{"b":[75.25,33.5,100,55.0],"n":0,"cm":[0,87.625,44.25],"p":null,"c":null},{"b":[50.5,33.5,75.25,55.0],"n":1,"cm":[45,56,41],"p":[45,56,41,88,11],"c":null},{"b":[50.5,12,75.25,33.5],"n":1,"cm":[45,74,29],"p":[45,74,29,42,94],"c":null},{"b":[75.25,12,100,33.5],"n":1,"cm":[50,79,31],"p":[50,79,31,45,79],"c":null}]}]}#8',
    ("0:Box(1,12,100,98),4├─ 1:Box(50.5,55.0,100,98),0├─ 2:Box(1,55.0,50.5,98),1├─ 3:Box(1,12,50.5,55.0),2│  ├─ 4:Box(25.75,33.5,50.5,55.0),0│  ├─ 5:Box(1,33.5,25.75,55.0),1│  ├─ 6:Box(1,12,25.75,33.5),0│  └─ 7:Box(25.75,12,50.5,33.5),1└─ 8:Box(50.5,12,100,55.0),10  -> (180, 25.52777777777778, 43.55555555555556)1  -> (0, 75.25, 76.5)2  -> (10, 1, 60)3  -> (125, 16.52, 43.16)4  -> (0, 38.125, 44.25)5  -> (95, 11, 53)6  -> (0, 13.375, 22.75)7  -> (30, 34, 12)8  -> (45, 56, 41)",4),
    [(50,79,31,45,79), (10,89,98,97,73), (20,100,71,14,91), (45,74,29,42,94)]
) 


testMap["GAD", "R"][4] = (
    "05", 1, "R",'{"b":[4,4,98,97],"n":17,"cm":[660,58.59848484848485,41.90909090909091],"p":null,"c":[{"b":[51.0,50.5,98,97],"n":6,"cm":[235,78.59574468085107,77.97872340425532],"p":null,"c":[{"b":[74.5,73.75,98,97],"n":2,"cm":[55,84.0,96.27272727272727],"p":null,"c":[{"b":[86.25,85.375,98,97],"n":0,"cm":[0,92.125,91.1875],"p":null,"c":null},{"b":[74.5,85.375,86.25,97],"n":2,"cm":[55,84.0,96.27272727272727],"p":null,"c":[{"b":[80.375,91.1875,86.25,97],"n":2,"cm":[55,84.0,96.27272727272727],"p":null,"c":[{"b":[83.3125,94.09375,86.25,97],"n":2,"cm":[55,84.0,96.27272727272727],"p":null,"c":[{"b":[84.78125,95.546875,86.25,97],"n":0,"cm":[0,85.515625,96.2734375],"p":null,"c":null},{"b":[83.3125,95.546875,84.78125,97],"n":1,"cm":[35,84,97],"p":[35,84,97,98,71],"c":null},{"b":[83.3125,94.09375,84.78125,95.546875],"n":1,"cm":[20,84,95],"p":[20,84,95,8,48],"c":null},{"b":[84.78125,94.09375,86.25,95.546875],"n":0,"cm":[0,85.515625,94.8203125],"p":null,"c":null}]},{"b":[80.375,94.09375,83.3125,97],"n":0,"cm":[0,81.84375,95.546875],"p":null,"c":null},{"b":[80.375,91.1875,83.3125,94.09375],"n":0,"cm":[0,81.84375,92.640625],"p":null,"c":null},{"b":[83.3125,91.1875,86.25,94.09375],"n":0,"cm":[0,84.78125,92.640625],"p":null,"c":null}]},{"b":[74.5,91.1875,80.375,97],"n":0,"cm":[0,77.4375,94.09375],"p":null,"c":null},{"b":[74.5,85.375,80.375,91.1875],"n":0,"cm":[0,77.4375,88.28125],"p":null,"c":null},{"b":[80.375,85.375,86.25,91.1875],"n":0,"cm":[0,83.3125,88.28125],"p":null,"c":null}]},{"b":[74.5,73.75,86.25,85.375],"n":0,"cm":[0,80.375,79.5625],"p":null,"c":null},{"b":[86.25,73.75,98,85.375],"n":0,"cm":[0,92.125,79.5625],"p":null,"c":null}]},{"b":[51.0,73.75,74.5,97],"n":2,"cm":[105,67.52380952380952,81.38095238095238],"p":null,"c":[{"b":[62.75,85.375,74.5,97],"n":0,"cm":[0,68.625,91.1875],"p":null,"c":null},{"b":[51.0,85.375,62.75,97],"n":0,"cm":[0,56.875,91.1875],"p":null,"c":null},{"b":[51.0,73.75,62.75,85.375],"n":0,"cm":[0,56.875,79.5625],"p":null,"c":null},{"b":[62.75,73.75,74.5,85.375],"n":2,"cm":[105,67.52380952380952,81.38095238095238],"p":null,"c":[{"b":[68.625,79.5625,74.5,85.375],"n":1,"cm":[40,70,82],"p":[40,70,82,80,99],"c":null},{"b":[62.75,79.5625,68.625,85.375],"n":1,"cm":[65,66,81],"p":[65,66,81,54,30],"c":null},{"b":[62.75,73.75,68.625,79.5625],"n":0,"cm":[0,65.6875,76.65625],"p":null,"c":null},{"b":[68.625,73.75,74.5,79.5625],"n":0,"cm":[0,71.5625,76.65625],"p":null,"c":null}]}]},{"b":[51.0,50.5,74.5,73.75],"n":0,"cm":[0,62.75,62.125],"p":null,"c":null},{"b":[74.5,50.5,98,73.75],"n":2,"cm":[75,90.13333333333334,59.8],"p":null,"c":[{"b":[86.25,62.125,98,73.75],"n":0,"cm":[0,92.125,67.9375],"p":null,"c":null},{"b":[74.5,62.125,86.25,73.75],"n":1,"cm":[5,78,71],"p":[5,78,71,52,25],"c":null},{"b":[74.5,50.5,86.25,62.125],"n":0,"cm":[0,80.375,56.3125],"p":null,"c":null},{"b":[86.25,50.5,98,62.125],"n":1,"cm":[70,91,59],"p":[70,91,59,39,64],"c":null}]}]},{"b":[4,50.5,51.0,97],"n":1,"cm":[30,43,74],"p":[30,43,74,74,37],"c":null},{"b":[4,4,51.0,50.5],"n":5,"cm":[165,16.636363636363637,19.12121212121212],"p":null,"c":[{"b":[27.5,27.25,51.0,50.5],"n":1,"cm":[5,51,49],"p":[5,51,49,24,90],"c":null},{"b":[4,27.25,27.5,50.5],"n":1,"cm":[30,17,41],"p":[30,17,41,61,38],"c":null},{"b":[4,4,27.5,27.25],"n":2,"cm":[85,4.235294117647059,9.705882352941176],"p":null,"c":[{"b":[15.75,15.625,27.5,27.25],"n":0,"cm":[0,21.625,21.4375],"p":null,"c":null},{"b":[4,15.625,15.75,27.25],"n":0,"cm":[0,9.875,21.4375],"p":null,"c":null},{"b":[4,4,15.75,15.625],"n":2,"cm":[85,4.235294117647059,9.705882352941176],"p":null,"c":[{"b":[9.875,9.8125,15.75,15.625],"n":0,"cm":[0,12.8125,12.71875],"p":null,"c":null},{"b":[4,9.8125,9.875,15.625],"n":1,"cm":[10,6,15],"p":[10,6,15,73,30],"c":null},{"b":[4,4,9.875,9.8125],"n":1,"cm":[75,4,9],"p":[75,4,9,29,24],"c":null},{"b":[9.875,4,15.75,9.8125],"n":0,"cm":[0,12.8125,6.90625],"p":null,"c":null}]},{"b":[15.75,4,27.5,15.625],"n":0,"cm":[0,21.625,9.8125],"p":null,"c":null}]},{"b":[27.5,4,51.0,27.25],"n":1,"cm":[45,36,19],"p":[45,36,19,22,45],"c":null}]},{"b":[51.0,4,98,50.5],"n":5,"cm":[230,70.30434782608695,17.217391304347824],"p":null,"c":[{"b":[74.5,27.25,98,50.5],"n":0,"cm":[0,86.25,38.875],"p":null,"c":null},{"b":[51.0,27.25,74.5,50.5],"n":1,"cm":[40,52,30],"p":[40,52,30,12,28],"c":null},{"b":[51.0,4,74.5,27.25],"n":3,"cm":[150,67.8,17.333333333333332],"p":null,"c":[{"b":[62.75,15.625,74.5,27.25],"n":2,"cm":[140,68.71428571428571,17.857142857142858],"p":null,"c":[{"b":[68.625,21.4375,74.5,27.25],"n":0,"cm":[0,71.5625,24.34375],"p":null,"c":null},{"b":[62.75,21.4375,68.625,27.25],"n":0,"cm":[0,65.6875,24.34375],"p":null,"c":null},{"b":[62.75,15.625,68.625,21.4375],"n":1,"cm":[40,68,20],"p":[40,68,20,78,82],"c":null},{"b":[68.625,15.625,74.5,21.4375],"n":1,"cm":[100,69,17],"p":[100,69,17,31,56],"c":null}]},{"b":[51.0,15.625,62.75,27.25],"n":0,"cm":[0,56.875,21.4375],"p":null,"c":null},{"b":[51.0,4,62.75,15.625],"n":1,"cm":[10,55,10],"p":[10,55,10,50,56],"c":null},{"b":[62.75,4,74.5,15.625],"n":0,"cm":[0,68.625,9.8125],"p":null,"c":null}]},{"b":[74.5,4,98,27.25],"n":1,"cm":[40,98,4],"p":[40,98,4,100,85],"c":null}]}]}#17',
    ("0:Box(4,4,98,97),9├─ 1:Box(51.0,50.5,98,97),3│  ├─ 2:Box(74.5,73.75,98,97),1│  ├─ 3:Box(51.0,73.75,74.5,97),2│  │  ├─ 4:Box(62.75,85.375,74.5,97),0│  │  ├─ 5:Box(51.0,85.375,62.75,97),0│  │  ├─ 6:Box(51.0,73.75,62.75,85.375),0│  │  └─ 7:Box(62.75,73.75,74.5,85.375),2│  │     ├─ 8:Box(68.625,79.5625,74.5,85.375),1│  │     ├─ 9:Box(62.75,79.5625,68.625,85.375),1│  │     ├─ 10:Box(62.75,73.75,68.625,79.5625),0│  │     └─ 11:Box(68.625,73.75,74.5,79.5625),0│  ├─ 12:Box(51.0,50.5,74.5,73.75),0│  └─ 13:Box(74.5,50.5,98,73.75),0├─ 14:Box(4,50.5,51.0,97),1├─ 15:Box(4,4,51.0,50.5),2│  ├─ 16:Box(27.5,27.25,51.0,50.5),0│  ├─ 17:Box(4,27.25,27.5,50.5),0│  ├─ 18:Box(4,4,27.5,27.25),1│  └─ 19:Box(27.5,4,51.0,27.25),1└─ 20:Box(51.0,4,98,50.5),3   ├─ 21:Box(74.5,27.25,98,50.5),0   ├─ 22:Box(51.0,27.25,74.5,50.5),0   ├─ 23:Box(51.0,4,74.5,27.25),2   │  ├─ 24:Box(62.75,15.625,74.5,27.25),2   │  │  ├─ 25:Box(68.625,21.4375,74.5,27.25),0   │  │  ├─ 26:Box(62.75,21.4375,68.625,27.25),0   │  │  ├─ 27:Box(62.75,15.625,68.625,21.4375),1   │  │  └─ 28:Box(68.625,15.625,74.5,21.4375),1   │  ├─ 29:Box(51.0,15.625,62.75,27.25),0   │  ├─ 30:Box(51.0,4,62.75,15.625),0   │  └─ 31:Box(62.75,4,74.5,15.625),0   └─ 32:Box(74.5,4,98,27.25),10  -> (390, 64.82051282051282, 41.87179487179487)1  -> (125, 70.16, 83.56)2  -> (20, 84, 95)3  -> (105, 67.52380952380952, 81.38095238095238)4  -> (0, 68.625, 91.1875)5  -> (0, 56.875, 91.1875)6  -> (0, 56.875, 79.5625)7  -> (105, 67.52380952380952, 81.38095238095238)8  -> (40, 70, 82)9  -> (65, 66, 81)10 -> (0, 65.6875, 76.65625)11 -> (0, 71.5625, 76.65625)12 -> (0, 62.75, 62.125)13 -> (0, 86.25, 62.125)14 -> (30, 43, 74)15 -> (55, 30.545454545454547, 18.272727272727273)16 -> (0, 39.25, 38.875)17 -> (0, 15.75, 38.875)18 -> (10, 6, 15)19 -> (45, 36, 19)20 -> (180, 75.22222222222223, 14.777777777777779)21 -> (0, 86.25, 38.875)22 -> (0, 62.75, 38.875)23 -> (140, 68.71428571428571, 17.857142857142858)24 -> (140, 68.71428571428571, 17.857142857142858)25 -> (0, 71.5625, 24.34375)26 -> (0, 65.6875, 24.34375)27 -> (40, 68, 20)28 -> (100, 69, 17)29 -> (0, 56.875, 21.4375)30 -> (0, 56.875, 9.8125)31 -> (0, 68.625, 9.8125)32 -> (40, 98, 4)",9),
    [(5,51,49,24,90), (75,4,9,29,24), (35,84,97,98,71), (30,17,41,61,38), (10,55,10,50,56), (5,78,71,52,25), (40,52,30,12,28), (70,91,59,39,64)]
) 


testMap["GAD", "R"][5] = (
    "06", 1, "R",'{"b":[1,11,87,98],"n":26,"cm":[1335,50.87265917602996,54.254681647940075],"p":null,"c":[{"b":[44.0,54.5,87,98],"n":8,"cm":[305,58.68852459016394,76.8688524590164],"p":null,"c":[{"b":[65.5,76.25,87,98],"n":1,"cm":[20,84,97],"p":[20,84,97,34,10],"c":null},{"b":[44.0,76.25,65.5,98],"n":1,"cm":[55,52,98],"p":[55,52,98,65,7],"c":null},{"b":[44.0,54.5,65.5,76.25],"n":5,"cm":[200,55.1,71.575],"p":null,"c":[{"b":[54.75,65.375,65.5,76.25],"n":3,"cm":[90,61.166666666666664,73.44444444444444],"p":null,"c":[{"b":[60.125,70.8125,65.5,76.25],"n":1,"cm":[55,63,75],"p":[55,63,75,97,86],"c":null},{"b":[54.75,70.8125,60.125,76.25],"n":1,"cm":[25,58,73],"p":[25,58,73,64,23],"c":null},{"b":[54.75,65.375,60.125,70.8125],"n":1,"cm":[10,59,66],"p":[10,59,66,48,92],"c":null},{"b":[60.125,65.375,65.5,70.8125],"n":0,"cm":[0,62.8125,68.09375],"p":null,"c":null}]},{"b":[44.0,65.375,54.75,76.25],"n":1,"cm":[85,49,73],"p":[85,49,73,96,76],"c":null},{"b":[44.0,54.5,54.75,65.375],"n":1,"cm":[25,54,60],"p":[25,54,60,62,27],"c":null},{"b":[54.75,54.5,65.5,65.375],"n":0,"cm":[0,60.125,59.9375],"p":null,"c":null}]},{"b":[65.5,54.5,87,76.25],"n":1,"cm":[30,78,60],"p":[30,78,60,40,54],"c":null}]},{"b":[1,54.5,44.0,98],"n":5,"cm":[260,33.67307692307692,79.40384615384616],"p":null,"c":[{"b":[22.5,76.25,44.0,98],"n":3,"cm":[175,33.77142857142857,81.54285714285714],"p":null,"c":[{"b":[33.25,87.125,44.0,98],"n":0,"cm":[0,38.625,92.5625],"p":null,"c":null},{"b":[22.5,87.125,33.25,98],"n":0,"cm":[0,27.875,92.5625],"p":null,"c":null},{"b":[22.5,76.25,33.25,87.125],"n":2,"cm":[80,24.0,85.75],"p":null,"c":[{"b":[27.875,81.6875,33.25,87.125],"n":0,"cm":[0,30.5625,84.40625],"p":null,"c":null},{"b":[22.5,81.6875,27.875,87.125],"n":2,"cm":[80,24.0,85.75],"p":null,"c":[{"b":[25.1875,84.40625,27.875,87.125],"n":0,"cm":[0,26.53125,85.765625],"p":null,"c":null},{"b":[22.5,84.40625,25.1875,87.125],"n":1,"cm":[60,24,87],"p":[60,24,87,17,38],"c":null},{"b":[22.5,81.6875,25.1875,84.40625],"n":1,"cm":[20,24,82],"p":[20,24,82,39,34],"c":null},{"b":[25.1875,81.6875,27.875,84.40625],"n":0,"cm":[0,26.53125,83.046875],"p":null,"c":null}]},{"b":[22.5,76.25,27.875,81.6875],"n":0,"cm":[0,25.1875,78.96875],"p":null,"c":null},{"b":[27.875,76.25,33.25,81.6875],"n":0,"cm":[0,30.5625,78.96875],"p":null,"c":null}]},{"b":[33.25,76.25,44.0,87.125],"n":1,"cm":[95,42,78],"p":[95,42,78,54,98],"c":null}]},{"b":[1,76.25,22.5,98],"n":0,"cm":[0,11.75,87.125],"p":null,"c":null},{"b":[1,54.5,22.5,76.25],"n":0,"cm":[0,11.75,65.375],"p":null,"c":null},{"b":[22.5,54.5,44.0,76.25],"n":2,"cm":[85,33.470588235294116,75.0],"p":null,"c":[{"b":[33.25,65.375,44.0,76.25],"n":1,"cm":[60,37,75],"p":[60,37,75,28,23],"c":null},{"b":[22.5,65.375,33.25,76.25],"n":1,"cm":[25,25,75],"p":[25,25,75,89,29],"c":null},{"b":[22.5,54.5,33.25,65.375],"n":0,"cm":[0,27.875,59.9375],"p":null,"c":null},{"b":[33.25,54.5,44.0,65.375],"n":0,"cm":[0,38.625,59.9375],"p":null,"c":null}]}]},{"b":[1,11,44.0,54.5],"n":4,"cm":[325,29.353846153846153,35.353846153846156],"p":null,"c":[{"b":[22.5,32.75,44.0,54.5],"n":2,"cm":[180,37.416666666666664,45.94444444444444],"p":null,"c":[{"b":[33.25,43.625,44.0,54.5],"n":2,"cm":[180,37.416666666666664,45.94444444444444],"p":null,"c":[{"b":[38.625,49.0625,44.0,54.5],"n":0,"cm":[0,41.3125,51.78125],"p":null,"c":null},{"b":[33.25,49.0625,38.625,54.5],"n":0,"cm":[0,35.9375,51.78125],"p":null,"c":null},{"b":[33.25,43.625,38.625,49.0625],"n":1,"cm":[95,36,45],"p":[95,36,45,43,9],"c":null},{"b":[38.625,43.625,44.0,49.0625],"n":1,"cm":[85,39,47],"p":[85,39,47,20,55],"c":null}]},{"b":[22.5,43.625,33.25,54.5],"n":0,"cm":[0,27.875,49.0625],"p":null,"c":null},{"b":[22.5,32.75,33.25,43.625],"n":0,"cm":[0,27.875,38.1875],"p":null,"c":null},{"b":[33.25,32.75,44.0,43.625],"n":0,"cm":[0,38.625,38.1875],"p":null,"c":null}]},{"b":[1,32.75,22.5,54.5],"n":0,"cm":[0,11.75,43.625],"p":null,"c":null},{"b":[1,11,22.5,32.75],"n":1,"cm":[75,1,14],"p":[75,1,14,77,12],"c":null},{"b":[22.5,11,44.0,32.75],"n":1,"cm":[70,39,31],"p":[70,39,31,55,33],"c":null}]},{"b":[44.0,11,87,54.5],"n":9,"cm":[445,71.28089887640449,37.86516853932584],"p":null,"c":[{"b":[65.5,32.75,87,54.5],"n":4,"cm":[215,81.20930232558139,43.95348837209303],"p":null,"c":[{"b":[76.25,43.625,87,54.5],"n":1,"cm":[90,85,51],"p":[90,85,51,37,18],"c":null},{"b":[65.5,43.625,76.25,54.5],"n":1,"cm":[25,66,54],"p":[25,66,54,21,16],"c":null},{"b":[65.5,32.75,76.25,43.625],"n":1,"cm":[5,74,37],"p":[5,74,37,85,96],"c":null},{"b":[76.25,32.75,87,43.625],"n":1,"cm":[95,82,35],"p":[95,82,35,43,10],"c":null}]},{"b":[44.0,32.75,65.5,54.5],"n":2,"cm":[155,57.58064516129032,39.25806451612903],"p":null,"c":[{"b":[54.75,43.625,65.5,54.5],"n":1,"cm":[55,55,47],"p":[55,55,47,60,94],"c":null},{"b":[44.0,43.625,54.75,54.5],"n":0,"cm":[0,49.375,49.0625],"p":null,"c":null},{"b":[44.0,32.75,54.75,43.625],"n":0,"cm":[0,49.375,38.1875],"p":null,"c":null},{"b":[54.75,32.75,65.5,43.625],"n":1,"cm":[100,59,35],"p":[100,59,35,97,61],"c":null}]},{"b":[44.0,11,65.5,32.75],"n":1,"cm":[30,54,14],"p":[30,54,14,25,11],"c":null},{"b":[65.5,11,87,32.75],"n":2,"cm":[45,82.55555555555556,19.88888888888889],"p":null,"c":[{"b":[76.25,21.875,87,32.75],"n":1,"cm":[25,87,27],"p":[25,87,27,73,51],"c":null},{"b":[65.5,21.875,76.25,32.75],"n":0,"cm":[0,70.875,27.3125],"p":null,"c":null},{"b":[65.5,11,76.25,21.875],"n":0,"cm":[0,70.875,16.4375],"p":null,"c":null},{"b":[76.25,11,87,21.875],"n":1,"cm":[20,77,11],"p":[20,77,11,58,83],"c":null}]}]}]}#26',
    ("0:Box(1,11,87,98),14├─ 1:Box(44.0,54.5,87,98),3│  ├─ 2:Box(65.5,76.25,87,98),0│  ├─ 3:Box(44.0,76.25,65.5,98),0│  ├─ 4:Box(44.0,54.5,65.5,76.25),2│  │  ├─ 5:Box(54.75,65.375,65.5,76.25),1│  │  ├─ 6:Box(44.0,65.375,54.75,76.25),0│  │  ├─ 7:Box(44.0,54.5,54.75,65.375),1│  │  └─ 8:Box(54.75,54.5,65.5,65.375),0│  └─ 9:Box(65.5,54.5,87,76.25),1├─ 10:Box(1,54.5,44.0,98),4│  ├─ 11:Box(22.5,76.25,44.0,98),3│  │  ├─ 12:Box(33.25,87.125,44.0,98),0│  │  ├─ 13:Box(22.5,87.125,33.25,98),0│  │  ├─ 14:Box(22.5,76.25,33.25,87.125),2│  │  │  ├─ 15:Box(27.875,81.6875,33.25,87.125),0│  │  │  ├─ 16:Box(22.5,81.6875,27.875,87.125),2│  │  │  │  ├─ 17:Box(25.1875,84.40625,27.875,87.125),0│  │  │  │  ├─ 18:Box(22.5,84.40625,25.1875,87.125),1│  │  │  │  ├─ 19:Box(22.5,81.6875,25.1875,84.40625),1│  │  │  │  └─ 20:Box(25.1875,81.6875,27.875,84.40625),0│  │  │  ├─ 21:Box(22.5,76.25,27.875,81.6875),0│  │  │  └─ 22:Box(27.875,76.25,33.25,81.6875),0│  │  └─ 23:Box(33.25,76.25,44.0,87.125),1│  ├─ 24:Box(1,76.25,22.5,98),0│  ├─ 25:Box(1,54.5,22.5,76.25),0│  └─ 26:Box(22.5,54.5,44.0,76.25),1├─ 27:Box(1,11,44.0,54.5),1└─ 28:Box(44.0,11,87,54.5),6   ├─ 29:Box(65.5,32.75,87,54.5),3   │  ├─ 30:Box(76.25,43.625,87,54.5),0   │  ├─ 31:Box(65.5,43.625,76.25,54.5),1   │  ├─ 32:Box(65.5,32.75,76.25,43.625),1   │  └─ 33:Box(76.25,32.75,87,43.625),1   ├─ 34:Box(44.0,32.75,65.5,54.5),2   │  ├─ 35:Box(54.75,43.625,65.5,54.5),1   │  ├─ 36:Box(44.0,43.625,54.75,54.5),0   │  ├─ 37:Box(44.0,32.75,54.75,43.625),0   │  └─ 38:Box(54.75,32.75,65.5,43.625),1   ├─ 39:Box(44.0,11,65.5,32.75),1   └─ 40:Box(65.5,11,87,32.75),00  -> (725, 52.92413793103448, 54.8)1  -> (110, 65.04545454545455, 67.5)2  -> (0, 76.25, 87.125)3  -> (0, 54.75, 87.125)4  -> (80, 60.1875, 70.3125)5  -> (55, 63, 75)6  -> (0, 49.375, 70.8125)7  -> (25, 54, 60)8  -> (0, 60.125, 59.9375)9  -> (30, 78, 60)10 -> (235, 34.59574468085106, 79.87234042553192)11 -> (175, 33.77142857142857, 81.54285714285714)12 -> (0, 38.625, 92.5625)13 -> (0, 27.875, 92.5625)14 -> (80, 24.0, 85.75)15 -> (0, 30.5625, 84.40625)16 -> (80, 24.0, 85.75)17 -> (0, 26.53125, 85.765625)18 -> (60, 24, 87)19 -> (20, 24, 82)20 -> (0, 26.53125, 83.046875)21 -> (0, 25.1875, 78.96875)22 -> (0, 30.5625, 78.96875)23 -> (95, 42, 78)24 -> (0, 11.75, 87.125)25 -> (0, 11.75, 65.375)26 -> (60, 37, 75)27 -> (70, 39, 31)28 -> (310, 65.66129032258064, 36.66129032258065)29 -> (125, 78.48, 38.88)30 -> (0, 81.625, 49.0625)31 -> (25, 66, 54)32 -> (5, 74, 37)33 -> (95, 82, 35)34 -> (155, 57.58064516129032, 39.25806451612903)35 -> (55, 55, 47)36 -> (0, 49.375, 49.0625)37 -> (0, 49.375, 38.1875)38 -> (100, 59, 35)39 -> (30, 54, 14)40 -> (0, 76.25, 21.875)",14),
    [(20,77,11,58,83), (25,25,75,89,29), (25,58,73,64,23), (75,1,14,77,12), (85,39,47,20,55), (85,49,73,96,76), (95,36,45,43,9), (10,59,66,48,92), (25,87,27,73,51), (20,84,97,34,10), (90,85,51,37,18), (55,52,98,65,7)]
) 


testMap["GAD", "R"][6] = (
    "07", 1, "R",'{"b":[8,5,95,82],"n":12,"cm":[720,60.666666666666664,43.71527777777778],"p":null,"c":[{"b":[51.5,43.5,95,82],"n":4,"cm":[195,67.74358974358974,66.66666666666667],"p":null,"c":[{"b":[73.25,62.75,95,82],"n":1,"cm":[30,77,82],"p":[30,77,82,15,30],"c":null},{"b":[51.5,62.75,73.25,82],"n":1,"cm":[70,64,72],"p":[70,64,72,74,8],"c":null},{"b":[51.5,43.5,73.25,62.75],"n":2,"cm":[95,67.57894736842105,57.89473684210526],"p":null,"c":[{"b":[62.375,53.125,73.25,62.75],"n":1,"cm":[80,69,59],"p":[80,69,59,90,50],"c":null},{"b":[51.5,53.125,62.375,62.75],"n":0,"cm":[0,56.9375,57.9375],"p":null,"c":null},{"b":[51.5,43.5,62.375,53.125],"n":1,"cm":[15,60,52],"p":[15,60,52,55,88],"c":null},{"b":[62.375,43.5,73.25,53.125],"n":0,"cm":[0,67.8125,48.3125],"p":null,"c":null}]},{"b":[73.25,43.5,95,62.75],"n":0,"cm":[0,84.125,53.125],"p":null,"c":null}]},{"b":[8,43.5,51.5,82],"n":2,"cm":[170,40.55882352941177,63.029411764705884],"p":null,"c":[{"b":[29.75,62.75,51.5,82],"n":1,"cm":[75,40,77],"p":[75,40,77,76,92],"c":null},{"b":[8,62.75,29.75,82],"n":0,"cm":[0,18.875,72.375],"p":null,"c":null},{"b":[8,43.5,29.75,62.75],"n":0,"cm":[0,18.875,53.125],"p":null,"c":null},{"b":[29.75,43.5,51.5,62.75],"n":1,"cm":[95,41,52],"p":[95,41,52,92,84],"c":null}]},{"b":[8,5,51.5,43.5],"n":2,"cm":[75,15.2,36.2],"p":null,"c":[{"b":[29.75,24.25,51.5,43.5],"n":0,"cm":[0,40.625,33.875],"p":null,"c":null},{"b":[8,24.25,29.75,43.5],"n":2,"cm":[75,15.2,36.2],"p":null,"c":[{"b":[18.875,33.875,29.75,43.5],"n":0,"cm":[0,24.3125,38.6875],"p":null,"c":null},{"b":[8,33.875,18.875,43.5],"n":1,"cm":[60,17,39],"p":[60,17,39,61,13],"c":null},{"b":[8,24.25,18.875,33.875],"n":1,"cm":[15,8,25],"p":[15,8,25,33,41],"c":null},{"b":[18.875,24.25,29.75,33.875],"n":0,"cm":[0,24.3125,29.0625],"p":null,"c":null}]},{"b":[8,5,29.75,24.25],"n":0,"cm":[0,18.875,14.625],"p":null,"c":null},{"b":[29.75,5,51.5,24.25],"n":0,"cm":[0,40.625,14.625],"p":null,"c":null}]},{"b":[51.5,5,95,43.5],"n":4,"cm":[280,80.125,18.017857142857142],"p":null,"c":[{"b":[73.25,24.25,95,43.5],"n":0,"cm":[0,84.125,33.875],"p":null,"c":null},{"b":[51.5,24.25,73.25,43.5],"n":1,"cm":[95,72,28],"p":[95,72,28,73,28],"c":null},{"b":[51.5,5,73.25,24.25],"n":0,"cm":[0,62.375,14.625],"p":null,"c":null},{"b":[73.25,5,95,24.25],"n":3,"cm":[185,84.29729729729729,12.891891891891891],"p":null,"c":[{"b":[84.125,14.625,95,24.25],"n":0,"cm":[0,89.5625,19.4375],"p":null,"c":null},{"b":[73.25,14.625,84.125,24.25],"n":2,"cm":[105,76.14285714285714,18.904761904761905],"p":null,"c":[{"b":[78.6875,19.4375,84.125,24.25],"n":0,"cm":[0,81.40625,21.84375],"p":null,"c":null},{"b":[73.25,19.4375,78.6875,24.25],"n":1,"cm":[40,78,22],"p":[40,78,22,71,29],"c":null},{"b":[73.25,14.625,78.6875,19.4375],"n":1,"cm":[65,75,17],"p":[65,75,17,28,58],"c":null},{"b":[78.6875,14.625,84.125,19.4375],"n":0,"cm":[0,81.40625,17.03125],"p":null,"c":null}]},{"b":[73.25,5,84.125,14.625],"n":0,"cm":[0,78.6875,9.8125],"p":null,"c":null},{"b":[84.125,5,95,14.625],"n":1,"cm":[80,95,5],"p":[80,95,5,22,85],"c":null}]}]}]}#12',
    ("0:Box(8,5,95,82),7├─ 1:Box(51.5,43.5,95,82),2│  ├─ 2:Box(73.25,62.75,95,82),0│  ├─ 3:Box(51.5,62.75,73.25,82),1│  ├─ 4:Box(51.5,43.5,73.25,62.75),1│  └─ 5:Box(73.25,43.5,95,62.75),0├─ 6:Box(8,43.5,51.5,82),1├─ 7:Box(8,5,51.5,43.5),2│  ├─ 8:Box(29.75,24.25,51.5,43.5),0│  ├─ 9:Box(8,24.25,29.75,43.5),2│  │  ├─ 10:Box(18.875,33.875,29.75,43.5),0│  │  ├─ 11:Box(8,33.875,18.875,43.5),1│  │  ├─ 12:Box(8,24.25,18.875,33.875),1│  │  └─ 13:Box(18.875,24.25,29.75,33.875),0│  ├─ 14:Box(8,5,29.75,24.25),0│  └─ 15:Box(29.75,5,51.5,24.25),0└─ 16:Box(51.5,5,95,43.5),2   ├─ 17:Box(73.25,24.25,95,43.5),0   ├─ 18:Box(51.5,24.25,73.25,43.5),1   ├─ 19:Box(51.5,5,73.25,24.25),0   └─ 20:Box(73.25,5,95,24.25),10  -> (410, 58.4390243902439, 42.36585365853659)1  -> (85, 63.294117647058826, 68.47058823529412)2  -> (0, 84.125, 72.375)3  -> (70, 64, 72)4  -> (15, 60, 52)5  -> (0, 84.125, 53.125)6  -> (75, 40, 77)7  -> (75, 15.2, 36.2)8  -> (0, 40.625, 33.875)9  -> (75, 15.2, 36.2)10 -> (0, 24.3125, 38.6875)11 -> (60, 17, 39)12 -> (15, 8, 25)13 -> (0, 24.3125, 29.0625)14 -> (0, 18.875, 14.625)15 -> (0, 40.625, 14.625)16 -> (175, 82.51428571428572, 17.485714285714284)17 -> (0, 84.125, 33.875)18 -> (95, 72, 28)19 -> (0, 62.375, 14.625)20 -> (80, 95, 5)",7),
    [(80,69,59,90,50), (95,41,52,92,84), (30,77,82,15,30), (40,78,22,71,29), (65,75,17,28,58)]
) 


testMap["GAD", "R"][7] = (
    "08", 1, "R",'{"b":[3,1,100,99],"n":28,"cm":[1600,53.115625,64.609375],"p":null,"c":[{"b":[51.5,50.0,100,99],"n":11,"cm":[830,71.92168674698796,83.46987951807229],"p":null,"c":[{"b":[75.75,74.5,100,99],"n":4,"cm":[285,83.6842105263158,91.47368421052632],"p":null,"c":[{"b":[87.875,86.75,100,99],"n":1,"cm":[60,89,95],"p":[60,89,95,18,6],"c":null},{"b":[75.75,86.75,87.875,99],"n":2,"cm":[195,79.53846153846153,91.53846153846153],"p":null,"c":[{"b":[81.8125,92.875,87.875,99],"n":0,"cm":[0,84.84375,95.9375],"p":null,"c":null},{"b":[75.75,92.875,81.8125,99],"n":1,"cm":[100,81,93],"p":[100,81,93,72,40],"c":null},{"b":[75.75,86.75,81.8125,92.875],"n":1,"cm":[95,78,90],"p":[95,78,90,59,67],"c":null},{"b":[81.8125,86.75,87.875,92.875],"n":0,"cm":[0,84.84375,89.8125],"p":null,"c":null}]},{"b":[75.75,74.5,87.875,86.75],"n":0,"cm":[0,81.8125,80.625],"p":null,"c":null},{"b":[87.875,74.5,100,86.75],"n":1,"cm":[30,100,84],"p":[30,100,84,71,16],"c":null}]},{"b":[51.5,74.5,75.75,99],"n":4,"cm":[330,58.25757575757576,85.27272727272727],"p":null,"c":[{"b":[63.625,86.75,75.75,99],"n":0,"cm":[0,69.6875,92.875],"p":null,"c":null},{"b":[51.5,86.75,63.625,99],"n":1,"cm":[90,54,99],"p":[90,54,99,81,61],"c":null},{"b":[51.5,74.5,63.625,86.75],"n":3,"cm":[240,59.854166666666664,80.125],"p":null,"c":[{"b":[57.5625,80.625,63.625,86.75],"n":1,"cm":[75,63,86],"p":[75,63,86,88,12],"c":null},{"b":[51.5,80.625,57.5625,86.75],"n":0,"cm":[0,54.53125,83.6875],"p":null,"c":null},{"b":[51.5,74.5,57.5625,80.625],"n":1,"cm":[85,56,76],"p":[85,56,76,87,85],"c":null},{"b":[57.5625,74.5,63.625,80.625],"n":1,"cm":[80,61,79],"p":[80,61,79,16,98],"c":null}]},{"b":[63.625,74.5,75.75,86.75],"n":0,"cm":[0,69.6875,80.625],"p":null,"c":null}]},{"b":[51.5,50.0,75.75,74.5],"n":1,"cm":[95,66,74],"p":[95,66,74,18,12],"c":null},{"b":[75.75,50.0,100,74.5],"n":2,"cm":[120,86.25,67.0],"p":null,"c":[{"b":[87.875,62.25,100,74.5],"n":1,"cm":[30,90,73],"p":[30,90,73,51,14],"c":null},{"b":[75.75,62.25,87.875,74.5],"n":1,"cm":[90,85,65],"p":[90,85,65,39,46],"c":null},{"b":[75.75,50.0,87.875,62.25],"n":0,"cm":[0,81.8125,56.125],"p":null,"c":null},{"b":[87.875,50.0,100,62.25],"n":0,"cm":[0,93.9375,56.125],"p":null,"c":null}]}]},{"b":[3,50.0,51.5,99],"n":8,"cm":[400,24.4875,69.9],"p":null,"c":[{"b":[27.25,74.5,51.5,99],"n":0,"cm":[0,39.375,86.75],"p":null,"c":null},{"b":[3,74.5,27.25,99],"n":3,"cm":[125,19.32,92.6],"p":null,"c":[{"b":[15.125,86.75,27.25,99],"n":2,"cm":[95,23.842105263157894,94.6842105263158],"p":null,"c":[{"b":[21.1875,92.875,27.25,99],"n":2,"cm":[95,23.842105263157894,94.6842105263158],"p":null,"c":[{"b":[24.21875,95.9375,27.25,99],"n":0,"cm":[0,25.734375,97.46875],"p":null,"c":null},{"b":[21.1875,95.9375,24.21875,99],"n":0,"cm":[0,22.703125,97.46875],"p":null,"c":null},{"b":[21.1875,92.875,24.21875,95.9375],"n":2,"cm":[95,23.842105263157894,94.6842105263158],"p":null,"c":[{"b":[22.703125,94.40625,24.21875,95.9375],"n":1,"cm":[80,24,95],"p":[80,24,95,47,90],"c":null},{"b":[21.1875,94.40625,22.703125,95.9375],"n":0,"cm":[0,21.9453125,95.171875],"p":null,"c":null},{"b":[21.1875,92.875,22.703125,94.40625],"n":0,"cm":[0,21.9453125,93.640625],"p":null,"c":null},{"b":[22.703125,92.875,24.21875,94.40625],"n":1,"cm":[15,23,93],"p":[15,23,93,86,60],"c":null}]},{"b":[24.21875,92.875,27.25,95.9375],"n":0,"cm":[0,25.734375,94.40625],"p":null,"c":null}]},{"b":[15.125,92.875,21.1875,99],"n":0,"cm":[0,18.15625,95.9375],"p":null,"c":null},{"b":[15.125,86.75,21.1875,92.875],"n":0,"cm":[0,18.15625,89.8125],"p":null,"c":null},{"b":[21.1875,86.75,27.25,92.875],"n":0,"cm":[0,24.21875,89.8125],"p":null,"c":null}]},{"b":[3,86.75,15.125,99],"n":0,"cm":[0,9.0625,92.875],"p":null,"c":null},{"b":[3,74.5,15.125,86.75],"n":1,"cm":[30,5,86],"p":[30,5,86,59,44],"c":null},{"b":[15.125,74.5,27.25,86.75],"n":0,"cm":[0,21.1875,80.625],"p":null,"c":null}]},{"b":[3,50.0,27.25,74.5],"n":2,"cm":[155,14.64516129032258,58.74193548387097],"p":null,"c":[{"b":[15.125,62.25,27.25,74.5],"n":1,"cm":[95,22,63],"p":[95,22,63,83,41],"c":null},{"b":[3,62.25,15.125,74.5],"n":0,"cm":[0,9.0625,68.375],"p":null,"c":null},{"b":[3,50.0,15.125,62.25],"n":1,"cm":[60,3,52],"p":[60,3,52,20,42],"c":null},{"b":[15.125,50.0,27.25,62.25],"n":0,"cm":[0,21.1875,56.125],"p":null,"c":null}]},{"b":[27.25,50.0,51.5,74.5],"n":3,"cm":[120,42.583333333333336,60.666666666666664],"p":null,"c":[{"b":[39.375,62.25,51.5,74.5],"n":0,"cm":[0,45.4375,68.375],"p":null,"c":null},{"b":[27.25,62.25,39.375,74.5],"n":1,"cm":[55,37,69],"p":[55,37,69,16,74],"c":null},{"b":[27.25,50.0,39.375,62.25],"n":0,"cm":[0,33.3125,56.125],"p":null,"c":null},{"b":[39.375,50.0,51.5,62.25],"n":2,"cm":[65,47.30769230769231,53.61538461538461],"p":null,"c":[{"b":[45.4375,56.125,51.5,62.25],"n":0,"cm":[0,48.46875,59.1875],"p":null,"c":null},{"b":[39.375,56.125,45.4375,62.25],"n":0,"cm":[0,42.40625,59.1875],"p":null,"c":null},{"b":[39.375,50.0,45.4375,56.125],"n":0,"cm":[0,42.40625,53.0625],"p":null,"c":null},{"b":[45.4375,50.0,51.5,56.125],"n":2,"cm":[65,47.30769230769231,53.61538461538461],"p":null,"c":[{"b":[48.46875,53.0625,51.5,56.125],"n":0,"cm":[0,49.984375,54.59375],"p":null,"c":null},{"b":[45.4375,53.0625,48.46875,56.125],"n":1,"cm":[20,48,55],"p":[20,48,55,53,99],"c":null},{"b":[45.4375,50.0,48.46875,53.0625],"n":1,"cm":[45,47,53],"p":[45,47,53,8,59],"c":null},{"b":[48.46875,50.0,51.5,53.0625],"n":0,"cm":[0,49.984375,51.53125],"p":null,"c":null}]}]}]}]},{"b":[3,1,51.5,50.0],"n":5,"cm":[200,13.825,18.25],"p":null,"c":[{"b":[27.25,25.5,51.5,50.0],"n":0,"cm":[0,39.375,37.75],"p":null,"c":null},{"b":[3,25.5,27.25,50.0],"n":2,"cm":[105,9.428571428571429,30.428571428571427],"p":null,"c":[{"b":[15.125,37.75,27.25,50.0],"n":0,"cm":[0,21.1875,43.875],"p":null,"c":null},{"b":[3,37.75,15.125,50.0],"n":0,"cm":[0,9.0625,43.875],"p":null,"c":null},{"b":[3,25.5,15.125,37.75],"n":2,"cm":[105,9.428571428571429,30.428571428571427],"p":null,"c":[{"b":[9.0625,31.625,15.125,37.75],"n":1,"cm":[40,15,36],"p":[40,15,36,71,9],"c":null},{"b":[3,31.625,9.0625,37.75],"n":0,"cm":[0,6.03125,34.6875],"p":null,"c":null},{"b":[3,25.5,9.0625,31.625],"n":1,"cm":[65,6,27],"p":[65,6,27,48,71],"c":null},{"b":[9.0625,25.5,15.125,31.625],"n":0,"cm":[0,12.09375,28.5625],"p":null,"c":null}]},{"b":[15.125,25.5,27.25,37.75],"n":0,"cm":[0,21.1875,31.625],"p":null,"c":null}]},{"b":[3,1,27.25,25.5],"n":2,"cm":[75,10.866666666666667,1.0],"p":null,"c":[{"b":[15.125,13.25,27.25,25.5],"n":0,"cm":[0,21.1875,19.375],"p":null,"c":null},{"b":[3,13.25,15.125,25.5],"n":0,"cm":[0,9.0625,19.375],"p":null,"c":null},{"b":[3,1,15.125,13.25],"n":2,"cm":[75,10.866666666666667,1.0],"p":null,"c":[{"b":[9.0625,7.125,15.125,13.25],"n":0,"cm":[0,12.09375,10.1875],"p":null,"c":null},{"b":[3,7.125,9.0625,13.25],"n":0,"cm":[0,6.03125,10.1875],"p":null,"c":null},{"b":[3,1,9.0625,7.125],"n":1,"cm":[5,9,1],"p":[5,9,1,16,54],"c":null},{"b":[9.0625,1,15.125,7.125],"n":1,"cm":[70,11,1],"p":[70,11,1,90,78],"c":null}]},{"b":[15.125,1,27.25,13.25],"n":0,"cm":[0,21.1875,7.125],"p":null,"c":null}]},{"b":[27.25,1,51.5,25.5],"n":1,"cm":[20,48,19],"p":[20,48,19,16,14],"c":null}]},{"b":[51.5,1,100,50.0],"n":4,"cm":[170,74.88235294117646,14.617647058823529],"p":null,"c":[{"b":[75.75,25.5,100,50.0],"n":0,"cm":[0,87.875,37.75],"p":null,"c":null},{"b":[51.5,25.5,75.75,50.0],"n":0,"cm":[0,63.625,37.75],"p":null,"c":null},{"b":[51.5,1,75.75,25.5],"n":2,"cm":[80,60.0625,22.1875],"p":null,"c":[{"b":[63.625,13.25,75.75,25.5],"n":1,"cm":[15,69,23],"p":[15,69,23,96,88],"c":null},{"b":[51.5,13.25,63.625,25.5],"n":1,"cm":[65,58,22],"p":[65,58,22,6,73],"c":null},{"b":[51.5,1,63.625,13.25],"n":0,"cm":[0,57.5625,7.125],"p":null,"c":null},{"b":[63.625,1,75.75,13.25],"n":0,"cm":[0,69.6875,7.125],"p":null,"c":null}]},{"b":[75.75,1,100,25.5],"n":2,"cm":[90,88.05555555555556,7.888888888888889],"p":null,"c":[{"b":[87.875,13.25,100,25.5],"n":1,"cm":[35,96,14],"p":[35,96,14,74,35],"c":null},{"b":[75.75,13.25,87.875,25.5],"n":0,"cm":[0,81.8125,19.375],"p":null,"c":null},{"b":[75.75,1,87.875,13.25],"n":1,"cm":[55,83,4],"p":[55,83,4,64,19],"c":null},{"b":[87.875,1,100,13.25],"n":0,"cm":[0,93.9375,7.125],"p":null,"c":null}]}]}]}#28',
    ("0:Box(3,1,100,99),4├─ 1:Box(51.5,50.0,100,99),2│  ├─ 2:Box(75.75,74.5,100,99),0│  ├─ 3:Box(51.5,74.5,75.75,99),1│  ├─ 4:Box(51.5,50.0,75.75,74.5),0│  └─ 5:Box(75.75,50.0,100,74.5),1├─ 6:Box(3,50.0,51.5,99),1├─ 7:Box(3,1,51.5,50.0),1└─ 8:Box(51.5,1,100,50.0),00  -> (255, 39.666666666666664, 63.96078431372549)1  -> (105, 70.71428571428571, 82.28571428571429)2  -> (0, 87.875, 86.75)3  -> (75, 63, 86)4  -> (0, 63.625, 62.25)5  -> (30, 90, 73)6  -> (80, 24, 95)7  -> (70, 11, 1)8  -> (0, 75.75, 25.5)",4),
    [(5,9,1,16,54), (55,83,4,64,19), (85,56,76,87,85), (90,54,99,81,61), (80,61,79,16,98), (40,15,36,71,9), (55,37,69,16,74), (20,48,19,16,14), (15,23,93,86,60), (95,78,90,59,67), (20,48,55,53,99), (65,58,22,6,73), (60,89,95,18,6), (95,22,63,83,41), (100,81,93,72,40), (60,3,52,20,42), (95,66,74,18,12), (35,96,14,74,35), (65,6,27,48,71), (90,85,65,39,46), (30,100,84,71,16), (30,5,86,59,44), (15,69,23,96,88), (45,47,53,8,59)]
) 


testMap["GAD", "R"][8] = (
    "09", 1, "R",'{"b":[2,2,100,100],"n":25,"cm":[1320,50.28030303030303,56.821969696969695],"p":null,"c":[{"b":[51.0,51.0,100,100],"n":5,"cm":[185,86.43243243243244,82.8108108108108],"p":null,"c":[{"b":[75.5,75.5,100,100],"n":4,"cm":[130,89.15384615384616,95.0],"p":null,"c":[{"b":[87.75,87.75,100,100],"n":2,"cm":[85,91.70588235294117,97.11764705882354],"p":null,"c":[{"b":[93.875,93.875,100,100],"n":1,"cm":[45,95,99],"p":[45,95,99,20,45],"c":null},{"b":[87.75,93.875,93.875,100],"n":1,"cm":[40,88,95],"p":[40,88,95,14,36],"c":null},{"b":[87.75,87.75,93.875,93.875],"n":0,"cm":[0,90.8125,90.8125],"p":null,"c":null},{"b":[93.875,87.75,100,93.875],"n":0,"cm":[0,96.9375,90.8125],"p":null,"c":null}]},{"b":[75.5,87.75,87.75,100],"n":1,"cm":[25,87,99],"p":[25,87,99,68,76],"c":null},{"b":[75.5,75.5,87.75,87.75],"n":1,"cm":[20,81,81],"p":[20,81,81,25,45],"c":null},{"b":[87.75,75.5,100,87.75],"n":0,"cm":[0,93.875,81.625],"p":null,"c":null}]},{"b":[51.0,75.5,75.5,100],"n":0,"cm":[0,63.25,87.75],"p":null,"c":null},{"b":[51.0,51.0,75.5,75.5],"n":0,"cm":[0,63.25,63.25],"p":null,"c":null},{"b":[75.5,51.0,100,75.5],"n":1,"cm":[55,80,54],"p":[55,80,54,81,55],"c":null}]},{"b":[2,51.0,51.0,100],"n":7,"cm":[525,31.742857142857144,77.3047619047619],"p":null,"c":[{"b":[26.5,75.5,51.0,100],"n":2,"cm":[190,39.5,98.5],"p":null,"c":[{"b":[38.75,87.75,51.0,100],"n":1,"cm":[95,45,97],"p":[95,45,97,40,22],"c":null},{"b":[26.5,87.75,38.75,100],"n":1,"cm":[95,34,100],"p":[95,34,100,76,59],"c":null},{"b":[26.5,75.5,38.75,87.75],"n":0,"cm":[0,32.625,81.625],"p":null,"c":null},{"b":[38.75,75.5,51.0,87.75],"n":0,"cm":[0,44.875,81.625],"p":null,"c":null}]},{"b":[2,75.5,26.5,100],"n":1,"cm":[90,16,85],"p":[90,16,85,12,95],"c":null},{"b":[2,51.0,26.5,75.5],"n":1,"cm":[60,2,53],"p":[60,2,53,18,55],"c":null},{"b":[26.5,51.0,51.0,75.5],"n":3,"cm":[185,41.08108108108108,59.67567567567568],"p":null,"c":[{"b":[38.75,63.25,51.0,75.5],"n":1,"cm":[40,49,66],"p":[40,49,66,47,100],"c":null},{"b":[26.5,63.25,38.75,75.5],"n":0,"cm":[0,32.625,69.375],"p":null,"c":null},{"b":[26.5,51.0,38.75,63.25],"n":1,"cm":[85,36,60],"p":[85,36,60,48,26],"c":null},{"b":[38.75,51.0,51.0,63.25],"n":1,"cm":[60,43,55],"p":[60,43,55,46,72],"c":null}]}]},{"b":[2,2,51.0,51.0],"n":6,"cm":[255,31.333333333333332,31.96078431372549],"p":null,"c":[{"b":[26.5,26.5,51.0,51.0],"n":1,"cm":[35,37,42],"p":[35,37,42,39,7],"c":null},{"b":[2,26.5,26.5,51.0],"n":2,"cm":[95,14.736842105263158,47.10526315789474],"p":null,"c":[{"b":[14.25,38.75,26.5,51.0],"n":1,"cm":[40,24,50],"p":[40,24,50,97,47],"c":null},{"b":[2,38.75,14.25,51.0],"n":1,"cm":[55,8,45],"p":[55,8,45,63,49],"c":null},{"b":[2,26.5,14.25,38.75],"n":0,"cm":[0,8.125,32.625],"p":null,"c":null},{"b":[14.25,26.5,26.5,38.75],"n":0,"cm":[0,20.375,32.625],"p":null,"c":null}]},{"b":[2,2,26.5,26.5],"n":0,"cm":[0,14.25,14.25],"p":null,"c":null},{"b":[26.5,2,51.0,26.5],"n":3,"cm":[125,42.36,17.64],"p":null,"c":[{"b":[38.75,14.25,51.0,26.5],"n":2,"cm":[115,43.17391304347826,19.0],"p":null,"c":[{"b":[44.875,20.375,51.0,26.5],"n":0,"cm":[0,47.9375,23.4375],"p":null,"c":null},{"b":[38.75,20.375,44.875,26.5],"n":0,"cm":[0,41.8125,23.4375],"p":null,"c":null},{"b":[38.75,14.25,44.875,20.375],"n":1,"cm":[35,39,19],"p":[35,39,19,28,57],"c":null},{"b":[44.875,14.25,51.0,20.375],"n":1,"cm":[80,45,19],"p":[80,45,19,19,89],"c":null}]},{"b":[26.5,14.25,38.75,26.5],"n":0,"cm":[0,32.625,20.375],"p":null,"c":null},{"b":[26.5,2,38.75,14.25],"n":1,"cm":[10,33,2],"p":[10,33,2,25,13],"c":null},{"b":[38.75,2,51.0,14.25],"n":0,"cm":[0,44.875,8.125],"p":null,"c":null}]}]},{"b":[51.0,2,100,51.0],"n":7,"cm":[355,72.46478873239437,30.845070422535212],"p":null,"c":[{"b":[75.5,26.5,100,51.0],"n":2,"cm":[85,81.17647058823529,40.294117647058826],"p":null,"c":[{"b":[87.75,38.75,100,51.0],"n":0,"cm":[0,93.875,44.875],"p":null,"c":null},{"b":[75.5,38.75,87.75,51.0],"n":1,"cm":[40,78,44],"p":[40,78,44,35,82],"c":null},{"b":[75.5,26.5,87.75,38.75],"n":1,"cm":[45,84,37],"p":[45,84,37,16,60],"c":null},{"b":[87.75,26.5,100,38.75],"n":0,"cm":[0,93.875,32.625],"p":null,"c":null}]},{"b":[51.0,26.5,75.5,51.0],"n":2,"cm":[105,56.142857142857146,42.904761904761905],"p":null,"c":[{"b":[63.25,38.75,75.5,51.0],"n":0,"cm":[0,69.375,44.875],"p":null,"c":null},{"b":[51.0,38.75,63.25,51.0],"n":2,"cm":[105,56.142857142857146,42.904761904761905],"p":null,"c":[{"b":[57.125,44.875,63.25,51.0],"n":1,"cm":[25,63,49],"p":[25,63,49,99,35],"c":null},{"b":[51.0,44.875,57.125,51.0],"n":0,"cm":[0,54.0625,47.9375],"p":null,"c":null},{"b":[51.0,38.75,57.125,44.875],"n":1,"cm":[80,54,41],"p":[80,54,41,98,34],"c":null},{"b":[57.125,38.75,63.25,44.875],"n":0,"cm":[0,60.1875,41.8125],"p":null,"c":null}]},{"b":[51.0,26.5,63.25,38.75],"n":0,"cm":[0,57.125,32.625],"p":null,"c":null},{"b":[63.25,26.5,75.5,38.75],"n":0,"cm":[0,69.375,32.625],"p":null,"c":null}]},{"b":[51.0,2,75.5,26.5],"n":2,"cm":[90,60.333333333333336,18.555555555555557],"p":null,"c":[{"b":[63.25,14.25,75.5,26.5],"n":0,"cm":[0,69.375,20.375],"p":null,"c":null},{"b":[51.0,14.25,63.25,26.5],"n":1,"cm":[70,59,21],"p":[70,59,21,71,61],"c":null},{"b":[51.0,2,63.25,14.25],"n":0,"cm":[0,57.125,8.125],"p":null,"c":null},{"b":[63.25,2,75.5,14.25],"n":1,"cm":[20,65,10],"p":[20,65,10,36,79],"c":null}]},{"b":[75.5,2,100,26.5],"n":1,"cm":[75,100,18],"p":[75,100,18,88,17],"c":null}]}]}#25',
    ("0:Box(2,2,100,100),9├─ 1:Box(51.0,51.0,100,100),2│  ├─ 2:Box(75.5,75.5,100,100),2│  │  ├─ 3:Box(87.75,87.75,100,100),1│  │  ├─ 4:Box(75.5,87.75,87.75,100),1│  │  ├─ 5:Box(75.5,75.5,87.75,87.75),0│  │  └─ 6:Box(87.75,75.5,100,87.75),0│  ├─ 7:Box(51.0,75.5,75.5,100),0│  ├─ 8:Box(51.0,51.0,75.5,75.5),0│  └─ 9:Box(75.5,51.0,100,75.5),0├─ 10:Box(2,51.0,51.0,100),3│  ├─ 11:Box(26.5,75.5,51.0,100),1│  ├─ 12:Box(2,75.5,26.5,100),0│  ├─ 13:Box(2,51.0,26.5,75.5),1│  └─ 14:Box(26.5,51.0,51.0,75.5),1├─ 15:Box(2,2,51.0,51.0),0└─ 16:Box(51.0,2,100,51.0),4   ├─ 17:Box(75.5,26.5,100,51.0),1   ├─ 18:Box(51.0,26.5,75.5,51.0),1   ├─ 19:Box(51.0,2,75.5,26.5),1   └─ 20:Box(75.5,2,100,26.5),10  -> (515, 57.23300970873787, 57.42718446601942)1  -> (65, 87.61538461538461, 96.53846153846153)2  -> (65, 87.61538461538461, 96.53846153846153)3  -> (40, 88, 95)4  -> (25, 87, 99)5  -> (0, 81.625, 81.625)6  -> (0, 93.875, 81.625)7  -> (0, 63.25, 87.75)8  -> (0, 63.25, 63.25)9  -> (0, 87.75, 63.25)10 -> (240, 31.0625, 72.89583333333333)11 -> (95, 45, 97)12 -> (0, 14.25, 87.75)13 -> (60, 2, 53)14 -> (85, 36, 60)15 -> (0, 26.5, 26.5)16 -> (210, 77.73809523809524, 27.642857142857142)17 -> (40, 78, 44)18 -> (25, 63, 49)19 -> (70, 59, 21)20 -> (75, 100, 18)",9),
    [(20,81,81,25,45), (45,84,37,16,60), (10,33,2,25,13), (90,16,85,12,95), (40,24,50,97,47), (55,80,54,81,55), (35,37,42,39,7), (35,39,19,28,57), (95,34,100,76,59), (20,65,10,36,79), (60,43,55,46,72), (80,54,41,98,34), (80,45,19,19,89), (55,8,45,63,49), (45,95,99,20,45), (40,49,66,47,100)]
) 


testMap["GAD", "R"][9] = (
    "10", 1, "R",'{"b":[6,8,97,90],"n":17,"cm":[995,50.63316582914573,32.62814070351759],"p":null,"c":[{"b":[51.5,49.0,97,90],"n":3,"cm":[160,70.46875,66.59375],"p":null,"c":[{"b":[74.25,69.5,97,90],"n":0,"cm":[0,85.625,79.75],"p":null,"c":null},{"b":[51.5,69.5,74.25,90],"n":1,"cm":[55,65,90],"p":[55,65,90,9,45],"c":null},{"b":[51.5,49.0,74.25,69.5],"n":1,"cm":[70,64,53],"p":[70,64,53,90,29],"c":null},{"b":[74.25,49.0,97,69.5],"n":1,"cm":[35,92,57],"p":[35,92,57,35,23],"c":null}]},{"b":[6,49.0,51.5,90],"n":1,"cm":[10,25,89],"p":[10,25,89,53,53],"c":null},{"b":[6,8,51.5,49.0],"n":7,"cm":[420,19.488095238095237,21.833333333333332],"p":null,"c":[{"b":[28.75,28.5,51.5,49.0],"n":0,"cm":[0,40.125,38.75],"p":null,"c":null},{"b":[6,28.5,28.75,49.0],"n":3,"cm":[135,14.407407407407407,40.03703703703704],"p":null,"c":[{"b":[17.375,38.75,28.75,49.0],"n":1,"cm":[70,18,46],"p":[70,18,46,60,83],"c":null},{"b":[6,38.75,17.375,49.0],"n":1,"cm":[15,9,39],"p":[15,9,39,36,87],"c":null},{"b":[6,28.5,17.375,38.75],"n":1,"cm":[50,11,32],"p":[50,11,32,92,18],"c":null},{"b":[17.375,28.5,28.75,38.75],"n":0,"cm":[0,23.0625,33.625],"p":null,"c":null}]},{"b":[6,8,28.75,28.5],"n":2,"cm":[155,14.129032258064516,15.0],"p":null,"c":[{"b":[17.375,18.25,28.75,28.5],"n":0,"cm":[0,23.0625,23.375],"p":null,"c":null},{"b":[6,18.25,17.375,28.5],"n":0,"cm":[0,11.6875,23.375],"p":null,"c":null},{"b":[6,8,17.375,18.25],"n":1,"cm":[85,6,15],"p":[85,6,15,62,5],"c":null},{"b":[17.375,8,28.75,18.25],"n":1,"cm":[70,24,15],"p":[70,24,15,40,88],"c":null}]},{"b":[28.75,8,51.5,28.5],"n":2,"cm":[130,31.153846153846153,11.076923076923077],"p":null,"c":[{"b":[40.125,18.25,51.5,28.5],"n":0,"cm":[0,45.8125,23.375],"p":null,"c":null},{"b":[28.75,18.25,40.125,28.5],"n":0,"cm":[0,34.4375,23.375],"p":null,"c":null},{"b":[28.75,8,40.125,18.25],"n":2,"cm":[130,31.153846153846153,11.076923076923077],"p":null,"c":[{"b":[34.4375,13.125,40.125,18.25],"n":0,"cm":[0,37.28125,15.6875],"p":null,"c":null},{"b":[28.75,13.125,34.4375,18.25],"n":1,"cm":[50,33,16],"p":[50,33,16,88,6],"c":null},{"b":[28.75,8,34.4375,13.125],"n":1,"cm":[80,30,8],"p":[80,30,8,41,40],"c":null},{"b":[34.4375,8,40.125,13.125],"n":0,"cm":[0,37.28125,10.5625],"p":null,"c":null}]},{"b":[40.125,8,51.5,18.25],"n":0,"cm":[0,45.8125,13.125],"p":null,"c":null}]}]},{"b":[51.5,8,97,49.0],"n":6,"cm":[405,75.72839506172839,29.012345679012345],"p":null,"c":[{"b":[74.25,28.5,97,49.0],"n":2,"cm":[105,96.47619047619048,46.19047619047619],"p":null,"c":[{"b":[85.625,38.75,97,49.0],"n":1,"cm":[100,97,47],"p":[100,97,47,27,84],"c":null},{"b":[74.25,38.75,85.625,49.0],"n":0,"cm":[0,79.9375,43.875],"p":null,"c":null},{"b":[74.25,28.5,85.625,38.75],"n":0,"cm":[0,79.9375,33.625],"p":null,"c":null},{"b":[85.625,28.5,97,38.75],"n":1,"cm":[5,86,30],"p":[5,86,30,81,10],"c":null}]},{"b":[51.5,28.5,74.25,49.0],"n":1,"cm":[75,73,42],"p":[75,73,42,94,34],"c":null},{"b":[51.5,8,74.25,28.5],"n":2,"cm":[180,61.44444444444444,18.333333333333332],"p":null,"c":[{"b":[62.875,18.25,74.25,28.5],"n":0,"cm":[0,68.5625,23.375],"p":null,"c":null},{"b":[51.5,18.25,62.875,28.5],"n":1,"cm":[80,62,20],"p":[80,62,20,83,13],"c":null},{"b":[51.5,8,62.875,18.25],"n":1,"cm":[100,61,17],"p":[100,61,17,24,93],"c":null},{"b":[62.875,8,74.25,18.25],"n":0,"cm":[0,68.5625,13.125],"p":null,"c":null}]},{"b":[74.25,8,97,28.5],"n":1,"cm":[45,89,10],"p":[45,89,10,94,91],"c":null}]}]}#17',
    ("0:Box(6,8,97,90),4├─ 1:Box(51.5,49.0,97,90),1├─ 2:Box(6,49.0,51.5,90),0├─ 3:Box(6,8,51.5,49.0),2│  ├─ 4:Box(28.75,28.5,51.5,49.0),0│  ├─ 5:Box(6,28.5,28.75,49.0),0│  ├─ 6:Box(6,8,28.75,28.5),2│  │  ├─ 7:Box(17.375,18.25,28.75,28.5),0│  │  ├─ 8:Box(6,18.25,17.375,28.5),0│  │  ├─ 9:Box(6,8,17.375,18.25),1│  │  └─ 10:Box(17.375,8,28.75,18.25),1│  └─ 11:Box(28.75,8,51.5,28.5),0└─ 12:Box(51.5,8,97,49.0),10  -> (310, 38.274193548387096, 28.951612903225808)1  -> (55, 65, 90)2  -> (0, 28.75, 69.5)3  -> (155, 14.129032258064516, 15.0)4  -> (0, 40.125, 38.75)5  -> (0, 17.375, 38.75)6  -> (155, 14.129032258064516, 15.0)7  -> (0, 23.0625, 23.375)8  -> (0, 11.6875, 23.375)9  -> (85, 6, 15)10 -> (70, 24, 15)11 -> (0, 40.125, 18.25)12 -> (100, 61, 17)",4),
    [(50,11,32,92,18), (70,64,53,90,29), (75,73,42,94,34), (10,25,89,53,53), (80,62,20,83,13), (45,89,10,94,91), (70,18,46,60,83), (80,30,8,41,40), (100,97,47,27,84), (35,92,57,35,23), (50,33,16,88,6), (5,86,30,81,10), (15,9,39,36,87)]
) 


testMap["GAD", "R"][10] = (
    "11", 1, "R",'{"b":[2,1,95,86],"n":14,"cm":[875,33.90857142857143,49.21714285714286],"p":null,"c":[{"b":[48.5,43.5,95,86],"n":2,"cm":[170,76.91176470588235,56.88235294117647],"p":null,"c":[{"b":[71.75,64.75,95,86],"n":0,"cm":[0,83.375,75.375],"p":null,"c":null},{"b":[48.5,64.75,71.75,86],"n":0,"cm":[0,60.125,75.375],"p":null,"c":null},{"b":[48.5,43.5,71.75,64.75],"n":1,"cm":[75,54,58],"p":[75,54,58,67,80],"c":null},{"b":[71.75,43.5,95,64.75],"n":1,"cm":[95,95,56],"p":[95,95,56,10,38],"c":null}]},{"b":[2,43.5,48.5,86],"n":5,"cm":[295,11.64406779661017,80.50847457627118],"p":null,"c":[{"b":[25.25,64.75,48.5,86],"n":1,"cm":[5,28,77],"p":[5,28,77,40,59],"c":null},{"b":[2,64.75,25.25,86],"n":3,"cm":[245,10.693877551020408,86.0],"p":null,"c":[{"b":[13.625,75.375,25.25,86],"n":1,"cm":[100,17,86],"p":[100,17,86,25,70],"c":null},{"b":[2,75.375,13.625,86],"n":2,"cm":[145,6.344827586206897,86.0],"p":null,"c":[{"b":[7.8125,80.6875,13.625,86],"n":1,"cm":[70,11,86],"p":[70,11,86,88,57],"c":null},{"b":[2,80.6875,7.8125,86],"n":1,"cm":[75,2,86],"p":[75,2,86,57,68],"c":null},{"b":[2,75.375,7.8125,80.6875],"n":0,"cm":[0,4.90625,78.03125],"p":null,"c":null},{"b":[7.8125,75.375,13.625,80.6875],"n":0,"cm":[0,10.71875,78.03125],"p":null,"c":null}]},{"b":[2,64.75,13.625,75.375],"n":0,"cm":[0,7.8125,70.0625],"p":null,"c":null},{"b":[13.625,64.75,25.25,75.375],"n":0,"cm":[0,19.4375,70.0625],"p":null,"c":null}]},{"b":[2,43.5,25.25,64.75],"n":1,"cm":[45,15,51],"p":[45,15,51,49,27],"c":null},{"b":[25.25,43.5,48.5,64.75],"n":0,"cm":[0,36.875,54.125],"p":null,"c":null}]},{"b":[2,1,48.5,43.5],"n":5,"cm":[325,21.56923076923077,27.81538461538462],"p":null,"c":[{"b":[25.25,22.25,48.5,43.5],"n":2,"cm":[145,31.586206896551722,32.48275862068966],"p":null,"c":[{"b":[36.875,32.875,48.5,43.5],"n":0,"cm":[0,42.6875,38.1875],"p":null,"c":null},{"b":[25.25,32.875,36.875,43.5],"n":1,"cm":[50,27,41],"p":[50,27,41,25,76],"c":null},{"b":[25.25,22.25,36.875,32.875],"n":1,"cm":[95,34,28],"p":[95,34,28,5,80],"c":null},{"b":[36.875,22.25,48.5,32.875],"n":0,"cm":[0,42.6875,27.5625],"p":null,"c":null}]},{"b":[2,22.25,25.25,43.5],"n":2,"cm":[90,8.0,37.111111111111114],"p":null,"c":[{"b":[13.625,32.875,25.25,43.5],"n":0,"cm":[0,19.4375,38.1875],"p":null,"c":null},{"b":[2,32.875,13.625,43.5],"n":2,"cm":[90,8.0,37.111111111111114],"p":null,"c":[{"b":[7.8125,38.1875,13.625,43.5],"n":1,"cm":[35,8,42],"p":[35,8,42,32,91],"c":null},{"b":[2,38.1875,7.8125,43.5],"n":0,"cm":[0,4.90625,40.84375],"p":null,"c":null},{"b":[2,32.875,7.8125,38.1875],"n":0,"cm":[0,4.90625,35.53125],"p":null,"c":null},{"b":[7.8125,32.875,13.625,38.1875],"n":1,"cm":[55,8,34],"p":[55,8,34,98,36],"c":null}]},{"b":[2,22.25,13.625,32.875],"n":0,"cm":[0,7.8125,27.5625],"p":null,"c":null},{"b":[13.625,22.25,25.25,32.875],"n":0,"cm":[0,19.4375,27.5625],"p":null,"c":null}]},{"b":[2,1,25.25,22.25],"n":1,"cm":[90,19,11],"p":[90,19,11,96,76],"c":null},{"b":[25.25,1,48.5,22.25],"n":0,"cm":[0,36.875,11.625],"p":null,"c":null}]},{"b":[48.5,1,95,43.5],"n":2,"cm":[85,72.3529411764706,7.117647058823529],"p":null,"c":[{"b":[71.75,22.25,95,43.5],"n":1,"cm":[20,93,27],"p":[20,93,27,32,70],"c":null},{"b":[48.5,22.25,71.75,43.5],"n":0,"cm":[0,60.125,32.875],"p":null,"c":null},{"b":[48.5,1,71.75,22.25],"n":1,"cm":[65,66,1],"p":[65,66,1,35,85],"c":null},{"b":[71.75,1,95,22.25],"n":0,"cm":[0,83.375,11.625],"p":null,"c":null}]}]}#14',
    ("0:Box(2,1,95,86),5├─ 1:Box(48.5,43.5,95,86),2│  ├─ 2:Box(71.75,64.75,95,86),0│  ├─ 3:Box(48.5,64.75,71.75,86),0│  ├─ 4:Box(48.5,43.5,71.75,64.75),1│  └─ 5:Box(71.75,43.5,95,64.75),1├─ 6:Box(2,43.5,48.5,86),1├─ 7:Box(2,1,48.5,43.5),1└─ 8:Box(48.5,1,95,43.5),10  -> (365, 49.19178082191781, 49.465753424657535)1  -> (170, 76.91176470588235, 56.88235294117647)2  -> (0, 83.375, 75.375)3  -> (0, 60.125, 75.375)4  -> (75, 54, 58)5  -> (95, 95, 56)6  -> (75, 2, 86)7  -> (55, 8, 34)8  -> (65, 66, 1)",5),
    [(70,11,86,88,57), (100,17,86,25,70), (45,15,51,49,27), (50,27,41,25,76), (90,19,11,96,76), (95,34,28,5,80), (35,8,42,32,91), (20,93,27,32,70), (5,28,77,40,59)]
) 




######################################################################
# FastSimulation test functions
######################################################################

# nm: test number
# max_mark: max total marks for test
# initl: list of all bodies (as tuples) passed to the constructor of the Fast Simulation
# result: desired output
# args: the value of p

def FSI_test(nm,max_mark,initl,result,args):
    s = fullnames["FSI", "G"]
    sim = FastSimulation([TestBody(*initl[i]) for i in range(len(initl))], args)
    testName = f"[{s}{'.'*(printsize-len(s))}{nm}]"
    out = FSIHelper(sim, len(initl), result, max_mark)
    mark, msg = out
    return (testName,mark,msg,max_mark)

testMap["FSI"] = FSI_test

def FSIHelper(sim, args, result, max_mark):
    store = [None]*args
    _,mes = tryWithTimeout(lambda: sim.run(store))
    if mes != None: return (0, mes)
    mark, msg = markmesFastsim((store, result))
    return mark*max_mark, msg

testMap["FSI", "G"] = [None] * 11

# From Description

testMap["FSI", "G"][0] = (
    "01", 1, get_three_circles(1), 
    '2 [Body(1.0,50.0,0.0,150.0,0.0), Body(2.0,125.0,0.0,0,0)];3 [Body(1.0,50.0,0.0,150.0,0.0), Body(1.0,100.0,0.0,100.0,0.0), Body(1.0,150.0,0.0,50.0,0.0)];3 [Body(1.0,50.0,0.0,150.0,0.0), Body(1.0,100.0,0.0,100.0,0.0), Body(1.0,150.0,0.0,50.0,0.0)];',
    0.01
)

testMap["FSI", "G"][1] = (
    "02", 1, get_three_circles(3), 
    '8 [Body(1.0,150.0,0.0,50.0,0.0), Body(1.0,100.0,0.0,100.0,0.0), Body(1.0,50.0,0.0,150.0,0.0), Body(2.0,47.7668244562803,14.776010333066978,114.72632809267327,96.63265308565366), Body(3.0,41.26678074548391,28.23212366975177,25.49507143503616,147.81745949826905), Body(2.0,95.5336489125606,29.552020666133956,76.48421872844885,64.4217687237691), Body(3.0,82.53356149096783,56.46424733950354,16.996714290024105,98.54497299884602), Body(5.0,131.6003946894074,68.54903500523355,0,0)];8 [Body(1.0,150.0,0.0,50.0,0.0), Body(1.0,100.0,0.0,100.0,0.0), Body(3.0,48.51121630418686,9.850673555377986,0,0), Body(3.0,41.26678074548391,28.23212366975177,25.49507143503616,147.81745949826905), Body(2.0,95.5336489125606,29.552020666133956,76.48421872844885,64.4217687237691), Body(3.0,82.53356149096783,56.46424733950354,16.996714290024105,98.54497299884602), Body(2.0,143.3004733688409,44.328030999200934,38.242109364224426,32.21088436188455), Body(3.0,123.80034223645174,84.6963710092553,8.498357145012053,49.27248649942301)];6 [Body(1.0,150.0,0.0,50.0,0.0), Body(1.0,100.0,0.0,100.0,0.0), Body(8.0,57.55016112176669,21.669054125957146,0,0), Body(3.0,82.53356149096783,56.46424733950354,16.996714290024105,98.54497299884602), Body(2.0,143.3004733688409,44.328030999200934,38.242109364224426,32.21088436188455), Body(3.0,123.80034223645174,84.6963710092553,8.498357145012053,49.27248649942301)];7 [Body(2.0,125.0,0.0,0,0), Body(1.0,50.0,0.0,150.0,0.0), Body(2.0,47.7668244562803,14.776010333066978,114.72632809267327,96.63265308565366), Body(3.0,41.26678074548391,28.23212366975177,25.49507143503616,147.81745949826905), Body(2.0,95.5336489125606,29.552020666133956,76.48421872844885,64.4217687237691), Body(3.0,82.53356149096783,56.46424733950354,16.996714290024105,98.54497299884602), Body(5.0,131.6003946894074,68.54903500523355,0,0)];8 [Body(1.0,150.0,0.0,50.0,0.0), Body(1.0,100.0,0.0,100.0,0.0), Body(3.0,48.51121630418686,9.850673555377986,0,0), Body(3.0,41.26678074548391,28.23212366975177,25.49507143503616,147.81745949826905), Body(2.0,95.5336489125606,29.552020666133956,76.48421872844885,64.4217687237691), Body(3.0,82.53356149096783,56.46424733950354,16.996714290024105,98.54497299884602), Body(2.0,143.3004733688409,44.328030999200934,38.242109364224426,32.21088436188455), Body(3.0,123.80034223645174,84.6963710092553,8.498357145012053,49.27248649942301)];6 [Body(1.0,150.0,0.0,50.0,0.0), Body(1.0,100.0,0.0,100.0,0.0), Body(8.0,57.55016112176669,21.669054125957146,0,0), Body(3.0,82.53356149096783,56.46424733950354,16.996714290024105,98.54497299884602), Body(2.0,143.3004733688409,44.328030999200934,38.242109364224426,32.21088436188455), Body(3.0,123.80034223645174,84.6963710092553,8.498357145012053,49.27248649942301)];7 [Body(2.0,125.0,0.0,0,0), Body(1.0,50.0,0.0,150.0,0.0), Body(2.0,47.7668244562803,14.776010333066978,114.72632809267327,96.63265308565366), Body(3.0,41.26678074548391,28.23212366975177,25.49507143503616,147.81745949826905), Body(2.0,95.5336489125606,29.552020666133956,76.48421872844885,64.4217687237691), Body(3.0,82.53356149096783,56.46424733950354,16.996714290024105,98.54497299884602), Body(5.0,131.6003946894074,68.54903500523355,0,0)];8 [Body(1.0,150.0,0.0,50.0,0.0), Body(1.0,100.0,0.0,100.0,0.0), Body(3.0,48.51121630418686,9.850673555377986,0,0), Body(3.0,41.26678074548391,28.23212366975177,25.49507143503616,147.81745949826905), Body(2.0,95.5336489125606,29.552020666133956,76.48421872844885,64.4217687237691), Body(3.0,82.53356149096783,56.46424733950354,16.996714290024105,98.54497299884602), Body(2.0,143.3004733688409,44.328030999200934,38.242109364224426,32.21088436188455), Body(3.0,123.80034223645174,84.6963710092553,8.498357145012053,49.27248649942301)];5 [Body(2.0,125.0,0.0,0,0), Body(8.0,57.55016112176669,21.669054125957146,0,0), Body(3.0,82.53356149096783,56.46424733950354,16.996714290024105,98.54497299884602), Body(2.0,143.3004733688409,44.328030999200934,38.242109364224426,32.21088436188455), Body(3.0,123.80034223645174,84.6963710092553,8.498357145012053,49.27248649942301)];',
    0.01
)

# Generated
testMap["FSI", "G"][2] = (
"03", 1, [(60,7,91,82,12), (30,83,44,5,96), (5,91,83,18,92), (40,64,69,34,57), (15,6,23,18,80)], 
'4 [Body(30,83,44,5,96), Body(15,6,23,18,80), Body(60,7,91,82,12), Body(45,67.0,70.55555555555556,0,0)];5 [Body(30,83,44,5,96), Body(15,6,23,18,80), Body(60,7,91,82,12), Body(40,64,69,34,57), Body(5,91,83,18,92)];5 [Body(30,83,44,5,96), Body(15,6,23,18,80), Body(60,7,91,82,12), Body(40,64,69,34,57), Body(5,91,83,18,92)];5 [Body(30,83,44,5,96), Body(15,6,23,18,80), Body(60,7,91,82,12), Body(40,64,69,34,57), Body(5,91,83,18,92)];4 [Body(30,83,44,5,96), Body(15,6,23,18,80), Body(60,7,91,82,12), Body(45,67.0,70.55555555555556,0,0)];',
0.01
)


testMap["FSI", "G"][3] = (
"04", 1, [(75,52,34,73,71), (5,53,37,78,33), (50,74,26,69,81), (100,25,1,26,51), (45,81,74,56,80)], 
'5 [Body(50,74,26,69,81), Body(100,25,1,26,51), Body(75,52,34,73,71), Body(5,53,37,78,33), Body(45,81,74,56,80)];5 [Body(50,74,26,69,81), Body(100,25,1,26,51), Body(75,52,34,73,71), Body(5,53,37,78,33), Body(45,81,74,56,80)];4 [Body(50,74,26,69,81), Body(100,25,1,26,51), Body(80,52.0625,34.1875,0,0), Body(45,81,74,56,80)];4 [Body(50,74,26,69,81), Body(100,25,1,26,51), Body(80,52.0625,34.1875,0,0), Body(45,81,74,56,80)];3 [Body(50,74,26,69,81), Body(180,37.02777777777778,15.75,0,0), Body(45,81,74,56,80)];',
0.01
)


testMap["FSI", "G"][4] = (
"05", 1, [(90,91,59,8,87), (10,5,49,71,10), (50,35,94,29,81), (35,72,59,57,26), (60,24,48,8,23), (100,3,35,59,65), (55,91,66,56,32), (5,38,66,39,33), (55,72,97,83,67), (75,32,74,47,86), (20,52,15,22,26), (60,43,20,36,20), (40,25,10,42,29), (10,15,95,56,61), (35,39,46,79,57), (60,80,4,100,55)], 
'10 [Body(60,80,4,100,55), Body(20,52,15,22,26), Body(305,22.081967213114755,33.049180327868854,0,0), Body(5,38,66,39,33), Body(10,15,95,56,61), Body(125,33.2,82.0,0,0), Body(90,91,59,8,87), Body(35,72,59,57,26), Body(55,91,66,56,32), Body(55,72,97,83,67)];11 [Body(80,73.0,6.75,0,0), Body(60,43,20,36,20), Body(40,25,10,42,29), Body(100,3,35,59,65), Body(10,5,49,71,10), Body(60,24,48,8,23), Body(35,39,46,79,57), Body(5,38,66,39,33), Body(10,15,95,56,61), Body(125,33.2,82.0,0,0), Body(235,83.72340425531915,69.53191489361703,0,0)];11 [Body(80,73.0,6.75,0,0), Body(60,43,20,36,20), Body(40,25,10,42,29), Body(170,10.529411764705882,40.411764705882355,0,0), Body(35,39,46,79,57), Body(5,38,66,39,33), Body(10,15,95,56,61), Body(75,32,74,47,86), Body(50,35,94,29,81), Body(180,87.30555555555556,61.138888888888886,0,0), Body(55,72,97,83,67)];13 [Body(60,80,4,100,55), Body(20,52,15,22,26), Body(60,43,20,36,20), Body(40,25,10,42,29), Body(170,10.529411764705882,40.411764705882355,0,0), Body(35,39,46,79,57), Body(5,38,66,39,33), Body(10,15,95,56,61), Body(125,33.2,82.0,0,0), Body(90,91,59,8,87), Body(35,72,59,57,26), Body(55,91,66,56,32), Body(55,72,97,83,67)];13 [Body(60,80,4,100,55), Body(20,52,15,22,26), Body(60,43,20,36,20), Body(40,25,10,42,29), Body(100,3,35,59,65), Body(10,5,49,71,10), Body(60,24,48,8,23), Body(35,39,46,79,57), Body(5,38,66,39,33), Body(10,15,95,56,61), Body(125,33.2,82.0,0,0), Body(180,87.30555555555556,61.138888888888886,0,0), Body(55,72,97,83,67)];11 [Body(80,73.0,6.75,0,0), Body(60,43,20,36,20), Body(40,25,10,42,29), Body(100,3,35,59,65), Body(10,5,49,71,10), Body(60,24,48,8,23), Body(35,39,46,79,57), Body(5,38,66,39,33), Body(10,15,95,56,61), Body(125,33.2,82.0,0,0), Body(235,83.72340425531915,69.53191489361703,0,0)];10 [Body(60,80,4,100,55), Body(20,52,15,22,26), Body(305,22.081967213114755,33.049180327868854,0,0), Body(5,38,66,39,33), Body(10,15,95,56,61), Body(125,33.2,82.0,0,0), Body(90,91,59,8,87), Body(35,72,59,57,26), Body(55,91,66,56,32), Body(55,72,97,83,67)];11 [Body(80,73.0,6.75,0,0), Body(60,43,20,36,20), Body(40,25,10,42,29), Body(170,10.529411764705882,40.411764705882355,0,0), Body(35,39,46,79,57), Body(5,38,66,39,33), Body(10,15,95,56,61), Body(75,32,74,47,86), Body(50,35,94,29,81), Body(180,87.30555555555556,61.138888888888886,0,0), Body(55,72,97,83,67)];7 [Body(80,73.0,6.75,0,0), Body(305,22.081967213114755,33.049180327868854,0,0), Body(5,38,66,39,33), Body(10,15,95,56,61), Body(125,33.2,82.0,0,0), Body(180,87.30555555555556,61.138888888888886,0,0), Body(55,72,97,83,67)];11 [Body(80,73.0,6.75,0,0), Body(60,43,20,36,20), Body(40,25,10,42,29), Body(170,10.529411764705882,40.411764705882355,0,0), Body(35,39,46,79,57), Body(5,38,66,39,33), Body(10,15,95,56,61), Body(75,32,74,47,86), Body(50,35,94,29,81), Body(180,87.30555555555556,61.138888888888886,0,0), Body(55,72,97,83,67)];9 [Body(60,80,4,100,55), Body(20,52,15,22,26), Body(60,43,20,36,20), Body(40,25,10,42,29), Body(170,10.529411764705882,40.411764705882355,0,0), Body(35,39,46,79,57), Body(140,32.07142857142857,82.35714285714286,0,0), Body(180,87.30555555555556,61.138888888888886,0,0), Body(55,72,97,83,67)];11 [Body(60,80,4,100,55), Body(20,52,15,22,26), Body(60,43,20,36,20), Body(40,25,10,42,29), Body(170,10.529411764705882,40.411764705882355,0,0), Body(35,39,46,79,57), Body(5,38,66,39,33), Body(10,15,95,56,61), Body(125,33.2,82.0,0,0), Body(180,87.30555555555556,61.138888888888886,0,0), Body(55,72,97,83,67)];8 [Body(60,80,4,100,55), Body(20,52,15,22,26), Body(60,43,20,36,20), Body(40,25,10,42,29), Body(170,10.529411764705882,40.411764705882355,0,0), Body(35,39,46,79,57), Body(140,32.07142857142857,82.35714285714286,0,0), Body(235,83.72340425531915,69.53191489361703,0,0)];10 [Body(80,73.0,6.75,0,0), Body(60,43,20,36,20), Body(40,25,10,42,29), Body(170,10.529411764705882,40.411764705882355,0,0), Body(35,39,46,79,57), Body(5,38,66,39,33), Body(10,15,95,56,61), Body(75,32,74,47,86), Body(50,35,94,29,81), Body(235,83.72340425531915,69.53191489361703,0,0)];13 [Body(60,80,4,100,55), Body(20,52,15,22,26), Body(60,43,20,36,20), Body(40,25,10,42,29), Body(100,3,35,59,65), Body(10,5,49,71,10), Body(60,24,48,8,23), Body(35,39,46,79,57), Body(5,38,66,39,33), Body(10,15,95,56,61), Body(125,33.2,82.0,0,0), Body(180,87.30555555555556,61.138888888888886,0,0), Body(55,72,97,83,67)];9 [Body(60,80,4,100,55), Body(20,52,15,22,26), Body(60,43,20,36,20), Body(40,25,10,42,29), Body(170,10.529411764705882,40.411764705882355,0,0), Body(35,39,46,79,57), Body(140,32.07142857142857,82.35714285714286,0,0), Body(180,87.30555555555556,61.138888888888886,0,0), Body(55,72,97,83,67)];',
0.01
)


testMap["FSI", "G"][5] = (
"06", 1, [(30,30,13,56,100), (80,45,23,9,95), (25,56,96,80,39), (100,30,8,49,81), (75,91,46,96,78), (5,71,82,89,99), (20,71,39,40,73), (15,96,2,72,50)], 
'8 [Body(15,96,2,72,50), Body(20,71,39,40,73), Body(75,91,46,96,78), Body(100,30,8,49,81), Body(30,30,13,56,100), Body(80,45,23,9,95), Body(25,56,96,80,39), Body(5,71,82,89,99)];7 [Body(15,96,2,72,50), Body(20,71,39,40,73), Body(75,91,46,96,78), Body(130,30.0,9.153846153846153,0,0), Body(80,45,23,9,95), Body(25,56,96,80,39), Body(5,71,82,89,99)];6 [Body(15,96,2,72,50), Body(20,71,39,40,73), Body(75,91,46,96,78), Body(210,35.714285714285715,14.428571428571429,0,0), Body(25,56,96,80,39), Body(5,71,82,89,99)];8 [Body(15,96,2,72,50), Body(20,71,39,40,73), Body(75,91,46,96,78), Body(100,30,8,49,81), Body(30,30,13,56,100), Body(80,45,23,9,95), Body(25,56,96,80,39), Body(5,71,82,89,99)];6 [Body(15,96,2,72,50), Body(20,71,39,40,73), Body(75,91,46,96,78), Body(210,35.714285714285715,14.428571428571429,0,0), Body(25,56,96,80,39), Body(5,71,82,89,99)];6 [Body(15,96,2,72,50), Body(20,71,39,40,73), Body(75,91,46,96,78), Body(210,35.714285714285715,14.428571428571429,0,0), Body(25,56,96,80,39), Body(5,71,82,89,99)];6 [Body(15,96,2,72,50), Body(20,71,39,40,73), Body(75,91,46,96,78), Body(210,35.714285714285715,14.428571428571429,0,0), Body(25,56,96,80,39), Body(5,71,82,89,99)];6 [Body(15,96,2,72,50), Body(20,71,39,40,73), Body(75,91,46,96,78), Body(210,35.714285714285715,14.428571428571429,0,0), Body(25,56,96,80,39), Body(5,71,82,89,99)];',
0.01
)


testMap["FSI", "G"][6] = (
"07", 1, [(5,27,77,33,75), (35,1,73,86,90), (15,24,25,24,94), (35,35,100,32,77), (30,41,56,15,50), (40,29,87,52,27), (95,26,34,75,51), (90,28,92,73,97)], 
'7 [Body(110,25.727272727272727,32.77272727272727,0,0), Body(30,41,56,15,50), Body(35,1,73,86,90), Body(5,27,77,33,75), Body(40,29,87,52,27), Body(90,28,92,73,97), Body(35,35,100,32,77)];6 [Body(110,25.727272727272727,32.77272727272727,0,0), Body(30,41,56,15,50), Body(35,1,73,86,90), Body(5,27,77,33,75), Body(130,28.307692307692307,90.46153846153847,0,0), Body(35,35,100,32,77)];5 [Body(15,24,25,24,94), Body(95,26,34,75,51), Body(30,41,56,15,50), Body(35,1,73,86,90), Body(170,29.647058823529413,92.02941176470588,0,0)];6 [Body(140,29.0,37.75,0,0), Body(35,1,73,86,90), Body(5,27,77,33,75), Body(40,29,87,52,27), Body(90,28,92,73,97), Body(35,35,100,32,77)];6 [Body(110,25.727272727272727,32.77272727272727,0,0), Body(30,41,56,15,50), Body(35,1,73,86,90), Body(5,27,77,33,75), Body(130,28.307692307692307,90.46153846153847,0,0), Body(35,35,100,32,77)];7 [Body(110,25.727272727272727,32.77272727272727,0,0), Body(30,41,56,15,50), Body(35,1,73,86,90), Body(5,27,77,33,75), Body(40,29,87,52,27), Body(90,28,92,73,97), Body(35,35,100,32,77)];5 [Body(15,24,25,24,94), Body(95,26,34,75,51), Body(30,41,56,15,50), Body(35,1,73,86,90), Body(170,29.647058823529413,92.02941176470588,0,0)];6 [Body(140,29.0,37.75,0,0), Body(35,1,73,86,90), Body(5,27,77,33,75), Body(40,29,87,52,27), Body(90,28,92,73,97), Body(35,35,100,32,77)];',
0.01
)


testMap["FSI", "G"][7] = (
"08", 1, [(20,64,26,76,92), (50,47,51,54,80), (65,12,41,35,63), (50,75,27,80,18), (45,48,46,29,83), (90,68,92,83,77), (40,21,46,37,12), (15,74,5,41,13), (65,46,52,76,60), (40,57,52,52,57)], 
'8 [Body(15,74,5,41,13), Body(20,64,26,76,92), Body(45,48,46,29,83), Body(50,75,27,80,18), Body(105,15.428571428571429,42.904761904761905,0,0), Body(40,57,52,52,57), Body(115,46.43478260869565,51.56521739130435,0,0), Body(90,68,92,83,77)];8 [Body(35,68.28571428571429,17.0,0,0), Body(45,48,46,29,83), Body(50,75,27,80,18), Body(105,15.428571428571429,42.904761904761905,0,0), Body(40,57,52,52,57), Body(50,47,51,54,80), Body(65,46,52,76,60), Body(90,68,92,83,77)];7 [Body(35,68.28571428571429,17.0,0,0), Body(45,48,46,29,83), Body(50,75,27,80,18), Body(65,12,41,35,63), Body(40,21,46,37,12), Body(155,49.16129032258065,51.67741935483871,0,0), Body(90,68,92,83,77)];7 [Body(15,74,5,41,13), Body(20,64,26,76,92), Body(45,48,46,29,83), Body(50,75,27,80,18), Body(105,15.428571428571429,42.904761904761905,0,0), Body(155,49.16129032258065,51.67741935483871,0,0), Body(90,68,92,83,77)];8 [Body(35,68.28571428571429,17.0,0,0), Body(45,48,46,29,83), Body(50,75,27,80,18), Body(105,15.428571428571429,42.904761904761905,0,0), Body(40,57,52,52,57), Body(50,47,51,54,80), Body(65,46,52,76,60), Body(90,68,92,83,77)];6 [Body(35,68.28571428571429,17.0,0,0), Body(45,48,46,29,83), Body(50,75,27,80,18), Body(105,15.428571428571429,42.904761904761905,0,0), Body(155,49.16129032258065,51.67741935483871,0,0), Body(90,68,92,83,77)];8 [Body(35,68.28571428571429,17.0,0,0), Body(45,48,46,29,83), Body(50,75,27,80,18), Body(65,12,41,35,63), Body(40,21,46,37,12), Body(40,57,52,52,57), Body(115,46.43478260869565,51.56521739130435,0,0), Body(90,68,92,83,77)];6 [Body(15,74,5,41,13), Body(20,64,26,76,92), Body(45,48,46,29,83), Body(50,75,27,80,18), Body(105,15.428571428571429,42.904761904761905,0,0), Body(245,56.08163265306123,66.48979591836735,0,0)];8 [Body(35,68.28571428571429,17.0,0,0), Body(45,48,46,29,83), Body(50,75,27,80,18), Body(105,15.428571428571429,42.904761904761905,0,0), Body(40,57,52,52,57), Body(50,47,51,54,80), Body(65,46,52,76,60), Body(90,68,92,83,77)];7 [Body(35,68.28571428571429,17.0,0,0), Body(45,48,46,29,83), Body(50,75,27,80,18), Body(105,15.428571428571429,42.904761904761905,0,0), Body(40,57,52,52,57), Body(115,46.43478260869565,51.56521739130435,0,0), Body(90,68,92,83,77)];',
0.01
)


testMap["FSI", "G"][8] = (
"09", 1, [(90,35,65,25,16), (45,100,67,76,68), (65,22,31,83,56)], 
'3 [Body(65,22,31,83,56), Body(90,35,65,25,16), Body(45,100,67,76,68)];3 [Body(65,22,31,83,56), Body(90,35,65,25,16), Body(45,100,67,76,68)];3 [Body(65,22,31,83,56), Body(90,35,65,25,16), Body(45,100,67,76,68)];',
0.01
)


testMap["FSI", "G"][9] = (
"10", 1, [(5,57,39,76,84), (80,18,4,13,87), (80,57,33,49,80), (35,67,23,59,14), (95,26,14,46,95), (75,5,6,55,69), (25,88,25,24,20), (35,15,6,82,63), (95,37,81,99,70), (60,18,93,81,84), (40,8,51,52,78), (40,51,70,43,41), (85,51,31,42,13), (95,64,81,56,26), (75,2,40,5,11), (35,56,13,31,45), (75,21,32,32,35), (15,89,61,24,87), (90,17,46,48,15)], 
'15 [Body(25,88,25,24,20), Body(35,56,13,31,45), Body(35,67,23,59,14), Body(80,57,33,49,80), Body(85,51,31,42,13), Body(5,57,39,76,84), Body(95,26,14,46,95), Body(190,12.31578947368421,5.157894736842105,0,0), Body(240,13.5625,39.75,0,0), Body(40,8,51,52,78), Body(60,18,93,81,84), Body(95,37,81,99,70), Body(15,89,61,24,87), Body(40,51,70,43,41), Body(95,64,81,56,26)];10 [Body(25,88,25,24,20), Body(70,61.5,18.0,0,0), Body(170,54.0,32.1764705882353,0,0), Body(95,26,14,46,95), Body(80,18,4,13,87), Body(35,15,6,82,63), Body(75,5,6,55,69), Body(240,13.5625,39.75,0,0), Body(195,25.205128205128204,78.53846153846153,0,0), Body(150,63.03333333333333,76.06666666666666,0,0)];15 [Body(25,88,25,24,20), Body(35,56,13,31,45), Body(35,67,23,59,14), Body(80,57,33,49,80), Body(85,51,31,42,13), Body(5,57,39,76,84), Body(95,26,14,46,95), Body(190,12.31578947368421,5.157894736842105,0,0), Body(240,13.5625,39.75,0,0), Body(40,8,51,52,78), Body(60,18,93,81,84), Body(95,37,81,99,70), Body(15,89,61,24,87), Body(40,51,70,43,41), Body(95,64,81,56,26)];13 [Body(25,88,25,24,20), Body(35,56,13,31,45), Body(35,67,23,59,14), Body(80,57,33,49,80), Body(85,51,31,42,13), Body(5,57,39,76,84), Body(95,26,14,46,95), Body(190,12.31578947368421,5.157894736842105,0,0), Body(240,13.5625,39.75,0,0), Body(195,25.205128205128204,78.53846153846153,0,0), Body(15,89,61,24,87), Body(40,51,70,43,41), Body(95,64,81,56,26)];11 [Body(25,88,25,24,20), Body(70,61.5,18.0,0,0), Body(170,54.0,32.1764705882353,0,0), Body(95,26,14,46,95), Body(115,17.08695652173913,4.608695652173913,0,0), Body(75,5,6,55,69), Body(75,21,32,32,35), Body(75,2,40,5,11), Body(90,17,46,48,15), Body(195,25.205128205128204,78.53846153846153,0,0), Body(150,63.03333333333333,76.06666666666666,0,0)];9 [Body(25,88,25,24,20), Body(70,61.5,18.0,0,0), Body(170,54.0,32.1764705882353,0,0), Body(95,26,14,46,95), Body(115,17.08695652173913,4.608695652173913,0,0), Body(75,5,6,55,69), Body(240,13.5625,39.75,0,0), Body(195,25.205128205128204,78.53846153846153,0,0), Body(150,63.03333333333333,76.06666666666666,0,0)];9 [Body(25,88,25,24,20), Body(35,56,13,31,45), Body(35,67,23,59,14), Body(170,54.0,32.1764705882353,0,0), Body(525,15.361904761904762,22.571428571428573,0,0), Body(195,25.205128205128204,78.53846153846153,0,0), Body(15,89,61,24,87), Body(40,51,70,43,41), Body(95,64,81,56,26)];10 [Body(25,88,25,24,20), Body(70,61.5,18.0,0,0), Body(170,54.0,32.1764705882353,0,0), Body(95,26,14,46,95), Body(80,18,4,13,87), Body(35,15,6,82,63), Body(75,5,6,55,69), Body(240,13.5625,39.75,0,0), Body(195,25.205128205128204,78.53846153846153,0,0), Body(150,63.03333333333333,76.06666666666666,0,0)];12 [Body(25,88,25,24,20), Body(70,61.5,18.0,0,0), Body(170,54.0,32.1764705882353,0,0), Body(95,26,14,46,95), Body(190,12.31578947368421,5.157894736842105,0,0), Body(240,13.5625,39.75,0,0), Body(40,8,51,52,78), Body(60,18,93,81,84), Body(95,37,81,99,70), Body(15,89,61,24,87), Body(40,51,70,43,41), Body(95,64,81,56,26)];8 [Body(265,59.18867924528302,27.75471698113208,0,0), Body(525,15.361904761904762,22.571428571428573,0,0), Body(40,8,51,52,78), Body(60,18,93,81,84), Body(95,37,81,99,70), Body(15,89,61,24,87), Body(40,51,70,43,41), Body(95,64,81,56,26)];14 [Body(25,88,25,24,20), Body(70,61.5,18.0,0,0), Body(170,54.0,32.1764705882353,0,0), Body(95,26,14,46,95), Body(190,12.31578947368421,5.157894736842105,0,0), Body(75,21,32,32,35), Body(75,2,40,5,11), Body(90,17,46,48,15), Body(40,8,51,52,78), Body(60,18,93,81,84), Body(95,37,81,99,70), Body(15,89,61,24,87), Body(40,51,70,43,41), Body(95,64,81,56,26)];12 [Body(25,88,25,24,20), Body(70,61.5,18.0,0,0), Body(170,54.0,32.1764705882353,0,0), Body(95,26,14,46,95), Body(190,12.31578947368421,5.157894736842105,0,0), Body(240,13.5625,39.75,0,0), Body(40,8,51,52,78), Body(60,18,93,81,84), Body(95,37,81,99,70), Body(15,89,61,24,87), Body(40,51,70,43,41), Body(95,64,81,56,26)];15 [Body(25,88,25,24,20), Body(35,56,13,31,45), Body(35,67,23,59,14), Body(80,57,33,49,80), Body(85,51,31,42,13), Body(5,57,39,76,84), Body(95,26,14,46,95), Body(190,12.31578947368421,5.157894736842105,0,0), Body(240,13.5625,39.75,0,0), Body(40,8,51,52,78), Body(60,18,93,81,84), Body(95,37,81,99,70), Body(15,89,61,24,87), Body(40,51,70,43,41), Body(95,64,81,56,26)];10 [Body(25,88,25,24,20), Body(70,61.5,18.0,0,0), Body(170,54.0,32.1764705882353,0,0), Body(525,15.361904761904762,22.571428571428573,0,0), Body(40,8,51,52,78), Body(60,18,93,81,84), Body(95,37,81,99,70), Body(15,89,61,24,87), Body(40,51,70,43,41), Body(95,64,81,56,26)];12 [Body(25,88,25,24,20), Body(70,61.5,18.0,0,0), Body(170,54.0,32.1764705882353,0,0), Body(95,26,14,46,95), Body(190,12.31578947368421,5.157894736842105,0,0), Body(75,21,32,32,35), Body(75,2,40,5,11), Body(90,17,46,48,15), Body(40,8,51,52,78), Body(60,18,93,81,84), Body(95,37,81,99,70), Body(150,63.03333333333333,76.06666666666666,0,0)];13 [Body(25,88,25,24,20), Body(35,56,13,31,45), Body(35,67,23,59,14), Body(80,57,33,49,80), Body(85,51,31,42,13), Body(5,57,39,76,84), Body(95,26,14,46,95), Body(190,12.31578947368421,5.157894736842105,0,0), Body(240,13.5625,39.75,0,0), Body(195,25.205128205128204,78.53846153846153,0,0), Body(15,89,61,24,87), Body(40,51,70,43,41), Body(95,64,81,56,26)];15 [Body(25,88,25,24,20), Body(70,61.5,18.0,0,0), Body(170,54.0,32.1764705882353,0,0), Body(95,26,14,46,95), Body(115,17.08695652173913,4.608695652173913,0,0), Body(75,5,6,55,69), Body(75,21,32,32,35), Body(75,2,40,5,11), Body(90,17,46,48,15), Body(40,8,51,52,78), Body(60,18,93,81,84), Body(95,37,81,99,70), Body(15,89,61,24,87), Body(40,51,70,43,41), Body(95,64,81,56,26)];8 [Body(25,88,25,24,20), Body(70,61.5,18.0,0,0), Body(170,54.0,32.1764705882353,0,0), Body(525,15.361904761904762,22.571428571428573,0,0), Body(195,25.205128205128204,78.53846153846153,0,0), Body(15,89,61,24,87), Body(40,51,70,43,41), Body(95,64,81,56,26)];14 [Body(25,88,25,24,20), Body(70,61.5,18.0,0,0), Body(170,54.0,32.1764705882353,0,0), Body(95,26,14,46,95), Body(190,12.31578947368421,5.157894736842105,0,0), Body(75,21,32,32,35), Body(75,2,40,5,11), Body(90,17,46,48,15), Body(40,8,51,52,78), Body(60,18,93,81,84), Body(95,37,81,99,70), Body(15,89,61,24,87), Body(40,51,70,43,41), Body(95,64,81,56,26)];',
0.01
)


testMap["FSI", "G"][10] = (
"11", 1, [(70,62,34,15,51), (65,82,37,54,6), (50,18,29,66,93), (5,12,70,86,5), (100,82,59,37,25), (100,83,9,43,51), (45,5,52,83,22), (25,25,84,5,88), (85,21,34,73,95), (90,55,38,33,57), (35,39,42,96,46), (45,50,62,6,29), (90,71,7,23,51), (5,45,82,39,27), (75,72,47,8,84), (20,4,22,78,58)], 
'12 [Body(190,77.3157894736842,8.052631578947368,0,0), Body(70,62,34,15,51), Body(90,55,38,33,57), Body(65,82,37,54,6), Body(20,4,22,78,58), Body(135,19.88888888888889,32.148148148148145,0,0), Body(35,39,42,96,46), Body(75,12.133333333333333,63.86666666666667,0,0), Body(75,72,47,8,84), Body(100,82,59,37,25), Body(45,50,62,6,29), Body(5,45,82,39,27)];10 [Body(190,77.3157894736842,8.052631578947368,0,0), Body(70,62,34,15,51), Body(90,55,38,33,57), Body(65,82,37,54,6), Body(190,21.736842105263158,32.89473684210526,0,0), Body(75,12.133333333333333,63.86666666666667,0,0), Body(75,72,47,8,84), Body(100,82,59,37,25), Body(45,50,62,6,29), Body(5,45,82,39,27)];11 [Body(190,77.3157894736842,8.052631578947368,0,0), Body(160,58.0625,36.25,0,0), Body(65,82,37,54,6), Body(20,4,22,78,58), Body(50,18,29,66,93), Body(85,21,34,73,95), Body(35,39,42,96,46), Body(45,5,52,83,22), Body(5,12,70,86,5), Body(25,25,84,5,88), Body(225,71.44444444444444,56.111111111111114,0,0)];8 [Body(415,70.62650602409639,23.457831325301203,0,0), Body(20,4,22,78,58), Body(135,19.88888888888889,32.148148148148145,0,0), Body(35,39,42,96,46), Body(45,5,52,83,22), Body(5,12,70,86,5), Body(25,25,84,5,88), Body(225,71.44444444444444,56.111111111111114,0,0)];9 [Body(190,77.3157894736842,8.052631578947368,0,0), Body(160,58.0625,36.25,0,0), Body(65,82,37,54,6), Body(190,21.736842105263158,32.89473684210526,0,0), Body(75,12.133333333333333,63.86666666666667,0,0), Body(75,72,47,8,84), Body(100,82,59,37,25), Body(45,50,62,6,29), Body(5,45,82,39,27)];9 [Body(100,83,9,43,51), Body(90,71,7,23,51), Body(160,58.0625,36.25,0,0), Body(65,82,37,54,6), Body(190,21.736842105263158,32.89473684210526,0,0), Body(75,12.133333333333333,63.86666666666667,0,0), Body(175,77.71428571428571,53.857142857142854,0,0), Body(45,50,62,6,29), Body(5,45,82,39,27)];8 [Body(415,70.62650602409639,23.457831325301203,0,0), Body(20,4,22,78,58), Body(135,19.88888888888889,32.148148148148145,0,0), Body(35,39,42,96,46), Body(45,5,52,83,22), Body(5,12,70,86,5), Body(25,25,84,5,88), Body(225,71.44444444444444,56.111111111111114,0,0)];10 [Body(415,70.62650602409639,23.457831325301203,0,0), Body(20,4,22,78,58), Body(135,19.88888888888889,32.148148148148145,0,0), Body(35,39,42,96,46), Body(45,5,52,83,22), Body(5,12,70,86,5), Body(25,25,84,5,88), Body(175,77.71428571428571,53.857142857142854,0,0), Body(45,50,62,6,29), Body(5,45,82,39,27)];13 [Body(190,77.3157894736842,8.052631578947368,0,0), Body(160,58.0625,36.25,0,0), Body(65,82,37,54,6), Body(20,4,22,78,58), Body(50,18,29,66,93), Body(85,21,34,73,95), Body(35,39,42,96,46), Body(45,5,52,83,22), Body(5,12,70,86,5), Body(25,25,84,5,88), Body(175,77.71428571428571,53.857142857142854,0,0), Body(45,50,62,6,29), Body(5,45,82,39,27)];14 [Body(190,77.3157894736842,8.052631578947368,0,0), Body(70,62,34,15,51), Body(90,55,38,33,57), Body(65,82,37,54,6), Body(20,4,22,78,58), Body(135,19.88888888888889,32.148148148148145,0,0), Body(35,39,42,96,46), Body(45,5,52,83,22), Body(5,12,70,86,5), Body(25,25,84,5,88), Body(75,72,47,8,84), Body(100,82,59,37,25), Body(45,50,62,6,29), Body(5,45,82,39,27)];13 [Body(190,77.3157894736842,8.052631578947368,0,0), Body(70,62,34,15,51), Body(90,55,38,33,57), Body(65,82,37,54,6), Body(20,4,22,78,58), Body(135,19.88888888888889,32.148148148148145,0,0), Body(35,39,42,96,46), Body(45,5,52,83,22), Body(5,12,70,86,5), Body(25,25,84,5,88), Body(175,77.71428571428571,53.857142857142854,0,0), Body(45,50,62,6,29), Body(5,45,82,39,27)];13 [Body(190,77.3157894736842,8.052631578947368,0,0), Body(70,62,34,15,51), Body(90,55,38,33,57), Body(65,82,37,54,6), Body(20,4,22,78,58), Body(135,19.88888888888889,32.148148148148145,0,0), Body(35,39,42,96,46), Body(45,5,52,83,22), Body(5,12,70,86,5), Body(25,25,84,5,88), Body(175,77.71428571428571,53.857142857142854,0,0), Body(45,50,62,6,29), Body(5,45,82,39,27)];11 [Body(100,83,9,43,51), Body(90,71,7,23,51), Body(160,58.0625,36.25,0,0), Body(65,82,37,54,6), Body(20,4,22,78,58), Body(135,19.88888888888889,32.148148148148145,0,0), Body(35,39,42,96,46), Body(75,12.133333333333333,63.86666666666667,0,0), Body(175,77.71428571428571,53.857142857142854,0,0), Body(45,50,62,6,29), Body(5,45,82,39,27)];10 [Body(415,70.62650602409639,23.457831325301203,0,0), Body(20,4,22,78,58), Body(135,19.88888888888889,32.148148148148145,0,0), Body(35,39,42,96,46), Body(45,5,52,83,22), Body(5,12,70,86,5), Body(25,25,84,5,88), Body(175,77.71428571428571,53.857142857142854,0,0), Body(45,50,62,6,29), Body(5,45,82,39,27)];12 [Body(190,77.3157894736842,8.052631578947368,0,0), Body(70,62,34,15,51), Body(90,55,38,33,57), Body(65,82,37,54,6), Body(20,4,22,78,58), Body(135,19.88888888888889,32.148148148148145,0,0), Body(35,39,42,96,46), Body(75,12.133333333333333,63.86666666666667,0,0), Body(75,72,47,8,84), Body(100,82,59,37,25), Body(45,50,62,6,29), Body(5,45,82,39,27)];8 [Body(415,70.62650602409639,23.457831325301203,0,0), Body(20,4,22,78,58), Body(135,19.88888888888889,32.148148148148145,0,0), Body(35,39,42,96,46), Body(45,5,52,83,22), Body(5,12,70,86,5), Body(25,25,84,5,88), Body(225,71.44444444444444,56.111111111111114,0,0)];',
0.01
)




######################################################################
# Gather all tests 
######################################################################

def runAllTests():
    vs  = [0,0,""]
    n = 0
    ranges = [("SIM", "C"), ("BOD", "T"), ("GAD", "A"), ("GAD", "R"), ("FSI", "G")]
    for x,y in ranges:
        f, tests = testMap[x], testMap[x,y]
        v = runTests(list(zip(range(n,n+len(tests)),[f]*len(tests),tests)))    
        for i in range(3): vs[i] += v[i]
        s = fullnames[x,y]
        testMap[x,y] = (tests, v, (f"\n -> {s}{'.'*(printsize-len(s))} : {v[0]} [{v[1]}]"))
        n+=len(tests)
    
    results =  f"Testing ...\n{vs[2]}"
    for x,y in ranges:
        results += testMap[x,y][2]
    results += f"\n-> *Total: {round(vs[0],2)} [{vs[1]}]"
    print(results)

######################################################################
# main test script (full, marks add up to ??)
######################################################################

# import student code
module = "nBodyProblem" # you solutions file here!
sid = 0
totalmarks = 55
fail = ""
signal.alarm(timeout)    
try:
    mod = importlib.import_module(module.split(".")[0])
    Body = mod.Body
    Simulation = mod.Simulation
    Gadget = mod.Gadget
    FastSimulation = mod.FastSimulation
    print("Imported and will test")
except Exception as err: 
    fail = f"Import Error: {err}\nTotal testing marks [{totalmarks}]: 0"
    print(fail)
signal.alarm(0)
    
if fail != "":
    res = (sid,0,0,0,0,0,fail)
else:
    res = runAllTests()

ModuleNotFoundError: No module named 'import_ipynb'